In [ ]:
import sys
sys.version

'3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0]'

In [ ]:
import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

In [ ]:
from huggingface_hub import login

# Hugging Face API 토큰 입력
login(token="")

# from huggingface_hub import whoami
# print(whoami())

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
import requests
import time
import ast
import re
import os
import json
import random
import sklearn
# from konlpy.tag import Okt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
# 지표를 하나만 설정할 경우
from sklearn.model_selection import cross_val_score
# 지표를 하나 이상 설정할 경우
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold


# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score, roc_curve

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


# 차원축소
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# 군집화
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift
from sklearn.cluster import estimate_bandwidth

# 시간 측정을 위한 시간 모듈
import datetime
from tqdm import tqdm

# 형태소 벡터를 생성하기 위한 라이브러리
from sklearn.feature_extraction.text import CountVectorizer

# 형태소 벡터를 학습 벡터로 변환한다.
from sklearn.feature_extraction.text import TfidfTransformer

# 한국어 형태소 분석
# from konlpy.tag import Okt, Hannanum, Kkma, Mecab, Komoran

# 저장
import pickle

# torch
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig, AutoModel
from torch.optim import AdamW
from torch.utils.data import WeightedRandomSampler
from transformers import get_scheduler
import argparse
from transformers import BertForMaskedLM



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
rs_sample = pd.read_csv('/content/drive/My Drive/25.1H Thesis/just_data_250423f.csv')

In [ ]:
result_dir = f"/content/drive/My Drive/cdf/"

In [ ]:
# 모델
# roberta-small
# roberta-base
# monologg/distilkobert
# lighthouse/mdeberta-v3-base-kor-further

tokenizer = AutoTokenizer.from_pretrained("lighthouse/mdeberta-v3-base-kor-further")
model_class = AutoModelForSequenceClassification

model_name = "lighthouse/mdeberta-v3-base-kor-further"

tokenizer_config.json:   0%|          | 0.00/424 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

In [ ]:

safe_model_name = model_name.split('/')[-1] if '/' in model_name else model_name
method = "pure_grd"

class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.texts = dataframe['txt'].tolist()
        self.labels = dataframe['target'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': tokenized['input_ids'].squeeze(0),  # 차원 축소 유지
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)  # 데이터 타입 명확히 설정
        }


def data_preparing(data_src, tokenizer, batch_size=16, shuffle=True, return_dataset=False):
    dataset = TextDataset(data_src, tokenizer)

    if return_dataset:
        return dataset

    dataloader = DataLoader(
        dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )
    return dataloader


def eval_model(
    model,
    eval_dataloader,
    device,
    num_cls=2,
    data_source=None,
    phase=None,
    wrong_predictions_list=None,
    result_file=None  # 추가
):
    total_loss = 0.0
    total_count = 0
    all_preds, all_labels, all_probs = [], [], []

    model.eval()
    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss
            total_loss += loss.item() * batch['labels'].size(0)
            total_count += batch['labels'].size(0)

            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = torch.argmax(logits, dim=-1)
            labels = batch['labels']

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = total_loss / total_count
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs) if not np.all(all_probs == all_probs[0]) else 0.0

    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    tpr_at_fpr_0001 = np.interp(0.01 / 100, fpr, tpr) if len(fpr) > 0 else 0.0

    print(f"Loss: {avg_loss:.4f} | Acc: {accuracy:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | AUROC: {auc:.4f} | TPR@FPR=0.01%: {tpr_at_fpr_0001:.4f}")

    if data_source is not None and 'txt' in data_source:
        txt_data = [text for text in data_source['txt']]
    else:
        txt_data = []

    # 결과 저장
    if result_file is not None:
        df_result = pd.DataFrame({
            'label': all_labels,
            'txt' : txt_data ,
            'pred': all_preds,
            'prob': all_probs
        })
        df_result.to_csv(result_file, index=False, encoding='utf-8-sig')

    return avg_loss, accuracy, f1, precision, recall, auc, tpr_at_fpr_0001, (fpr, tpr)



def train_model(
    model,
    train_dataloader,
    valid_dataloader,
    test_dataloader,
    device,
    learning_rate=3e-5,
    num_epochs=5,
    accumulation_steps=2,
    num_cls=2,
    run_id=0,
    early_stopping_patience=2,
    train_data=None,
    val_data=None,
    test_data=None,
    auroc_weight=0.0,
    result_file=None
):
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    model.to(device)

    print(f'-+-+-+ Run {run_id+1}: Initial Validation Metrics +-+-+-')
    eval_model(model, valid_dataloader, device, num_cls, data_source=val_data, phase="initial_val")
    print('-+-+-+ END OF INITIAL EVALUATION +-+-+-')

    train_loss_log = []
    valid_loss_log = []
    best_combined_score = 0.0  # 최고 결합 점수 초기화
    best_model_state = None
    early_stop_counter = 0
    all_wrong_predictions = []
    grad_norms_log = []
    epoch_stats_log = []  # 에폭별 통계

    # 최적 성능 기록용
    best_valid_auroc = 0.0
    best_valid_tpr = 0.0

    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_count = 0
        phase_name = f"val_epoch{epoch+1}"
        valid_result_file = f"{result_dir}valid_result_run{run_id+1}_epoch{epoch+1}_{safe_model_name}_{method}.csv"
        model.train()  # 에폭 시작 시 그래디언트 초기화

        epoch_grad_norms = []

        for step, batch in enumerate(train_dataloader):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss / accumulation_steps
            total_train_loss += loss.item()
            total_count += 1

            loss.backward()
            total_norm = 0
            for p in model.parameters():
                if p.grad is not None:
                     param_norm = p.grad.data.norm(2)
                     total_norm += param_norm.item() ** 2

            total_norm = total_norm ** 0.5
            epoch_grad_norms.append(total_norm)
            grad_norms_log.append((epoch, step, total_norm))

            if (step + 1) % accumulation_steps == 0  or (step + 1) == len(train_dataloader) :
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()

        epoch_avg_norm = np.mean(epoch_grad_norms) if epoch_grad_norms else 0
        epoch_std_norm = np.std(epoch_grad_norms) if epoch_grad_norms else 0
        epoch_stats_log.append((epoch, epoch_avg_norm, epoch_std_norm))

        print(f"Epoch {epoch+1} - Gradient norm stats: Avg: {epoch_avg_norm:.4f}, Std: {epoch_std_norm:.4f}")

        avg_train_loss = total_train_loss / total_count
        train_loss_log.append(avg_train_loss)
        if val_data is None:
            raise ValueError("Validation data is None.")
        if test_data is None:
            raise ValueError("Test data is None.")

        print(f'Run {run_id+1}, Epoch {epoch + 1} 성능 평가:')
        # 현재 에폭에서 검증 데이터 평가 및 틀린 예측 저장
        phase_name = f"val_epoch{epoch+1}"

        avg_valid_loss, valid_acc, valid_f1, valid_precision, valid_recall, valid_auc, valid_tpr, _ = eval_model(
            model, valid_dataloader, device, num_cls,
            data_source=val_data,
            phase=phase_name,
            result_file=valid_result_file
        )
        valid_loss_log.append(avg_valid_loss)


       # AUROC와 TPR@FPR=0.01%를 결합한 점수 계산
        combined_score = auroc_weight * valid_auc + (1 - auroc_weight) * valid_tpr

        # Save the best model based on combined score
        if combined_score > best_combined_score:
            best_combined_score = combined_score
            best_valid_auroc = valid_auc  # 최고 성능일 때 AUROC 값 저장
            best_valid_tpr = valid_tpr    # 최고 성능일 때 TPR 값 저장
            best_model_state = model.state_dict().copy()
            early_stop_counter = 0
            print(f"Run {run_id+1}: New best model found with combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {valid_auc:.4f} (weight: {auroc_weight:.2f})")
            print(f"  - TPR @ FPR=0.01%: {valid_tpr:.4f} (weight: {(1-auroc_weight):.2f})")
        else:
            early_stop_counter += 1
            print(f"Run {run_id+1}: No improvement. Early stop counter = {early_stop_counter}/{early_stopping_patience}")

        # Early stopping 조건 만족 시 중단
        if early_stop_counter >= early_stopping_patience:
            print(f"Run {run_id+1}: Early stopping triggered at epoch {epoch + 1}")
            break

    # 최고 성능 모델로 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Run {run_id+1}: Restored best model for final evaluation")

    # 그래디언트 노름 로그 저장
    grad_norms_df = pd.DataFrame(grad_norms_log, columns=['epoch', 'step', 'batch_norm'])
    grad_norms_df.to_csv(f"{result_dir}grad_norms_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8-sig')

    epoch_stats_df = pd.DataFrame(epoch_stats_log, columns=['epoch', 'avg_norm', 'std_norm'])
    epoch_stats_df.to_csv(f"{result_dir}epoch_grad_stats_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8-sig')

    print(f'----- Run {run_id+1}: Final Evaluation on Test Data -----')
    test_result_file = f"{result_dir}test_result_run{run_id+1}_{safe_model_name}_{method}.csv"
    test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, _ = eval_model(
        model, test_dataloader, device, num_cls,
        data_source=test_data,
        phase="test",
        result_file=test_result_file
    )
    loss_log_df = pd.DataFrame({'train_loss_log': train_loss_log,'valid_loss_log': valid_loss_log})
    loss_log_df.to_csv(f"{result_dir}loss_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8')


    return train_loss_log, valid_loss_log, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score

# seed 추가
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True) # 추가


def run_multiple_experiments(
    model_class,
    model_name,
    train_data,
    val_data,
    test_data,
    tokenizer,
    device,
    num_runs=3,
    learning_rate=3e-5,
    num_epochs=5,
    batch_size=16,
    accumulation_steps=2,
    num_cls=2,
    seed=42,
    auroc_weight=0.0  # AUROC 가중치 추가
):
    # seed=42 설정
    set_seed(seed)

    # 결과 저장을 위한 리스트
    all_test_metrics = []
    best_overall_score = 0.0  # 전체 실험에서 최고 결합 점수
    best_overall_auroc = 0.0  # 최고 모델의 AUROC
    best_overall_tpr = 0.0    # 최고 모델의 TPR
    best_overall_model_state = None
    best_run_id = -1

    # 데이터로더 생성
    train_dataset = data_preparing(train_data, tokenizer, batch_size=batch_size, shuffle=True, return_dataset=True)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    valid_dataloader = data_preparing(val_data, tokenizer, batch_size=batch_size, shuffle=False)
    test_dataloader = data_preparing(test_data, tokenizer, batch_size=batch_size, shuffle=False)

    print(f"\nUsing combined metric with AUROC weight: {auroc_weight:.2f}, TPR@FPR=0.01% weight: {(1-auroc_weight):.2f}")

    for run in range(num_runs):
        print(f"\n===== Starting Run {run+1}/{num_runs} =====")

        # 새로운 모델 인스턴스 생성
        model = model_class.from_pretrained(model_name, num_labels=num_cls)

        # 모델 학습 - 원본 데이터 전달
        _, _, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score= train_model(
            model=model,
            train_dataloader=train_dataloader,
            valid_dataloader=valid_dataloader,
            test_dataloader=test_dataloader,
            train_data=train_data,
            val_data=val_data,
            test_data=test_data,
            device=device,
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            accumulation_steps=accumulation_steps,
            num_cls=num_cls,
            run_id=run
        )

        # 테스트 메트릭 저장
        run_metrics = {
            'loss': test_loss,
            'accuracy': test_acc,
            'f1': test_f1,
            'precision': test_precision,
            'recall': test_recall,
            'auc': test_auc,
            'tpr_at_fpr_0001': test_tpr,
            'valid_auroc': best_valid_auroc,
            'valid_tpr': best_valid_tpr,
            'combined_score': best_combined_score
        }
        all_test_metrics.append(run_metrics)

        # Update best overall model based on combined score
        if best_combined_score > best_overall_score:
            best_overall_score = best_combined_score
            best_overall_auroc = best_valid_auroc
            best_overall_tpr = best_valid_tpr
            best_overall_model_state = best_model_state
            best_run_id = run

            # best model 저장
            torch.save(best_model_state, f"{result_dir}best_model_run_{run+1}_score_{best_combined_score:.4f}_auroc_{best_valid_auroc:.4f}_tpr_{best_valid_tpr:.4f}_{safe_model_name}_{method}.pt")
            print(f"New overall best model saved from run {run+1}")
            print(f"  - Combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {best_valid_auroc:.4f}")
            print(f"  - TPR @ FPR=0.01%: {best_valid_tpr:.4f}")

    # 평균 성능 계산
    avg_metrics = {metric: np.mean([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}
    std_metrics = {metric: np.std([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}

    print("\n===== Average Test Results Across All Runs =====")
    print(f"Avg Test Loss: {avg_metrics['loss']:.4f} ± {std_metrics['loss']:.4f}")
    print(f"Avg Test Accuracy: {avg_metrics['accuracy']:.4f} ± {std_metrics['accuracy']:.4f}")
    print(f"Avg Test F1 Score: {avg_metrics['f1']:.4f} ± {std_metrics['f1']:.4f}")
    print(f"Avg Test Precision: {avg_metrics['precision']:.4f} ± {std_metrics['precision']:.4f}")
    print(f"Avg Test Recall: {avg_metrics['recall']:.4f} ± {std_metrics['recall']:.4f}")
    print(f"Avg Test AUROC: {avg_metrics['auc']:.4f} ± {std_metrics['auc']:.4f}")
    print(f"Avg Test TPR @ FPR=0.01%: {avg_metrics['tpr_at_fpr_0001']:.4f} ± {std_metrics['tpr_at_fpr_0001']:.4f}")
    print(f"Avg Validation AUROC: {avg_metrics['valid_auroc']:.4f} ± {std_metrics['valid_auroc']:.4f}")
    print(f"Avg Validation TPR @ FPR=0.01%: {avg_metrics['valid_tpr']:.4f} ± {std_metrics['valid_tpr']:.4f}")
    print(f"Avg Combined Score: {avg_metrics['combined_score']:.4f} ± {std_metrics['combined_score']:.4f}")

    print(f"\n===== Best Model from Run {best_run_id+1} =====")
    print(f"Combined Score: {best_overall_score:.4f}")
    print(f"Best Validation AUROC: {best_overall_auroc:.4f}")
    print(f"Best Validation TPR: {best_overall_tpr:.4f}")
    print(f"Saved as: best_model_run_{best_run_id+1}_score_{best_overall_score:.4f}_auroc_{best_overall_auroc:.4f}_tpr_{best_overall_tpr:.4f}_{safe_model_name}_{method}.pt")

    # 평균 성능 저장
    summary_df = pd.DataFrame({
      'metric': list(avg_metrics.keys()),
      'mean': [avg_metrics[m] for m in avg_metrics],
      'std': [std_metrics[m] for m in std_metrics]
    })
    summary_file_path = os.path.join(result_dir, f"summary_metrics_{safe_model_name}_{method}.csv")
    summary_df.to_csv(summary_file_path, index=False, encoding='utf-8')

    # best overall model 저장
    torch.save(best_overall_model_state, f"{result_dir}best_model_overall_{safe_model_name}_{method}.pt")
    print(f"Best overall model saved as: best_model_overall_{safe_model_name}_{method}.pt")

    return all_test_metrics, avg_metrics, std_metrics, best_overall_model_state, best_run_id


def main():
   # 데이터 분할 (7:1.5:1.5)

    prompt_list = rs_sample['info.essay_prompt'].unique().tolist()
    train_prompts, temp_prompts = train_test_split(prompt_list,
                                                   test_size=0.3,
                                                   random_state=42,
                                                   )
    val_prompts, test_prompts = train_test_split(temp_prompts,
                                                 test_size=0.5,
                                                 random_state=42,
                                                 )
    train_data = rs_sample[rs_sample['info.essay_prompt'].isin(train_prompts)].reset_index(drop=True)
    val_data = rs_sample[rs_sample['info.essay_prompt'].isin(val_prompts)].reset_index(drop=True)
    test_data = rs_sample[rs_sample['info.essay_prompt'].isin(test_prompts)].reset_index(drop=True)



    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    auroc_weight = 0.0  # 이 값을 조정하여 AUROC의 중요도 설정 (0~1 사이 값)

    all_test_metrics, avg_metrics, std_metrics, best_model_state, best_run_id = run_multiple_experiments(
        model_class=model_class,
        model_name=model_name,
        train_data=train_data,
        val_data=val_data,
        test_data=test_data,
        tokenizer=tokenizer,
        device=device,
        num_runs=1,
        learning_rate=3e-5,
        num_epochs=5,
        batch_size=16,
        accumulation_steps=2,
        seed=42,
        auroc_weight=auroc_weight  # AUROC 가중치 전달
    )

    # Final evaluation with best model
    print("\n===== Final Evaluation with Best Overall Model =====")
    final_model = model_class.from_pretrained(model_name, num_labels=2)
    final_model.load_state_dict(best_model_state)
    final_model.to(device)

    test_dataloader = data_preparing(test_data, tokenizer, batch_size=16, shuffle=False)
    _, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr,_ = eval_model(final_model, test_dataloader, device)


start_time = time.time()

if __name__ == "__main__":
    main()

elapsed_time_sec = time.time() - start_time
elapsed_time_min = elapsed_time_sec / 60
print(f"소요 시간: {elapsed_time_min:.2f}분")


Using combined metric with AUROC weight: 0.00, TPR@FPR=0.01% weight: 1.00

===== Starting Run 1/1 =====


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at lighthouse/mdeberta-v3-base-kor-further and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 1: Initial Validation Metrics +-+-+-
Loss: 0.6705 | Acc: 0.6088 | F1: 0.0000 | Precision: 0.0000 | Recall: 0.0000 | AUROC: 0.4433 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Epoch 1 - Gradient norm stats: Avg: 0.7749, Std: 2.1587
Run 1, Epoch 1 성능 평가:
Loss: 0.0611 | Acc: 0.9781 | F1: 0.9727 | Precision: 0.9469 | Recall: 1.0000 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9952
Run 1: New best model found with combined score: 0.9952
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9952 (weight: 1.00)
Epoch 2 - Gradient norm stats: Avg: 0.1508, Std: 0.7642
Run 1, Epoch 2 성능 평가:
Loss: 0.0135 | Acc: 0.9959 | F1: 0.9948 | Precision: 0.9897 | Recall: 1.0000 | AUROC: 0.9999 | TPR@FPR=0.01%: 0.6671
Run 1: No improvement. Early stop counter = 1/2
Epoch 3 - Gradient norm stats: Avg: 0.0613, Std: 0.5044
Run 1, Epoch 3 성능 평가:
Loss: 0.0372 | Acc: 0.9927 | F1: 0.9906 | Precision: 1.0000 | Recall: 0.9813 | AUROC: 0.9998 | TPR@FPR=0.01%: 0.9813
Run 1: No improvement. Early s

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at lighthouse/mdeberta-v3-base-kor-further and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loss: 0.0435 | Acc: 0.9913 | F1: 0.9891 | Precision: 0.9993 | Recall: 0.9792 | AUROC: 0.9999 | TPR@FPR=0.01%: 0.9772
소요 시간: 96.33분


In [ ]:
# static

safe_model_name = model_name.split('/')[-1] if '/' in model_name else model_name
method = "static_grd" #static

class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.texts = dataframe['txt'].tolist()
        self.labels = dataframe['target'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': tokenized['input_ids'].squeeze(0),  # 차원 축소 유지
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)  # 데이터 타입 명확히 설정
        }

def re_weighting_sequence(target_series, mode='static'):
    wrong_cnt = np.sum(target_series)*1.0
    right_cnt = len(target_series)*1.0 - wrong_cnt
    if wrong_cnt == 0:
        wrong_cnt = 1
    if mode == 'adaptive':
        res_series = [right_cnt*1.0 / wrong_cnt if e == 1 else 1 for e in target_series]
    elif mode == 'static':
        res_series = [2 if e == 1 else 1 for e in target_series]

    return res_series

def wrong_first_dataloader(model, tokenized_datasets, batch_size=16, num_cls=2, current_epoch=1, device=torch.device("cpu")):
    if current_epoch == 1:
        return DataLoader(tokenized_datasets, shuffle=True, batch_size=batch_size)

    eval_dataloader = DataLoader(tokenized_datasets, shuffle=False, batch_size=1)

    all_preds = []
    all_labels = []
    right_wrong_list = []

    model.eval()
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )


        predictions = torch.argmax(outputs.logits, dim=-1)
        all_preds.append(predictions.item())
        all_labels.append(batch["labels"].item())

        right_wrong_list.append(1 if predictions.item() != batch["labels"].item() else 0)

    # 샘플링 가중치 설정
    weights = re_weighting_sequence(right_wrong_list)

    # 가중치 길이가 맞는지 확인
    if len(weights) != len(tokenized_datasets):
        print(f"Warning: weights length {len(weights)} does not match dataset size {len(tokenized_datasets)}")

    sampler = WeightedRandomSampler(weights=weights, num_samples=len(tokenized_datasets), replacement=True)

    return DataLoader(tokenized_datasets, batch_size=batch_size, sampler=sampler)




def data_preparing(data_src, tokenizer, batch_size=16, shuffle=True, return_dataset=False):
    dataset = TextDataset(data_src, tokenizer)

    if return_dataset:
        return dataset

    dataloader = DataLoader(
        dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )
    return dataloader


def eval_model(
    model,
    eval_dataloader,
    device,
    num_cls=2,
    data_source=None,
    phase=None,
    wrong_predictions_list=None,
    result_file=None  # 추가
):
    total_loss = 0.0
    total_count = 0
    all_preds, all_labels, all_probs = [], [], []

    model.eval()
    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss
            total_loss += loss.item() * batch['labels'].size(0)
            total_count += batch['labels'].size(0)

            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = torch.argmax(logits, dim=-1)
            labels = batch['labels']

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = total_loss / total_count
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs) if not np.all(all_probs == all_probs[0]) else 0.0

    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    tpr_at_fpr_0001 = np.interp(0.01 / 100, fpr, tpr) if len(fpr) > 0 else 0.0

    print(f"Loss: {avg_loss:.4f} | Acc: {accuracy:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | AUROC: {auc:.4f} | TPR@FPR=0.01%: {tpr_at_fpr_0001:.4f}")

    if data_source is not None and 'txt' in data_source:
        txt_data = [text for text in data_source['txt']]
    else:
        txt_data = []

    # 결과 저장
    if result_file is not None:
        df_result = pd.DataFrame({
            'label': all_labels,
            'txt' : txt_data ,
            'pred': all_preds,
            'prob': all_probs
        })
        df_result.to_csv(result_file, index=False, encoding='utf-8-sig')

    return avg_loss, accuracy, f1, precision, recall, auc, tpr_at_fpr_0001, (fpr, tpr)


def train_model(
    model,
    train_dataloader,
    valid_dataloader,
    test_dataloader,
    device,
    learning_rate=3e-5,
    num_epochs=5,
    accumulation_steps=2,
    num_cls=2,
    run_id=0,
    early_stopping_patience=2,
    train_data=None,
    val_data=None,
    test_data=None,
    auroc_weight=0.0,
    result_file=None
):
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    model.to(device)
    print(f'-+-+-+ Run {run_id+1}: Initial Validation Metrics +-+-+-')
    eval_model(model, valid_dataloader, device, num_cls)
    print('-+-+-+ END OF INITIAL EVALUATION +-+-+-')


    train_loss_log = []
    valid_loss_log = []
    best_combined_score = 0.0  # 최고 결합 점수 초기화
    best_model_state = None
    early_stop_counter = 0
    all_wrong_predictions = []
    grad_norms_log = []
    epoch_stats_log = []  # 에폭별 통계

    # 최적 성능 기록용
    best_valid_auroc = 0.0
    best_valid_tpr = 0.0

    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_count = 0
        phase_name = f"val_epoch{epoch+1}"
        valid_result_file = f"{result_dir}valid_result_run{run_id+1}_epoch{epoch+1}_{safe_model_name}_{method}.csv"
        model.train()  # 에폭 시작 시 그래디언트 초기화
        epoch_grad_norms = []

        train_dataloader_current = wrong_first_dataloader(
            model,
            train_dataloader.dataset,
            batch_size=train_dataloader.batch_size,
            num_cls=num_cls,
            current_epoch=epoch+1,
            device=device
        )

        for step, batch in enumerate(train_dataloader_current) :
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss / accumulation_steps
            total_train_loss += loss.item()
            total_count += 1

            loss.backward()
            total_norm = 0

            for p in model.parameters():
                if p.grad is not None:
                     param_norm = p.grad.data.norm(2)
                     total_norm += param_norm.item() ** 2

            total_norm = total_norm ** 0.5
            epoch_grad_norms.append(total_norm)
            grad_norms_log.append((epoch, step, total_norm))

            if (step + 1) % accumulation_steps == 0  or (step + 1) == len(train_dataloader) :
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()

        epoch_avg_norm = np.mean(epoch_grad_norms) if epoch_grad_norms else 0
        epoch_std_norm = np.std(epoch_grad_norms) if epoch_grad_norms else 0
        epoch_stats_log.append((epoch, epoch_avg_norm, epoch_std_norm))

        print(f"Epoch {epoch+1} - Gradient norm stats: Avg: {epoch_avg_norm:.4f}, Std: {epoch_std_norm:.4f}")

        avg_train_loss = total_train_loss / total_count
        train_loss_log.append(avg_train_loss)
        if val_data is None:
            raise ValueError("Validation data is None.")
        if test_data is None:
            raise ValueError("Test data is None.")

        print(f'Run {run_id+1}, Epoch {epoch + 1} 성능 평가:')
        # 현재 에폭에서 검증 데이터 평가 및 틀린 예측 저장
        phase_name = f"val_epoch{epoch+1}"


        avg_valid_loss, valid_acc, valid_f1, valid_precision, valid_recall, valid_auc, valid_tpr, _ = eval_model(
            model, valid_dataloader, device, num_cls,
            data_source=val_data,
            phase=phase_name,
            result_file=valid_result_file
        )
        valid_loss_log.append(avg_valid_loss)



       # AUROC와 TPR@FPR=0.01%를 결합한 점수 계산
        combined_score = auroc_weight * valid_auc + (1 - auroc_weight) * valid_tpr

        # Save the best model based on combined score
        if combined_score > best_combined_score:
            best_combined_score = combined_score
            best_valid_auroc = valid_auc  # 최고 성능일 때 AUROC 값 저장
            best_valid_tpr = valid_tpr    # 최고 성능일 때 TPR 값 저장
            best_model_state = model.state_dict().copy()
            early_stop_counter = 0
            print(f"Run {run_id+1}: New best model found with combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {valid_auc:.4f} (weight: {auroc_weight:.2f})")
            print(f"  - TPR @ FPR=0.01%: {valid_tpr:.4f} (weight: {(1-auroc_weight):.2f})")
        else:
            early_stop_counter += 1
            print(f"Run {run_id+1}: No improvement. Early stop counter = {early_stop_counter}/{early_stopping_patience}")

        # Early stopping 조건 만족 시 중단
        if early_stop_counter >= early_stopping_patience:
            print(f"Run {run_id+1}: Early stopping triggered at epoch {epoch + 1}")
            break

    # 최고 성능 모델로 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Run {run_id+1}: Restored best model for final evaluation")

    # 그래디언트 노름 로그 저장
    grad_norms_df = pd.DataFrame(grad_norms_log, columns=['epoch', 'step', 'batch_norm'])
    grad_norms_df.to_csv(f"{result_dir}grad_norms_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8-sig')

    epoch_stats_df = pd.DataFrame(epoch_stats_log, columns=['epoch', 'avg_norm', 'std_norm'])
    epoch_stats_df.to_csv(f"{result_dir}epoch_grad_stats_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8-sig')


    print(f'----- Run {run_id+1}: Final Evaluation on Test Data -----')
    test_result_file = f"{result_dir}test_result_run{run_id+1}_{safe_model_name}_{method}.csv"
    test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, _ = eval_model(
        model, test_dataloader, device, num_cls,
        data_source=test_data,
        phase="test",
        result_file=test_result_file
    )
    loss_log_df = pd.DataFrame({'train_loss_log': train_loss_log,'valid_loss_log': valid_loss_log})
    loss_log_df.to_csv(f"{result_dir}loss_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8')


    return train_loss_log, valid_loss_log, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score

# seed 추가
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True) # 추가


def run_multiple_experiments(
    model_class,
    model_name,
    train_data,
    val_data,
    test_data,
    tokenizer,
    device,
    num_runs=3,
    learning_rate=3e-5,
    num_epochs=5,
    batch_size=16,
    accumulation_steps=2,
    num_cls=2,
    seed=42,
    auroc_weight=0.0  # AUROC 가중치 추가
):
    # seed=42 설정
    set_seed(seed)

    # 결과 저장을 위한 리스트
    all_test_metrics = []
    best_overall_score = 0.0  # 전체 실험에서 최고 결합 점수
    best_overall_auroc = 0.0  # 최고 모델의 AUROC
    best_overall_tpr = 0.0    # 최고 모델의 TPR
    best_overall_model_state = None
    best_run_id = -1

    # 데이터로더 생성
    train_dataset = data_preparing(train_data, tokenizer, batch_size=batch_size, shuffle=True, return_dataset=True)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    valid_dataloader = data_preparing(val_data, tokenizer, batch_size=batch_size, shuffle=False)
    test_dataloader = data_preparing(test_data, tokenizer, batch_size=batch_size, shuffle=False)

    print(f"\nUsing combined metric with AUROC weight: {auroc_weight:.2f}, TPR@FPR=0.01% weight: {(1-auroc_weight):.2f}")

    for run in range(num_runs):
        print(f"\n===== Starting Run {run+1}/{num_runs} =====")

        # 새로운 모델 인스턴스 생성
        model = model_class.from_pretrained(model_name, num_labels=num_cls)

        # 모델 학습 - 원본 데이터 전달
        _, _, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score= train_model(
            model=model,
            train_dataloader=train_dataloader,
            valid_dataloader=valid_dataloader,
            test_dataloader=test_dataloader,
            train_data=train_data,
            val_data=val_data,
            test_data=test_data,
            device=device,
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            accumulation_steps=accumulation_steps,
            num_cls=num_cls,
            run_id=run
        )

        # 테스트 메트릭 저장
        run_metrics = {
            'loss': test_loss,
            'accuracy': test_acc,
            'f1': test_f1,
            'precision': test_precision,
            'recall': test_recall,
            'auc': test_auc,
            'tpr_at_fpr_0001': test_tpr,
            'valid_auroc': best_valid_auroc,
            'valid_tpr': best_valid_tpr,
            'combined_score': best_combined_score
        }
        all_test_metrics.append(run_metrics)

        # Update best overall model based on combined score
        if best_combined_score > best_overall_score:
            best_overall_score = best_combined_score
            best_overall_auroc = best_valid_auroc
            best_overall_tpr = best_valid_tpr
            best_overall_model_state = best_model_state
            best_run_id = run

            # best model 저장
            torch.save(best_model_state, f"{result_dir}best_model_run_{run+1}_score_{best_combined_score:.4f}_auroc_{best_valid_auroc:.4f}_tpr_{best_valid_tpr:.4f}_{safe_model_name}_{method}.pt")
            print(f"New overall best model saved from run {run+1}")
            print(f"  - Combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {best_valid_auroc:.4f}")
            print(f"  - TPR @ FPR=0.01%: {best_valid_tpr:.4f}")

    # 평균 성능 계산
    avg_metrics = {metric: np.mean([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}
    std_metrics = {metric: np.std([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}

    print("\n===== Average Test Results Across All Runs =====")
    print(f"Avg Test Loss: {avg_metrics['loss']:.4f} ± {std_metrics['loss']:.4f}")
    print(f"Avg Test Accuracy: {avg_metrics['accuracy']:.4f} ± {std_metrics['accuracy']:.4f}")
    print(f"Avg Test F1 Score: {avg_metrics['f1']:.4f} ± {std_metrics['f1']:.4f}")
    print(f"Avg Test Precision: {avg_metrics['precision']:.4f} ± {std_metrics['precision']:.4f}")
    print(f"Avg Test Recall: {avg_metrics['recall']:.4f} ± {std_metrics['recall']:.4f}")
    print(f"Avg Test AUROC: {avg_metrics['auc']:.4f} ± {std_metrics['auc']:.4f}")
    print(f"Avg Test TPR @ FPR=0.01%: {avg_metrics['tpr_at_fpr_0001']:.4f} ± {std_metrics['tpr_at_fpr_0001']:.4f}")
    print(f"Avg Validation AUROC: {avg_metrics['valid_auroc']:.4f} ± {std_metrics['valid_auroc']:.4f}")
    print(f"Avg Validation TPR @ FPR=0.01%: {avg_metrics['valid_tpr']:.4f} ± {std_metrics['valid_tpr']:.4f}")
    print(f"Avg Combined Score: {avg_metrics['combined_score']:.4f} ± {std_metrics['combined_score']:.4f}")

    print(f"\n===== Best Model from Run {best_run_id+1} =====")
    print(f"Combined Score: {best_overall_score:.4f}")
    print(f"Best Validation AUROC: {best_overall_auroc:.4f}")
    print(f"Best Validation TPR: {best_overall_tpr:.4f}")
    print(f"Saved as: best_model_run_{best_run_id+1}_score_{best_overall_score:.4f}_auroc_{best_overall_auroc:.4f}_tpr_{best_overall_tpr:.4f}_{safe_model_name}_{method}.pt")

    # 평균 성능 저장
    summary_df = pd.DataFrame({
      'metric': list(avg_metrics.keys()),
      'mean': [avg_metrics[m] for m in avg_metrics],
      'std': [std_metrics[m] for m in std_metrics]
    })
    summary_file_path = os.path.join(result_dir, f"summary_metrics_{safe_model_name}_{method}.csv")
    summary_df.to_csv(summary_file_path, index=False, encoding='utf-8')

    # best overall model 저장
    torch.save(best_overall_model_state, f"{result_dir}best_model_overall_{safe_model_name}_{method}.pt")
    print(f"Best overall model saved as: best_model_overall_{safe_model_name}_{method}.pt")

    return all_test_metrics, avg_metrics, std_metrics, best_overall_model_state, best_run_id


def main():
   # 데이터 분할 (7:1.5:1.5)

    prompt_list = rs_sample['info.essay_prompt'].unique().tolist()
    train_prompts, temp_prompts = train_test_split(prompt_list,
                                                   test_size=0.3,
                                                   random_state=42,
                                                   )
    val_prompts, test_prompts = train_test_split(temp_prompts,
                                                 test_size=0.5,
                                                 random_state=42,
                                                 )
    train_data = rs_sample[rs_sample['info.essay_prompt'].isin(train_prompts)].reset_index(drop=True)
    val_data = rs_sample[rs_sample['info.essay_prompt'].isin(val_prompts)].reset_index(drop=True)
    test_data = rs_sample[rs_sample['info.essay_prompt'].isin(test_prompts)].reset_index(drop=True)


    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    auroc_weight = 0.0  # 이 값을 조정하여 AUROC의 중요도 설정 (0~1 사이 값)

    all_test_metrics, avg_metrics, std_metrics, best_model_state, best_run_id = run_multiple_experiments(
        model_class=model_class,
        model_name=model_name,
        train_data=train_data,
        val_data=val_data,
        test_data=test_data,
        tokenizer=tokenizer,
        device=device,
        num_runs=1,
        learning_rate=3e-5,
        num_epochs=5,
        batch_size=16,
        accumulation_steps=2,
        seed=42,
        auroc_weight=auroc_weight  # AUROC 가중치 전달
    )

    # Final evaluation with best model
    print("\n===== Final Evaluation with Best Overall Model =====")
    final_model = model_class.from_pretrained(model_name, num_labels=2)
    final_model.load_state_dict(best_model_state)
    final_model.to(device)

    test_dataloader = data_preparing(test_data, tokenizer, batch_size=16, shuffle=False)
    _, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr,_ = eval_model(final_model, test_dataloader, device)


start_time = time.time()

if __name__ == "__main__":
    main()

elapsed_time_sec = time.time() - start_time
elapsed_time_min = elapsed_time_sec / 60
print(f"소요 시간: {elapsed_time_min:.2f}분")


Using combined metric with AUROC weight: 0.00, TPR@FPR=0.01% weight: 1.00

===== Starting Run 1/1 =====


config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/559M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/559M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at lighthouse/mdeberta-v3-base-kor-further and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 1: Initial Validation Metrics +-+-+-
Loss: 0.6705 | Acc: 0.6088 | F1: 0.0000 | Precision: 0.0000 | Recall: 0.0000 | AUROC: 0.4433 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Epoch 1 - Gradient norm stats: Avg: 0.7749, Std: 2.1587
Run 1, Epoch 1 성능 평가:
Loss: 0.0611 | Acc: 0.9781 | F1: 0.9727 | Precision: 0.9469 | Recall: 1.0000 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9952
Run 1: New best model found with combined score: 0.9952
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9952 (weight: 1.00)
Epoch 2 - Gradient norm stats: Avg: 0.2832, Std: 1.2616
Run 1, Epoch 2 성능 평가:
Loss: 0.0014 | Acc: 0.9992 | F1: 0.9990 | Precision: 0.9993 | Recall: 0.9986 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9986
Run 1: New best model found with combined score: 0.9986
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9986 (weight: 1.00)
Epoch 3 - Gradient norm stats: Avg: 0.0376, Std: 0.3209
Run 1, Epoch 3 성능 평가:
Loss: 0.0073 | Acc: 0.9970 | F1: 0.9962 | Precision: 0.9972 | R

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at lighthouse/mdeberta-v3-base-kor-further and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loss: 0.0244 | Acc: 0.9932 | F1: 0.9915 | Precision: 0.9986 | Recall: 0.9846 | AUROC: 0.9999 | TPR@FPR=0.01%: 0.9711
소요 시간: 166.38분


In [ ]:
# 모델
# roberta-small
# roberta-base
# monologg/distilkobert
# lighthouse/mdeberta-v3-base-kor-further

tokenizer = AutoTokenizer.from_pretrained("monologg/distilkobert")
model_class = AutoModelForSequenceClassification

model_name = "monologg/distilkobert"

tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

The repository for monologg/distilkobert contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/monologg/distilkobert.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


tokenization_kobert.py:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/monologg/distilkobert:
- tokenization_kobert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_78b3253a26.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

In [ ]:
result_dir = f"/content/drive/My Drive/cdf/"

safe_model_name = model_name.split('/')[-1] if '/' in model_name else model_name
method = "pure_grd"

class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.texts = dataframe['txt'].tolist()
        self.labels = dataframe['target'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': tokenized['input_ids'].squeeze(0),  # 차원 축소 유지
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)  # 데이터 타입 명확히 설정
        }


def data_preparing(data_src, tokenizer, batch_size=16, shuffle=True, return_dataset=False):
    dataset = TextDataset(data_src, tokenizer)

    if return_dataset:
        return dataset

    dataloader = DataLoader(
        dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )
    return dataloader


def eval_model(
    model,
    eval_dataloader,
    device,
    num_cls=2,
    data_source=None,
    phase=None,
    wrong_predictions_list=None,
    result_file=None  # 추가
):
    total_loss = 0.0
    total_count = 0
    all_preds, all_labels, all_probs = [], [], []

    model.eval()
    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss
            total_loss += loss.item() * batch['labels'].size(0)
            total_count += batch['labels'].size(0)

            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = torch.argmax(logits, dim=-1)
            labels = batch['labels']

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = total_loss / total_count
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs) if not np.all(all_probs == all_probs[0]) else 0.0

    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    tpr_at_fpr_0001 = np.interp(0.01 / 100, fpr, tpr) if len(fpr) > 0 else 0.0

    print(f"Loss: {avg_loss:.4f} | Acc: {accuracy:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | AUROC: {auc:.4f} | TPR@FPR=0.01%: {tpr_at_fpr_0001:.4f}")

    if data_source is not None and 'txt' in data_source:
        txt_data = [text for text in data_source['txt']]
    else:
        txt_data = []

    # 결과 저장
    if result_file is not None:
        df_result = pd.DataFrame({
            'label': all_labels,
            'txt' : txt_data ,
            'pred': all_preds,
            'prob': all_probs
        })
        df_result.to_csv(result_file, index=False, encoding='utf-8-sig')

    return avg_loss, accuracy, f1, precision, recall, auc, tpr_at_fpr_0001, (fpr, tpr)



def train_model(
    model,
    train_dataloader,
    valid_dataloader,
    test_dataloader,
    device,
    learning_rate=3e-5,
    num_epochs=5,
    accumulation_steps=2,
    num_cls=2,
    run_id=0,
    early_stopping_patience=2,
    train_data=None,
    val_data=None,
    test_data=None,
    auroc_weight=0.0,
    result_file=None
):
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    model.to(device)

    print(f'-+-+-+ Run {run_id+1}: Initial Validation Metrics +-+-+-')
    eval_model(model, valid_dataloader, device, num_cls, data_source=val_data, phase="initial_val")
    print('-+-+-+ END OF INITIAL EVALUATION +-+-+-')

    train_loss_log = []
    valid_loss_log = []
    best_combined_score = 0.0  # 최고 결합 점수 초기화
    best_model_state = None
    early_stop_counter = 0
    all_wrong_predictions = []
    grad_norms_log = []
    epoch_stats_log = []  # 에폭별 통계

    # 최적 성능 기록용
    best_valid_auroc = 0.0
    best_valid_tpr = 0.0

    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_count = 0
        phase_name = f"val_epoch{epoch+1}"
        valid_result_file = f"{result_dir}valid_result_run{run_id+1}_epoch{epoch+1}_{safe_model_name}_{method}.csv"
        model.train()  # 에폭 시작 시 그래디언트 초기화

        epoch_grad_norms = []

        for step, batch in enumerate(train_dataloader):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss / accumulation_steps
            total_train_loss += loss.item()
            total_count += 1

            loss.backward()
            total_norm = 0
            for p in model.parameters():
                if p.grad is not None:
                     param_norm = p.grad.data.norm(2)
                     total_norm += param_norm.item() ** 2

            total_norm = total_norm ** 0.5
            epoch_grad_norms.append(total_norm)
            grad_norms_log.append((epoch, step, total_norm))

            if (step + 1) % accumulation_steps == 0  or (step + 1) == len(train_dataloader) :
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()

        epoch_avg_norm = np.mean(epoch_grad_norms) if epoch_grad_norms else 0
        epoch_std_norm = np.std(epoch_grad_norms) if epoch_grad_norms else 0
        epoch_stats_log.append((epoch, epoch_avg_norm, epoch_std_norm))

        print(f"Epoch {epoch+1} - Gradient norm stats: Avg: {epoch_avg_norm:.4f}, Std: {epoch_std_norm:.4f}")

        avg_train_loss = total_train_loss / total_count
        train_loss_log.append(avg_train_loss)
        if val_data is None:
            raise ValueError("Validation data is None.")
        if test_data is None:
            raise ValueError("Test data is None.")

        print(f'Run {run_id+1}, Epoch {epoch + 1} 성능 평가:')
        # 현재 에폭에서 검증 데이터 평가 및 틀린 예측 저장
        phase_name = f"val_epoch{epoch+1}"

        avg_valid_loss, valid_acc, valid_f1, valid_precision, valid_recall, valid_auc, valid_tpr, _ = eval_model(
            model, valid_dataloader, device, num_cls,
            data_source=val_data,
            phase=phase_name,
            result_file=valid_result_file
        )
        valid_loss_log.append(avg_valid_loss)


       # AUROC와 TPR@FPR=0.01%를 결합한 점수 계산
        combined_score = auroc_weight * valid_auc + (1 - auroc_weight) * valid_tpr

        # Save the best model based on combined score
        if combined_score > best_combined_score:
            best_combined_score = combined_score
            best_valid_auroc = valid_auc  # 최고 성능일 때 AUROC 값 저장
            best_valid_tpr = valid_tpr    # 최고 성능일 때 TPR 값 저장
            best_model_state = model.state_dict().copy()
            early_stop_counter = 0
            print(f"Run {run_id+1}: New best model found with combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {valid_auc:.4f} (weight: {auroc_weight:.2f})")
            print(f"  - TPR @ FPR=0.01%: {valid_tpr:.4f} (weight: {(1-auroc_weight):.2f})")
        else:
            early_stop_counter += 1
            print(f"Run {run_id+1}: No improvement. Early stop counter = {early_stop_counter}/{early_stopping_patience}")

        # Early stopping 조건 만족 시 중단
        if early_stop_counter >= early_stopping_patience:
            print(f"Run {run_id+1}: Early stopping triggered at epoch {epoch + 1}")
            break

    # 최고 성능 모델로 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Run {run_id+1}: Restored best model for final evaluation")

    # 그래디언트 노름 로그 저장
    grad_norms_df = pd.DataFrame(grad_norms_log, columns=['epoch', 'step', 'batch_norm'])
    grad_norms_df.to_csv(f"{result_dir}grad_norms_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8-sig')

    epoch_stats_df = pd.DataFrame(epoch_stats_log, columns=['epoch', 'avg_norm', 'std_norm'])
    epoch_stats_df.to_csv(f"{result_dir}epoch_grad_stats_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8-sig')

    print(f'----- Run {run_id+1}: Final Evaluation on Test Data -----')
    test_result_file = f"{result_dir}test_result_run{run_id+1}_{safe_model_name}_{method}.csv"
    test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, _ = eval_model(
        model, test_dataloader, device, num_cls,
        data_source=test_data,
        phase="test",
        result_file=test_result_file
    )
    loss_log_df = pd.DataFrame({'train_loss_log': train_loss_log,'valid_loss_log': valid_loss_log})
    loss_log_df.to_csv(f"{result_dir}loss_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8')


    return train_loss_log, valid_loss_log, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score

# seed 추가
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True) # 추가


def run_multiple_experiments(
    model_class,
    model_name,
    train_data,
    val_data,
    test_data,
    tokenizer,
    device,
    num_runs=3,
    learning_rate=3e-5,
    num_epochs=5,
    batch_size=16,
    accumulation_steps=2,
    num_cls=2,
    seed=42,
    auroc_weight=0.0  # AUROC 가중치 추가
):
    # seed=42 설정
    set_seed(seed)

    # 결과 저장을 위한 리스트
    all_test_metrics = []
    best_overall_score = 0.0  # 전체 실험에서 최고 결합 점수
    best_overall_auroc = 0.0  # 최고 모델의 AUROC
    best_overall_tpr = 0.0    # 최고 모델의 TPR
    best_overall_model_state = None
    best_run_id = -1

    # 데이터로더 생성
    train_dataset = data_preparing(train_data, tokenizer, batch_size=batch_size, shuffle=True, return_dataset=True)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    valid_dataloader = data_preparing(val_data, tokenizer, batch_size=batch_size, shuffle=False)
    test_dataloader = data_preparing(test_data, tokenizer, batch_size=batch_size, shuffle=False)

    print(f"\nUsing combined metric with AUROC weight: {auroc_weight:.2f}, TPR@FPR=0.01% weight: {(1-auroc_weight):.2f}")

    for run in range(num_runs):
        print(f"\n===== Starting Run {run+1}/{num_runs} =====")

        # 새로운 모델 인스턴스 생성
        model = model_class.from_pretrained(model_name, num_labels=num_cls)

        # 모델 학습 - 원본 데이터 전달
        _, _, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score= train_model(
            model=model,
            train_dataloader=train_dataloader,
            valid_dataloader=valid_dataloader,
            test_dataloader=test_dataloader,
            train_data=train_data,
            val_data=val_data,
            test_data=test_data,
            device=device,
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            accumulation_steps=accumulation_steps,
            num_cls=num_cls,
            run_id=run
        )

        # 테스트 메트릭 저장
        run_metrics = {
            'loss': test_loss,
            'accuracy': test_acc,
            'f1': test_f1,
            'precision': test_precision,
            'recall': test_recall,
            'auc': test_auc,
            'tpr_at_fpr_0001': test_tpr,
            'valid_auroc': best_valid_auroc,
            'valid_tpr': best_valid_tpr,
            'combined_score': best_combined_score
        }
        all_test_metrics.append(run_metrics)

        # Update best overall model based on combined score
        if best_combined_score > best_overall_score:
            best_overall_score = best_combined_score
            best_overall_auroc = best_valid_auroc
            best_overall_tpr = best_valid_tpr
            best_overall_model_state = best_model_state
            best_run_id = run

            # best model 저장
            torch.save(best_model_state, f"{result_dir}best_model_run_{run+1}_score_{best_combined_score:.4f}_auroc_{best_valid_auroc:.4f}_tpr_{best_valid_tpr:.4f}_{safe_model_name}_{method}.pt")
            print(f"New overall best model saved from run {run+1}")
            print(f"  - Combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {best_valid_auroc:.4f}")
            print(f"  - TPR @ FPR=0.01%: {best_valid_tpr:.4f}")

    # 평균 성능 계산
    avg_metrics = {metric: np.mean([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}
    std_metrics = {metric: np.std([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}

    print("\n===== Average Test Results Across All Runs =====")
    print(f"Avg Test Loss: {avg_metrics['loss']:.4f} ± {std_metrics['loss']:.4f}")
    print(f"Avg Test Accuracy: {avg_metrics['accuracy']:.4f} ± {std_metrics['accuracy']:.4f}")
    print(f"Avg Test F1 Score: {avg_metrics['f1']:.4f} ± {std_metrics['f1']:.4f}")
    print(f"Avg Test Precision: {avg_metrics['precision']:.4f} ± {std_metrics['precision']:.4f}")
    print(f"Avg Test Recall: {avg_metrics['recall']:.4f} ± {std_metrics['recall']:.4f}")
    print(f"Avg Test AUROC: {avg_metrics['auc']:.4f} ± {std_metrics['auc']:.4f}")
    print(f"Avg Test TPR @ FPR=0.01%: {avg_metrics['tpr_at_fpr_0001']:.4f} ± {std_metrics['tpr_at_fpr_0001']:.4f}")
    print(f"Avg Validation AUROC: {avg_metrics['valid_auroc']:.4f} ± {std_metrics['valid_auroc']:.4f}")
    print(f"Avg Validation TPR @ FPR=0.01%: {avg_metrics['valid_tpr']:.4f} ± {std_metrics['valid_tpr']:.4f}")
    print(f"Avg Combined Score: {avg_metrics['combined_score']:.4f} ± {std_metrics['combined_score']:.4f}")

    print(f"\n===== Best Model from Run {best_run_id+1} =====")
    print(f"Combined Score: {best_overall_score:.4f}")
    print(f"Best Validation AUROC: {best_overall_auroc:.4f}")
    print(f"Best Validation TPR: {best_overall_tpr:.4f}")
    print(f"Saved as: best_model_run_{best_run_id+1}_score_{best_overall_score:.4f}_auroc_{best_overall_auroc:.4f}_tpr_{best_overall_tpr:.4f}_{safe_model_name}_{method}.pt")

    # 평균 성능 저장
    summary_df = pd.DataFrame({
      'metric': list(avg_metrics.keys()),
      'mean': [avg_metrics[m] for m in avg_metrics],
      'std': [std_metrics[m] for m in std_metrics]
    })
    summary_file_path = os.path.join(result_dir, f"summary_metrics_{safe_model_name}_{method}.csv")
    summary_df.to_csv(summary_file_path, index=False, encoding='utf-8')

    # best overall model 저장
    torch.save(best_overall_model_state, f"{result_dir}best_model_overall_{safe_model_name}_{method}.pt")
    print(f"Best overall model saved as: best_model_overall_{safe_model_name}_{method}.pt")

    return all_test_metrics, avg_metrics, std_metrics, best_overall_model_state, best_run_id


def main():
   # 데이터 분할 (7:1.5:1.5)

    prompt_list = rs_sample['info.essay_prompt'].unique().tolist()
    train_prompts, temp_prompts = train_test_split(prompt_list,
                                                   test_size=0.3,
                                                   random_state=42,
                                                   )
    val_prompts, test_prompts = train_test_split(temp_prompts,
                                                 test_size=0.5,
                                                 random_state=42,
                                                 )
    train_data = rs_sample[rs_sample['info.essay_prompt'].isin(train_prompts)].reset_index(drop=True)
    val_data = rs_sample[rs_sample['info.essay_prompt'].isin(val_prompts)].reset_index(drop=True)
    test_data = rs_sample[rs_sample['info.essay_prompt'].isin(test_prompts)].reset_index(drop=True)



    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    auroc_weight = 0.0  # 이 값을 조정하여 AUROC의 중요도 설정 (0~1 사이 값)

    all_test_metrics, avg_metrics, std_metrics, best_model_state, best_run_id = run_multiple_experiments(
        model_class=model_class,
        model_name=model_name,
        train_data=train_data,
        val_data=val_data,
        test_data=test_data,
        tokenizer=tokenizer,
        device=device,
        num_runs=3,
        learning_rate=3e-5,
        num_epochs=5,
        batch_size=16,
        accumulation_steps=2,
        seed=42,
        auroc_weight=auroc_weight  # AUROC 가중치 전달
    )

    # Final evaluation with best model
    print("\n===== Final Evaluation with Best Overall Model =====")
    final_model = model_class.from_pretrained(model_name, num_labels=2)
    final_model.load_state_dict(best_model_state)
    final_model.to(device)

    test_dataloader = data_preparing(test_data, tokenizer, batch_size=16, shuffle=False)
    _, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr,_ = eval_model(final_model, test_dataloader, device)


start_time = time.time()

if __name__ == "__main__":
    main()

elapsed_time_sec = time.time() - start_time
elapsed_time_min = elapsed_time_sec / 60
print(f"소요 시간: {elapsed_time_min:.2f}분")


Using combined metric with AUROC weight: 0.00, TPR@FPR=0.01% weight: 1.00

===== Starting Run 1/3 =====


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at monologg/distilkobert and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 1: Initial Validation Metrics +-+-+-
Loss: 0.6841 | Acc: 0.6345 | F1: 0.2798 | Precision: 0.6107 | Recall: 0.1814 | AUROC: 0.5498 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Epoch 1 - Gradient norm stats: Avg: 3.3586, Std: 5.5792
Run 1, Epoch 1 성능 평가:
Loss: 0.0365 | Acc: 0.9886 | F1: 0.9854 | Precision: 0.9902 | Recall: 0.9806 | AUROC: 0.9990 | TPR@FPR=0.01%: 0.4494
Run 1: New best model found with combined score: 0.4494
  - AUROC: 0.9990 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.4494 (weight: 1.00)
Epoch 2 - Gradient norm stats: Avg: 1.7633, Std: 5.1843
Run 1, Epoch 2 성능 평가:
Loss: 0.0417 | Acc: 0.9862 | F1: 0.9824 | Precision: 0.9813 | Recall: 0.9834 | AUROC: 0.9988 | TPR@FPR=0.01%: 0.5499
Run 1: New best model found with combined score: 0.5499
  - AUROC: 0.9988 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.5499 (weight: 1.00)
Epoch 3 - Gradient norm stats: Avg: 0.7414, Std: 3.1940
Run 1, Epoch 3 성능 평가:
Loss: 0.0265 | Acc: 0.9916 | F1: 0.9893 | Precision: 0.9869 | R

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at monologg/distilkobert and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 2: Initial Validation Metrics +-+-+-
Loss: 0.6962 | Acc: 0.4470 | F1: 0.3222 | Precision: 0.3095 | Recall: 0.3359 | AUROC: 0.4241 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Epoch 1 - Gradient norm stats: Avg: 2.6360, Std: 4.5191
Run 2, Epoch 1 성능 평가:
Loss: 0.0184 | Acc: 0.9940 | F1: 0.9924 | Precision: 0.9883 | Recall: 0.9965 | AUROC: 0.9997 | TPR@FPR=0.01%: 0.5492
Run 2: New best model found with combined score: 0.5492
  - AUROC: 0.9997 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.5492 (weight: 1.00)
Epoch 2 - Gradient norm stats: Avg: 1.1573, Std: 3.3349
Run 2, Epoch 2 성능 평가:
Loss: 0.0329 | Acc: 0.9892 | F1: 0.9863 | Precision: 0.9743 | Recall: 0.9986 | AUROC: 0.9997 | TPR@FPR=0.01%: 0.8227
Run 2: New best model found with combined score: 0.8227
  - AUROC: 0.9997 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.8227 (weight: 1.00)
Epoch 3 - Gradient norm stats: Avg: 0.8185, Std: 3.2709
Run 2, Epoch 3 성능 평가:
Loss: 0.0395 | Acc: 0.9875 | F1: 0.9843 | Precision: 0.9704 | R

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at monologg/distilkobert and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 3: Initial Validation Metrics +-+-+-
Loss: 0.7051 | Acc: 0.4134 | F1: 0.5709 | Precision: 0.3999 | Recall: 0.9972 | AUROC: 0.5802 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Epoch 1 - Gradient norm stats: Avg: 2.9930, Std: 4.8505
Run 3, Epoch 1 성능 평가:
Loss: 0.0334 | Acc: 0.9886 | F1: 0.9854 | Precision: 0.9916 | Recall: 0.9792 | AUROC: 0.9994 | TPR@FPR=0.01%: 0.6413
Run 3: New best model found with combined score: 0.6413
  - AUROC: 0.9994 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.6413 (weight: 1.00)
Epoch 2 - Gradient norm stats: Avg: 1.1898, Std: 3.3614
Run 3, Epoch 2 성능 평가:
Loss: 0.0244 | Acc: 0.9932 | F1: 0.9914 | Precision: 0.9850 | Recall: 0.9979 | AUROC: 0.9997 | TPR@FPR=0.01%: 0.7791
Run 3: New best model found with combined score: 0.7791
  - AUROC: 0.9997 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.7791 (weight: 1.00)
Epoch 3 - Gradient norm stats: Avg: 0.8705, Std: 3.3233
Run 3, Epoch 3 성능 평가:
Loss: 0.0340 | Acc: 0.9894 | F1: 0.9867 | Precision: 0.9737 | R

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at monologg/distilkobert and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loss: 0.0895 | Acc: 0.9831 | F1: 0.9796 | Precision: 0.9618 | Recall: 0.9980 | AUROC: 0.9994 | TPR@FPR=0.01%: 0.8566
소요 시간: 59.09분


## 베이스라인

In [ ]:
# kobert 출력

tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")
model_class = AutoModelForSequenceClassification
model_name = "monologg/kobert"

tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

The repository for monologg/kobert contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/monologg/kobert.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


tokenization_kobert.py:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/monologg/kobert:
- tokenization_kobert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_78b3253a26.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

In [ ]:
result_dir = f"/content/drive/My Drive/cdf/"

In [ ]:
# baseline

result_dir = f"/content/drive/My Drive/cdf/"

safe_model_name = model_name.split('/')[-1] if '/' in model_name else model_name
method = "pure_gradient"

class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.texts = dataframe['txt'].tolist()
        self.labels = dataframe['target'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': tokenized['input_ids'].squeeze(0),  # 차원 축소 유지
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)  # 데이터 타입 명확히 설정
        }


def data_preparing(data_src, tokenizer, batch_size=16, shuffle=True, return_dataset=False):
    dataset = TextDataset(data_src, tokenizer)

    if return_dataset:
        return dataset

    dataloader = DataLoader(
        dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )
    return dataloader


def eval_model(
    model,
    eval_dataloader,
    device,
    num_cls=2,
    data_source=None,
    phase=None,
    wrong_predictions_list=None,
    result_file=None  # 추가
):
    total_loss = 0.0
    total_count = 0
    all_preds, all_labels, all_probs = [], [], []

    model.eval()
    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss
            total_loss += loss.item() * batch['labels'].size(0)
            total_count += batch['labels'].size(0)

            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = torch.argmax(logits, dim=-1)
            labels = batch['labels']

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = total_loss / total_count
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs) if not np.all(all_probs == all_probs[0]) else 0.0

    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    tpr_at_fpr_0001 = np.interp(0.01 / 100, fpr, tpr) if len(fpr) > 0 else 0.0

    print(f"Loss: {avg_loss:.4f} | Acc: {accuracy:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | AUROC: {auc:.4f} | TPR@FPR=0.01%: {tpr_at_fpr_0001:.4f}")

    if data_source is not None and 'txt' in data_source:
        txt_data = [text for text in data_source['txt']]
    else:
        txt_data = []

    # 결과 저장
    if result_file is not None:
        df_result = pd.DataFrame({
            'label': all_labels,
            'txt' : txt_data ,
            'pred': all_preds,
            'prob': all_probs
        })
        df_result.to_csv(result_file, index=False, encoding='utf-8-sig')

    return avg_loss, accuracy, f1, precision, recall, auc, tpr_at_fpr_0001, (fpr, tpr)



def train_model(
    model,
    train_dataloader,
    valid_dataloader,
    test_dataloader,
    device,
    learning_rate=3e-5,
    num_epochs=5,
    accumulation_steps=2,
    num_cls=2,
    run_id=0,
    early_stopping_patience=2,
    train_data=None,
    val_data=None,
    test_data=None,
    auroc_weight=0.0,
    result_file=None,
    track_gradient_stability=True  # 그래디언트 안정성 추적 여부
):
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    model.to(device)

    print(f'-+-+-+ Run {run_id+1}: Initial Validation Metrics +-+-+-')
    eval_model(model, valid_dataloader, device, num_cls, data_source=val_data, phase="initial_val")
    print('-+-+-+ END OF INITIAL EVALUATION +-+-+-')

    train_loss_log = []
    valid_loss_log = []
    best_combined_score = 0.0  # 최고 결합 점수 초기화
    best_model_state = None
    early_stop_counter = 0
    all_wrong_predictions = []

    # 최적 성능 기록용
    best_valid_auroc = 0.0
    best_valid_tpr = 0.0

    # 그래디언트 안정성 측정을 위한 변수들
    grad_norms_log = []  # 그래디언트 노름 기록
    grad_variances_log = []  # 그래디언트 분산 기록
    cosine_similarities_log = []  # 연속된 배치 간 그래디언트 방향 유사도
    weight_changes_log = []  # 가중치 변화량

    # 이전 그래디언트 값을 저장하기 위한 변수
    prev_grad = None
    # 이전 가중치 상태를 저장하기 위한 변수
    prev_weights = None

    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_count = 0
        phase_name = f"val_epoch{epoch+1}"
        valid_result_file = f"{result_dir}valid_result_run{run_id+1}_epoch{epoch+1}_{safe_model_name}_{method}.csv"
        model.train()  # 에폭 시작 시 그래디언트 초기화

        # 에폭별 그래디언트 정보 저장
        epoch_grad_norms = []
        epoch_grad_values = []  # 그래디언트 값들 수집용
        epoch_cos_similarities = []
        epoch_weight_changes = []

        for step, batch in enumerate(train_dataloader):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss / accumulation_steps
            total_train_loss += loss.item()
            total_count += 1

            loss.backward()

            # 그래디언트 안정성 측정
            if track_gradient_stability:
                # 이전 가중치 저장 (첫 스텝이면 초기화)
                if prev_weights is None and step == 0:
                    prev_weights = {name: param.clone().detach() for name, param in model.named_parameters()}

                # 그래디언트 노름 계산
                total_norm = 0
                flat_grad = []
                for p in model.parameters():
                    if p.grad is not None:
                        param_norm = p.grad.data.norm(2)
                        total_norm += param_norm.item() ** 2
                        flat_grad.append(p.grad.data.flatten())

                total_norm = total_norm ** 0.5
                epoch_grad_norms.append(total_norm)

                # 그래디언트 벡터 병합 (분산 및 방향 계산용)
                current_grad = torch.cat(flat_grad) if flat_grad else None

                # 그래디언트 값 수집 (분산 계산용)
                if current_grad is not None:
                    epoch_grad_values.append(current_grad.cpu().numpy())

                # 코사인 유사도 계산 (이전 그래디언트와 현재 그래디언트 간)
                if prev_grad is not None and current_grad is not None:
                    cos_sim = torch.nn.functional.cosine_similarity(
                        prev_grad.unsqueeze(0),
                        current_grad.unsqueeze(0)
                    ).item()
                    epoch_cos_similarities.append(cos_sim)

                # 현재 그래디언트를 이전 그래디언트로 저장
                prev_grad = current_grad.clone().detach() if current_grad is not None else None

            if (step + 1) % accumulation_steps == 0  or (step + 1) == len(train_dataloader) :
                optimizer.step()
                lr_scheduler.step()

                # 가중치 변화량 계산
                if track_gradient_stability and prev_weights is not None:
                    weight_change = 0
                    for name, param in model.named_parameters():
                        if name in prev_weights:
                            change = torch.norm(param.data - prev_weights[name]).item()
                            weight_change += change
                    epoch_weight_changes.append(weight_change)

                    # 현재 가중치를 이전 가중치로 업데이트
                    prev_weights = {name: param.clone().detach() for name, param in model.named_parameters()}

                optimizer.zero_grad()

        # 에폭의 그래디언트 안정성 지표 계산 및 저장
        if track_gradient_stability:
            # 그래디언트 노름의 평균 및 표준편차
            avg_grad_norm = np.mean(epoch_grad_norms) if epoch_grad_norms else 0
            std_grad_norm = np.std(epoch_grad_norms) if epoch_grad_norms else 0
            grad_norms_log.append((avg_grad_norm, std_grad_norm))

            # 그래디언트 분산 계산
            if epoch_grad_values:
                # 각 파라미터별 분산의 평균을 계산
                grad_variance = np.mean([np.var(vals) for vals in zip(*epoch_grad_values)]) if epoch_grad_values else 0
                grad_variances_log.append(grad_variance)

            # 코사인 유사도의 평균
            avg_cos_sim = np.mean(epoch_cos_similarities) if epoch_cos_similarities else 0
            cosine_similarities_log.append(avg_cos_sim)

            # 가중치 변화량의 평균
            avg_weight_change = np.mean(epoch_weight_changes) if epoch_weight_changes else 0
            weight_changes_log.append(avg_weight_change)

            # 그래디언트 안정성 지표 출력
            print(f"Epoch {epoch+1} Gradient Stability Metrics:")
            print(f"  - Gradient Norm: {avg_grad_norm:.4f} ± {std_grad_norm:.4f}")
            print(f"  - Gradient Direction Consistency (Cosine Similarity): {avg_cos_sim:.4f}")
            print(f"  - Weight Change Magnitude: {avg_weight_change:.4f}")

        avg_train_loss = total_train_loss / total_count
        train_loss_log.append(avg_train_loss)
        if val_data is None:
            raise ValueError("Validation data is None.")
        if test_data is None:
            raise ValueError("Test data is None.")

        print(f'Run {run_id+1}, Epoch {epoch + 1} 성능 평가:')
        # 현재 에폭에서 검증 데이터 평가 및 틀린 예측 저장
        phase_name = f"val_epoch{epoch+1}"

        avg_valid_loss, valid_acc, valid_f1, valid_precision, valid_recall, valid_auc, valid_tpr, _ = eval_model(
            model, valid_dataloader, device, num_cls,
            data_source=val_data,
            phase=phase_name,
            result_file=valid_result_file
        )
        valid_loss_log.append(avg_valid_loss)

        # AUROC와 TPR@FPR=0.01%를 결합한 점수 계산
        combined_score = auroc_weight * valid_auc + (1 - auroc_weight) * valid_tpr

        # Save the best model based on combined score
        if combined_score > best_combined_score:
            best_combined_score = combined_score
            best_valid_auroc = valid_auc  # 최고 성능일 때 AUROC 값 저장
            best_valid_tpr = valid_tpr    # 최고 성능일 때 TPR 값 저장
            best_model_state = model.state_dict().copy()
            early_stop_counter = 0
            print(f"Run {run_id+1}: New best model found with combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {valid_auc:.4f} (weight: {auroc_weight:.2f})")
            print(f"  - TPR @ FPR=0.01%: {valid_tpr:.4f} (weight: {(1-auroc_weight):.2f})")
        else:
            early_stop_counter += 1
            print(f"Run {run_id+1}: No improvement. Early stop counter = {early_stop_counter}/{early_stopping_patience}")

        # Early stopping 조건 만족 시 중단
        if early_stop_counter >= early_stopping_patience:
            print(f"Run {run_id+1}: Early stopping triggered at epoch {epoch + 1}")
            break

    # 그래디언트 안정성 지표 저장
    if track_gradient_stability:
        gradient_metrics_df = pd.DataFrame({
            'epoch': list(range(1, len(grad_norms_log) + 1)),
            'avg_grad_norm': [x[0] for x in grad_norms_log],
            'std_grad_norm': [x[1] for x in grad_norms_log],
            'grad_variance': grad_variances_log if grad_variances_log else [0] * len(grad_norms_log),
            'cosine_similarity': cosine_similarities_log if cosine_similarities_log else [0] * len(grad_norms_log),
            'weight_change': weight_changes_log if weight_changes_log else [0] * len(grad_norms_log)
        })
        gradient_metrics_df.to_csv(f"{result_dir}gradient_stability_metrics_run{run_id+1}_{safe_model_name}_{method}.csv", index=False)

        # 그래디언트 안정성 지표 시각화
        plt.figure(figsize=(15, 10))

        # 그래디언트 노름 플롯
        plt.subplot(2, 2, 1)
        plt.errorbar(
            range(1, len(grad_norms_log) + 1),
            [x[0] for x in grad_norms_log],
            yerr=[x[1] for x in grad_norms_log],
            fmt='o-',
            capsize=5
        )
        plt.title('Gradient Norm per Epoch')
        plt.xlabel('Epoch')
        plt.ylabel('Gradient Norm')
        plt.grid(True)

        # 그래디언트 방향 일관성 플롯
        plt.subplot(2, 2, 2)
        plt.plot(range(1, len(cosine_similarities_log) + 1), cosine_similarities_log, 'o-')
        plt.title('Gradient Direction Consistency (Cosine Similarity)')
        plt.xlabel('Epoch')
        plt.ylabel('Cosine Similarity')
        plt.grid(True)
        plt.ylim(-1, 1)

        # 가중치 변화량 플롯
        plt.subplot(2, 2, 3)
        plt.plot(range(1, len(weight_changes_log) + 1), weight_changes_log, 'o-')
        plt.title('Weight Change Magnitude')
        plt.xlabel('Epoch')
        plt.ylabel('Weight Change')
        plt.grid(True)

        # 학습 손실과 검증 손실 플롯
        plt.subplot(2, 2, 4)
        plt.plot(range(1, len(train_loss_log) + 1), train_loss_log, 'o-', label='Train Loss')
        plt.plot(range(1, len(valid_loss_log) + 1), valid_loss_log, 'o-', label='Valid Loss')
        plt.title('Training and Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)

        plt.tight_layout()
        plt.savefig(f"{result_dir}gradient_stability_metrics_run{run_id+1}_{safe_model_name}_{method}.png")
        plt.close()

    # 최고 성능 모델로 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Run {run_id+1}: Restored best model for final evaluation")

    print(f'----- Run {run_id+1}: Final Evaluation on Test Data -----')
    test_result_file = f"{result_dir}test_result_run{run_id+1}_{safe_model_name}_{method}.csv"
    test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, _ = eval_model(
        model, test_dataloader, device, num_cls,
        data_source=test_data,
        phase="test",
        result_file=test_result_file
    )
    loss_log_df = pd.DataFrame({'train_loss_log': train_loss_log,'valid_loss_log': valid_loss_log})
    loss_log_df.to_csv(f"{result_dir}loss_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8')

    # 그래디언트 안정성 지표도 함께 반환
    gradient_stability_metrics = {
        'grad_norms': grad_norms_log,
        'grad_variances': grad_variances_log,
        'cosine_similarities': cosine_similarities_log,
        'weight_changes': weight_changes_log
    } if track_gradient_stability else None

    return train_loss_log, valid_loss_log, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score, gradient_stability_metrics

# seed 추가
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True) # 추가


def run_multiple_experiments(
    model_class,
    model_name,
    train_data,
    val_data,
    test_data,
    tokenizer,
    device,
    num_runs=3,
    learning_rate=3e-5,
    num_epochs=5,
    batch_size=16,
    accumulation_steps=2,
    num_cls=2,
    seed=42,
    auroc_weight=0.0,  # AUROC 가중치 추가
    track_gradient_stability=True  # 그래디언트 안정성 추적 여부
):


    # seed=42 설정
    set_seed(seed)

    # 결과 저장을 위한 리스트
    all_test_metrics = []
    best_overall_score = 0.0  # 전체 실험에서 최고 결합 점수
    best_overall_auroc = 0.0  # 최고 모델의 AUROC
    best_overall_tpr = 0.0    # 최고 모델의 TPR
    best_overall_model_state = None
    best_run_id = -1

    # 그래디언트 안정성 측정 결과 저장
    all_gradient_metrics = []

    # 데이터로더 생성
    train_dataset = data_preparing(train_data, tokenizer, batch_size=batch_size, shuffle=True, return_dataset=True)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    valid_dataloader = data_preparing(val_data, tokenizer, batch_size=batch_size, shuffle=False)
    test_dataloader = data_preparing(test_data, tokenizer, batch_size=batch_size, shuffle=False)

    print(f"\nUsing combined metric with AUROC weight: {auroc_weight:.2f}, TPR@FPR=0.01% weight: {(1-auroc_weight):.2f}")

    for run in range(num_runs):
        print(f"\n===== Starting Run {run+1}/{num_runs} =====")

        # 새로운 모델 인스턴스 생성
        model = model_class.from_pretrained(model_name, num_labels=num_cls)

        # 모델 학습 - 원본 데이터 전달
        _, _, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score, gradient_metrics = train_model(
            model=model,
            train_dataloader=train_dataloader,
            valid_dataloader=valid_dataloader,
            test_dataloader=test_dataloader,
            train_data=train_data,
            val_data=val_data,
            test_data=test_data,
            device=device,
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            accumulation_steps=accumulation_steps,
            num_cls=num_cls,
            run_id=run,
            track_gradient_stability=track_gradient_stability
        )

        # 그래디언트 안정성 측정 결과 저장
        if track_gradient_stability and gradient_metrics is not None:
            all_gradient_metrics.append(gradient_metrics)

        # 테스트 메트릭 저장
        run_metrics = {
            'loss': test_loss,
            'accuracy': test_acc,
            'f1': test_f1,
            'precision': test_precision,
            'recall': test_recall,
            'auc': test_auc,
            'tpr_at_fpr_0001': test_tpr,
            'valid_auroc': best_valid_auroc,
            'valid_tpr': best_valid_tpr,
            'combined_score': best_combined_score
        }
        all_test_metrics.append(run_metrics)

        # Update best overall model based on combined score
        if best_combined_score > best_overall_score:
            best_overall_score = best_combined_score
            best_overall_auroc = best_valid_auroc
            best_overall_tpr = best_valid_tpr
            best_overall_model_state = best_model_state
            best_run_id = run

            # best model 저장
            torch.save(best_model_state, f"{result_dir}best_model_run_{run+1}_score_{best_combined_score:.4f}_auroc_{best_valid_auroc:.4f}_tpr_{best_valid_tpr:.4f}_{safe_model_name}_{method}.pt")
            print(f"New overall best model saved from run {run+1}")

    # 그래디언트 안정성 지표 분석
    if track_gradient_stability and all_gradient_metrics:
        # 모든 실행에서의 그래디언트 안정성 지표 평균 계산
        avg_grad_norms = []
        avg_cosine_sims = []
        avg_weight_changes = []

        for run_idx, metrics in enumerate(all_gradient_metrics):
            grad_norms = [x[0] for x in metrics['grad_norms']]  # 평균 그래디언트 노름만 사용
            cosine_sims = metrics['cosine_similarities']
            weight_changes = metrics['weight_changes']

            avg_grad_norms.append(np.mean(grad_norms))
            avg_cosine_sims.append(np.mean(cosine_sims) if cosine_sims else 0)
            avg_weight_changes.append(np.mean(weight_changes) if weight_changes else 0)

        print("\n===== Gradient Stability Analysis Across All Runs =====")
        print(f"Avg Gradient Norm: {np.mean(avg_grad_norms):.4f} ± {np.std(avg_grad_norms):.4f}")
        print(f"Avg Gradient Direction Consistency: {np.mean(avg_cosine_sims):.4f} ± {np.std(avg_cosine_sims):.4f}")
        print(f"Avg Weight Change: {np.mean(avg_weight_changes):.4f} ± {np.std(avg_weight_changes):.4f}")

        # 그래디언트 안정성 지표 요약 저장
        gradient_summary_df = pd.DataFrame({
            'run': range(1, len(all_gradient_metrics) + 1),
            'avg_grad_norm': avg_grad_norms,
            'avg_cosine_similarity': avg_cosine_sims,
            'avg_weight_change': avg_weight_changes
        })
        gradient_summary_df.to_csv(f"{result_dir}gradient_stability_summary_{safe_model_name}_{method}.csv", index=False)

    # 평균 성능 계산
    avg_metrics = {metric: np.mean([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}
    std_metrics = {metric: np.std([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}

    print("\n===== Average Test Results Across All Runs =====")
    print(f"Avg Test Loss: {avg_metrics['loss']:.4f} ± {std_metrics['loss']:.4f}")
    print(f"Avg Test Accuracy: {avg_metrics['accuracy']:.4f} ± {std_metrics['accuracy']:.4f}")
    print(f"Avg Test F1 Score: {avg_metrics['f1']:.4f} ± {std_metrics['f1']:.4f}")
    print(f"Avg Test Precision: {avg_metrics['precision']:.4f} ± {std_metrics['precision']:.4f}")
    print(f"Avg Test Recall: {avg_metrics['recall']:.4f} ± {std_metrics['recall']:.4f}")
    print(f"Avg Test AUROC: {avg_metrics['auc']:.4f} ± {std_metrics['auc']:.4f}")
    print(f"Avg Test TPR @ FPR=0.01%: {avg_metrics['tpr_at_fpr_0001']:.4f} ± {std_metrics['tpr_at_fpr_0001']:.4f}")

    # 평균 성능 저장
    summary_df = pd.DataFrame({
      'metric': list(avg_metrics.keys()),
      'mean': [avg_metrics[m] for m in avg_metrics],
      'std': [std_metrics[m] for m in std_metrics]
    })
    summary_file_path = os.path.join(result_dir, f"summary_metrics_{safe_model_name}_{method}.csv")
    summary_df.to_csv(summary_file_path, index=False, encoding='utf-8')

    # best overall model 저장
    torch.save(best_overall_model_state, f"{result_dir}best_model_overall_{safe_model_name}_{method}.pt")
    print(f"Best overall model saved as: best_model_overall_{safe_model_name}_{method}.pt")

    return all_test_metrics, avg_metrics, std_metrics, best_overall_model_state, best_run_id


def main():
   # 데이터 분할 (7:1.5:1.5)

    prompt_list = rs_sample['info.essay_prompt'].unique().tolist()
    train_prompts, temp_prompts = train_test_split(prompt_list,
                                                   test_size=0.3,
                                                   random_state=42,
                                                   )
    val_prompts, test_prompts = train_test_split(temp_prompts,
                                                 test_size=0.5,
                                                 random_state=42,
                                                 )
    train_data = rs_sample[rs_sample['info.essay_prompt'].isin(train_prompts)].reset_index(drop=True)
    val_data = rs_sample[rs_sample['info.essay_prompt'].isin(val_prompts)].reset_index(drop=True)
    test_data = rs_sample[rs_sample['info.essay_prompt'].isin(test_prompts)].reset_index(drop=True)



    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    auroc_weight = 0.0  # 이 값을 조정하여 AUROC의 중요도 설정 (0~1 사이 값)
    track_gradient_stability = True

    all_test_metrics, avg_metrics, std_metrics, best_model_state, best_run_id, all_gradient_metrics = run_multiple_experiments(
        model_class=model_class,
        model_name=model_name,
        train_data=train_data,
        val_data=val_data,
        test_data=test_data,
        tokenizer=tokenizer,
        device=device,
        num_runs=3,
        learning_rate=3e-5,
        num_epochs=5,
        batch_size=16,
        accumulation_steps=2,
        seed=42,
        auroc_weight=auroc_weight,  # AUROC 가중치 전달
        track_gradient_stability=track_gradient_stability)


    # Final evaluation with best model
    print("\n===== Final Evaluation with Best Overall Model =====")
    final_model = model_class.from_pretrained(model_name, num_labels=2)
    final_model.load_state_dict(best_model_state)
    final_model.to(device)

    test_dataloader = data_preparing(test_data, tokenizer, batch_size=16, shuffle=False)
    _, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr,_ = eval_model(final_model, test_dataloader, device)


start_time = time.time()

if __name__ == "__main__":
    main()

elapsed_time_sec = time.time() - start_time
elapsed_time_min = elapsed_time_sec / 60
print(f"소요 시간: {elapsed_time_min:.2f}분")


NameError: name 'model_name' is not defined

In [ ]:
# robert small
# baseline

result_dir = f"/content/drive/My Drive/cdf/"

safe_model_name = model_name.split('/')[-1] if '/' in model_name else model_name
method = "pure"

class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.texts = dataframe['txt'].tolist()
        self.labels = dataframe['target'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': tokenized['input_ids'].squeeze(0),  # 차원 축소 유지
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)  # 데이터 타입 명확히 설정
        }


def data_preparing(data_src, tokenizer, batch_size=16, shuffle=True, return_dataset=False):
    dataset = TextDataset(data_src, tokenizer)

    if return_dataset:
        return dataset

    dataloader = DataLoader(
        dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )
    return dataloader


def eval_model(
    model,
    eval_dataloader,
    device,
    num_cls=2,
    data_source=None,
    phase=None,
    wrong_predictions_list=None,
    result_file=None  # 추가
):
    total_loss = 0.0
    total_count = 0
    all_preds, all_labels, all_probs = [], [], []

    model.eval()
    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss
            total_loss += loss.item() * batch['labels'].size(0)
            total_count += batch['labels'].size(0)

            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = torch.argmax(logits, dim=-1)
            labels = batch['labels']

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = total_loss / total_count
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs) if not np.all(all_probs == all_probs[0]) else 0.0

    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    tpr_at_fpr_0001 = np.interp(0.01 / 100, fpr, tpr) if len(fpr) > 0 else 0.0

    print(f"Loss: {avg_loss:.4f} | Acc: {accuracy:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | AUROC: {auc:.4f} | TPR@FPR=0.01%: {tpr_at_fpr_0001:.4f}")

    if data_source is not None and 'txt' in data_source:
        txt_data = [text for text in data_source['txt']]
    else:
        txt_data = []

    # 결과 저장
    if result_file is not None:
        df_result = pd.DataFrame({
            'label': all_labels,
            'txt' : txt_data ,
            'pred': all_preds,
            'prob': all_probs
        })
        df_result.to_csv(result_file, index=False, encoding='utf-8-sig')

    return avg_loss, accuracy, f1, precision, recall, auc, tpr_at_fpr_0001, (fpr, tpr)



def train_model(
    model,
    train_dataloader,
    valid_dataloader,
    test_dataloader,
    device,
    learning_rate=3e-5,
    num_epochs=5,
    accumulation_steps=2,
    num_cls=2,
    run_id=0,
    early_stopping_patience=2,
    train_data=None,
    val_data=None,
    test_data=None,
    auroc_weight=0.0,
    result_file=None
):
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    model.to(device)

    print(f'-+-+-+ Run {run_id+1}: Initial Validation Metrics +-+-+-')
    eval_model(model, valid_dataloader, device, num_cls, data_source=val_data, phase="initial_val")
    print('-+-+-+ END OF INITIAL EVALUATION +-+-+-')

    train_loss_log = []
    valid_loss_log = []
    best_combined_score = 0.0  # 최고 결합 점수 초기화
    best_model_state = None
    early_stop_counter = 0
    all_wrong_predictions = []

    # 최적 성능 기록용
    best_valid_auroc = 0.0
    best_valid_tpr = 0.0

    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_count = 0
        phase_name = f"val_epoch{epoch+1}"
        valid_result_file = f"{result_dir}valid_result_run{run_id+1}_epoch{epoch+1}_{safe_model_name}_{method}.csv"
        model.train()  # 에폭 시작 시 그래디언트 초기화

        for step, batch in enumerate(train_dataloader):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss / accumulation_steps
            total_train_loss += loss.item()
            total_count += 1

            loss.backward()

            if (step + 1) % accumulation_steps == 0  or (step + 1) == len(train_dataloader) :
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()


        avg_train_loss = total_train_loss / total_count
        train_loss_log.append(avg_train_loss)
        if val_data is None:
            raise ValueError("Validation data is None.")
        if test_data is None:
            raise ValueError("Test data is None.")

        print(f'Run {run_id+1}, Epoch {epoch + 1} 성능 평가:')
        # 현재 에폭에서 검증 데이터 평가 및 틀린 예측 저장
        phase_name = f"val_epoch{epoch+1}"

        avg_valid_loss, valid_acc, valid_f1, valid_precision, valid_recall, valid_auc, valid_tpr, _ = eval_model(
            model, valid_dataloader, device, num_cls,
            data_source=val_data,
            phase=phase_name,
            result_file=valid_result_file
        )
        valid_loss_log.append(avg_valid_loss)


       # AUROC와 TPR@FPR=0.01%를 결합한 점수 계산
        combined_score = auroc_weight * valid_auc + (1 - auroc_weight) * valid_tpr

        # Save the best model based on combined score
        if combined_score > best_combined_score:
            best_combined_score = combined_score
            best_valid_auroc = valid_auc  # 최고 성능일 때 AUROC 값 저장
            best_valid_tpr = valid_tpr    # 최고 성능일 때 TPR 값 저장
            best_model_state = model.state_dict().copy()
            early_stop_counter = 0
            print(f"Run {run_id+1}: New best model found with combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {valid_auc:.4f} (weight: {auroc_weight:.2f})")
            print(f"  - TPR @ FPR=0.01%: {valid_tpr:.4f} (weight: {(1-auroc_weight):.2f})")
        else:
            early_stop_counter += 1
            print(f"Run {run_id+1}: No improvement. Early stop counter = {early_stop_counter}/{early_stopping_patience}")

        # Early stopping 조건 만족 시 중단
        if early_stop_counter >= early_stopping_patience:
            print(f"Run {run_id+1}: Early stopping triggered at epoch {epoch + 1}")
            break

    # 최고 성능 모델로 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Run {run_id+1}: Restored best model for final evaluation")


    print(f'----- Run {run_id+1}: Final Evaluation on Test Data -----')
    test_result_file = f"{result_dir}test_result_run{run_id+1}_{safe_model_name}_{method}.csv"
    test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, _ = eval_model(
        model, test_dataloader, device, num_cls,
        data_source=test_data,
        phase="test",
        result_file=test_result_file
    )
    loss_log_df = pd.DataFrame({'train_loss_log': train_loss_log,'valid_loss_log': valid_loss_log})
    loss_log_df.to_csv(f"{result_dir}loss_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8')


    return train_loss_log, valid_loss_log, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score

# seed 추가
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True) # 추가


def run_multiple_experiments(
    model_class,
    model_name,
    train_data,
    val_data,
    test_data,
    tokenizer,
    device,
    num_runs=3,
    learning_rate=3e-5,
    num_epochs=5,
    batch_size=16,
    accumulation_steps=2,
    num_cls=2,
    seed=42,
    auroc_weight=0.0  # AUROC 가중치 추가
):
    # seed=42 설정
    set_seed(seed)

    # 결과 저장을 위한 리스트
    all_test_metrics = []
    best_overall_score = 0.0  # 전체 실험에서 최고 결합 점수
    best_overall_auroc = 0.0  # 최고 모델의 AUROC
    best_overall_tpr = 0.0    # 최고 모델의 TPR
    best_overall_model_state = None
    best_run_id = -1

    # 데이터로더 생성
    train_dataset = data_preparing(train_data, tokenizer, batch_size=batch_size, shuffle=True, return_dataset=True)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    valid_dataloader = data_preparing(val_data, tokenizer, batch_size=batch_size, shuffle=False)
    test_dataloader = data_preparing(test_data, tokenizer, batch_size=batch_size, shuffle=False)

    print(f"\nUsing combined metric with AUROC weight: {auroc_weight:.2f}, TPR@FPR=0.01% weight: {(1-auroc_weight):.2f}")

    for run in range(num_runs):
        print(f"\n===== Starting Run {run+1}/{num_runs} =====")

        # 새로운 모델 인스턴스 생성
        model = model_class.from_pretrained(model_name, num_labels=num_cls)

        # 모델 학습 - 원본 데이터 전달
        _, _, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score= train_model(
            model=model,
            train_dataloader=train_dataloader,
            valid_dataloader=valid_dataloader,
            test_dataloader=test_dataloader,
            train_data=train_data,
            val_data=val_data,
            test_data=test_data,
            device=device,
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            accumulation_steps=accumulation_steps,
            num_cls=num_cls,
            run_id=run
        )

        # 테스트 메트릭 저장
        run_metrics = {
            'loss': test_loss,
            'accuracy': test_acc,
            'f1': test_f1,
            'precision': test_precision,
            'recall': test_recall,
            'auc': test_auc,
            'tpr_at_fpr_0001': test_tpr,
            'valid_auroc': best_valid_auroc,
            'valid_tpr': best_valid_tpr,
            'combined_score': best_combined_score
        }
        all_test_metrics.append(run_metrics)

        # Update best overall model based on combined score
        if best_combined_score > best_overall_score:
            best_overall_score = best_combined_score
            best_overall_auroc = best_valid_auroc
            best_overall_tpr = best_valid_tpr
            best_overall_model_state = best_model_state
            best_run_id = run

            # best model 저장
            torch.save(best_model_state, f"{result_dir}best_model_run_{run+1}_score_{best_combined_score:.4f}_auroc_{best_valid_auroc:.4f}_tpr_{best_valid_tpr:.4f}_{safe_model_name}_{method}.pt")
            print(f"New overall best model saved from run {run+1}")
            print(f"  - Combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {best_valid_auroc:.4f}")
            print(f"  - TPR @ FPR=0.01%: {best_valid_tpr:.4f}")

    # 평균 성능 계산
    avg_metrics = {metric: np.mean([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}
    std_metrics = {metric: np.std([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}

    print("\n===== Average Test Results Across All Runs =====")
    print(f"Avg Test Loss: {avg_metrics['loss']:.4f} ± {std_metrics['loss']:.4f}")
    print(f"Avg Test Accuracy: {avg_metrics['accuracy']:.4f} ± {std_metrics['accuracy']:.4f}")
    print(f"Avg Test F1 Score: {avg_metrics['f1']:.4f} ± {std_metrics['f1']:.4f}")
    print(f"Avg Test Precision: {avg_metrics['precision']:.4f} ± {std_metrics['precision']:.4f}")
    print(f"Avg Test Recall: {avg_metrics['recall']:.4f} ± {std_metrics['recall']:.4f}")
    print(f"Avg Test AUROC: {avg_metrics['auc']:.4f} ± {std_metrics['auc']:.4f}")
    print(f"Avg Test TPR @ FPR=0.01%: {avg_metrics['tpr_at_fpr_0001']:.4f} ± {std_metrics['tpr_at_fpr_0001']:.4f}")
    print(f"Avg Validation AUROC: {avg_metrics['valid_auroc']:.4f} ± {std_metrics['valid_auroc']:.4f}")
    print(f"Avg Validation TPR @ FPR=0.01%: {avg_metrics['valid_tpr']:.4f} ± {std_metrics['valid_tpr']:.4f}")
    print(f"Avg Combined Score: {avg_metrics['combined_score']:.4f} ± {std_metrics['combined_score']:.4f}")

    print(f"\n===== Best Model from Run {best_run_id+1} =====")
    print(f"Combined Score: {best_overall_score:.4f}")
    print(f"Best Validation AUROC: {best_overall_auroc:.4f}")
    print(f"Best Validation TPR: {best_overall_tpr:.4f}")
    print(f"Saved as: best_model_run_{best_run_id+1}_score_{best_overall_score:.4f}_auroc_{best_overall_auroc:.4f}_tpr_{best_overall_tpr:.4f}_{safe_model_name}_{method}.pt")

    # 평균 성능 저장
    summary_df = pd.DataFrame({
      'metric': list(avg_metrics.keys()),
      'mean': [avg_metrics[m] for m in avg_metrics],
      'std': [std_metrics[m] for m in std_metrics]
    })
    summary_file_path = os.path.join(result_dir, f"summary_metrics_{safe_model_name}_{method}.csv")
    summary_df.to_csv(summary_file_path, index=False, encoding='utf-8')

    # best overall model 저장
    torch.save(best_overall_model_state, f"{result_dir}best_model_overall_{safe_model_name}_{method}.pt")
    print(f"Best overall model saved as: best_model_overall_{safe_model_name}_{method}.pt")

    return all_test_metrics, avg_metrics, std_metrics, best_overall_model_state, best_run_id


def main():
   # 데이터 분할 (7:1.5:1.5)

    prompt_list = rs_sample['info.essay_prompt'].unique().tolist()
    train_prompts, temp_prompts = train_test_split(prompt_list,
                                                   test_size=0.3,
                                                   random_state=42,
                                                   )
    val_prompts, test_prompts = train_test_split(temp_prompts,
                                                 test_size=0.5,
                                                 random_state=42,
                                                 )
    train_data = rs_sample[rs_sample['info.essay_prompt'].isin(train_prompts)].reset_index(drop=True)
    val_data = rs_sample[rs_sample['info.essay_prompt'].isin(val_prompts)].reset_index(drop=True)
    test_data = rs_sample[rs_sample['info.essay_prompt'].isin(test_prompts)].reset_index(drop=True)



    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    auroc_weight = 0.0  # 이 값을 조정하여 AUROC의 중요도 설정 (0~1 사이 값)

    all_test_metrics, avg_metrics, std_metrics, best_model_state, best_run_id = run_multiple_experiments(
        model_class=model_class,
        model_name=model_name,
        train_data=train_data,
        val_data=val_data,
        test_data=test_data,
        tokenizer=tokenizer,
        device=device,
        num_runs=3,
        learning_rate=3e-5,
        num_epochs=5,
        batch_size=16,
        accumulation_steps=2,
        seed=42,
        auroc_weight=auroc_weight  # AUROC 가중치 전달
    )

    # Final evaluation with best model
    print("\n===== Final Evaluation with Best Overall Model =====")
    final_model = model_class.from_pretrained(model_name, num_labels=2)
    final_model.load_state_dict(best_model_state)
    final_model.to(device)

    test_dataloader = data_preparing(test_data, tokenizer, batch_size=16, shuffle=False)
    _, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr,_ = eval_model(final_model, test_dataloader, device)


start_time = time.time()

if __name__ == "__main__":
    main()

elapsed_time_sec = time.time() - start_time
elapsed_time_min = elapsed_time_sec / 60
print(f"소요 시간: {elapsed_time_min:.2f}분")


Using combined metric with AUROC weight: 0.00, TPR@FPR=0.01% weight: 1.00

===== Starting Run 1/3 =====


config.json:   0%|          | 0.00/545 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/273M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 1: Initial Validation Metrics +-+-+-
Loss: 0.6854 | Acc: 0.6372 | F1: 0.5879 | Precision: 0.5291 | Recall: 0.6614 | AUROC: 0.6909 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 1, Epoch 1 성능 평가:
Loss: 0.0133 | Acc: 0.9940 | F1: 0.9923 | Precision: 0.9993 | Recall: 0.9855 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9557
Run 1: New best model found with combined score: 0.9557
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9557 (weight: 1.00)
Run 1, Epoch 2 성능 평가:
Loss: 0.0024 | Acc: 0.9992 | F1: 0.9990 | Precision: 0.9979 | Recall: 1.0000 | AUROC: 1.0000 | TPR@FPR=0.01%: 1.0000
Run 1: New best model found with combined score: 1.0000
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 1.0000 (weight: 1.00)
Run 1, Epoch 3 성능 평가:
Loss: 0.0530 | Acc: 0.9865 | F1: 0.9830 | Precision: 0.9665 | Recall: 1.0000 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9953
Run 1: No improvement. Early stop counter = 1/2
Run 1, Epoch 4 성능 평가:
Loss: 0.0033 | Acc: 0.9989 | F1: 0.9986 | Pr

...

-+-+-+ Run 2: Initial Validation Metrics +-+-+-
Loss: 0.7058 | Acc: 0.3178 | F1: 0.3176 | Precision: 0.2609 | Recall: 0.4058 | AUROC: 0.2430 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 2, Epoch 1 성능 평가:
Loss: 0.1080 | Acc: 0.9648 | F1: 0.9569 | Precision: 0.9174 | Recall: 1.0000 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9529
Run 2: New best model found with combined score: 0.9529
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9529 (weight: 1.00)
Run 2, Epoch 2 성능 평가:
Loss: 0.1358 | Acc: 0.9607 | F1: 0.9522 | Precision: 0.9087 | Recall: 1.0000 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9952
Run 2: New best model found with combined score: 0.9952
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9952 (weight: 1.00)
Run 2, Epoch 3 성능 평가:
Loss: 0.0077 | Acc: 0.9981 | F1: 0.9976 | Precision: 0.9952 | Recall: 1.0000 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9972
Run 2: New best model found with combined score: 0.9972
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.99

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 3: Initial Validation Metrics +-+-+-
Loss: 0.7125 | Acc: 0.3912 | F1: 0.5624 | Precision: 0.3912 | Recall: 1.0000 | AUROC: 0.6180 | TPR@FPR=0.01%: 0.0208
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 3, Epoch 1 성능 평가:
Loss: 0.0079 | Acc: 0.9981 | F1: 0.9976 | Precision: 0.9952 | Recall: 1.0000 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9688
Run 3: New best model found with combined score: 0.9688
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9688 (weight: 1.00)
Run 3, Epoch 2 성능 평가:
Loss: 0.0069 | Acc: 0.9978 | F1: 0.9972 | Precision: 0.9959 | Recall: 0.9986 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9813
Run 3: New best model found with combined score: 0.9813
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9813 (weight: 1.00)
Run 3, Epoch 3 성능 평가:
Loss: 0.0039 | Acc: 0.9989 | F1: 0.9986 | Precision: 0.9972 | Recall: 1.0000 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9965
Run 3: New best model found with combined score: 0.9965
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.99

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loss: 0.0103 | Acc: 0.9962 | F1: 0.9953 | Precision: 0.9940 | Recall: 0.9966 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9866
소요 시간: 93.93분


In [ ]:
# static
result_dir = f"/content/drive/My Drive/cdf/"

safe_model_name = model_name.split('/')[-1] if '/' in model_name else model_name
method = "static" #static

class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.texts = dataframe['txt'].tolist()
        self.labels = dataframe['target'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': tokenized['input_ids'].squeeze(0),  # 차원 축소 유지
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)  # 데이터 타입 명확히 설정
        }

def re_weighting_sequence(target_series, mode='static'):
    wrong_cnt = np.sum(target_series)*1.0
    right_cnt = len(target_series)*1.0 - wrong_cnt
    if wrong_cnt == 0:
        wrong_cnt = 1
    if mode == 'adaptive':
        res_series = [right_cnt*1.0 / wrong_cnt if e == 1 else 1 for e in target_series]
    elif mode == 'static':
        res_series = [2 if e == 1 else 1 for e in target_series]

    return res_series

def wrong_first_dataloader(model, tokenized_datasets, batch_size=16, num_cls=2, current_epoch=1, device=torch.device("cpu")):
    if current_epoch == 1:
        return DataLoader(tokenized_datasets, shuffle=True, batch_size=batch_size)

    eval_dataloader = DataLoader(tokenized_datasets, shuffle=False, batch_size=1)

    all_preds = []
    all_labels = []
    right_wrong_list = []

    model.eval()
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )


        predictions = torch.argmax(outputs.logits, dim=-1)
        all_preds.append(predictions.item())
        all_labels.append(batch["labels"].item())

        right_wrong_list.append(1 if predictions.item() != batch["labels"].item() else 0)

    # 샘플링 가중치 설정
    weights = re_weighting_sequence(right_wrong_list)

    # 가중치 길이가 맞는지 확인
    if len(weights) != len(tokenized_datasets):
        print(f"Warning: weights length {len(weights)} does not match dataset size {len(tokenized_datasets)}")

    sampler = WeightedRandomSampler(weights=weights, num_samples=len(tokenized_datasets), replacement=True)

    return DataLoader(tokenized_datasets, batch_size=batch_size, sampler=sampler)




def data_preparing(data_src, tokenizer, batch_size=16, shuffle=True, return_dataset=False):
    dataset = TextDataset(data_src, tokenizer)

    if return_dataset:
        return dataset

    dataloader = DataLoader(
        dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )
    return dataloader


def eval_model(
    model,
    eval_dataloader,
    device,
    num_cls=2,
    data_source=None,
    phase=None,
    wrong_predictions_list=None,
    result_file=None  # 추가
):
    total_loss = 0.0
    total_count = 0
    all_preds, all_labels, all_probs = [], [], []

    model.eval()
    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss
            total_loss += loss.item() * batch['labels'].size(0)
            total_count += batch['labels'].size(0)

            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = torch.argmax(logits, dim=-1)
            labels = batch['labels']

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = total_loss / total_count
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs) if not np.all(all_probs == all_probs[0]) else 0.0

    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    tpr_at_fpr_0001 = np.interp(0.01 / 100, fpr, tpr) if len(fpr) > 0 else 0.0

    print(f"Loss: {avg_loss:.4f} | Acc: {accuracy:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | AUROC: {auc:.4f} | TPR@FPR=0.01%: {tpr_at_fpr_0001:.4f}")

    if data_source is not None and 'txt' in data_source:
        txt_data = [text for text in data_source['txt']]
    else:
        txt_data = []

    # 결과 저장
    if result_file is not None:
        df_result = pd.DataFrame({
            'label': all_labels,
            'txt' : txt_data ,
            'pred': all_preds,
            'prob': all_probs
        })
        df_result.to_csv(result_file, index=False, encoding='utf-8-sig')

    return avg_loss, accuracy, f1, precision, recall, auc, tpr_at_fpr_0001, (fpr, tpr)


def train_model(
    model,
    train_dataloader,
    valid_dataloader,
    test_dataloader,
    device,
    learning_rate=3e-5,
    num_epochs=5,
    accumulation_steps=2,
    num_cls=2,
    run_id=0,
    early_stopping_patience=2,
    train_data=None,
    val_data=None,
    test_data=None,
    auroc_weight=0.0,
    result_file=None
):
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    model.to(device)
    print(f'-+-+-+ Run {run_id+1}: Initial Validation Metrics +-+-+-')
    eval_model(model, valid_dataloader, device, num_cls)
    print('-+-+-+ END OF INITIAL EVALUATION +-+-+-')


    train_loss_log = []
    valid_loss_log = []
    best_combined_score = 0.0  # 최고 결합 점수 초기화
    best_model_state = None
    early_stop_counter = 0
    all_wrong_predictions = []

    # 최적 성능 기록용
    best_valid_auroc = 0.0
    best_valid_tpr = 0.0

    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_count = 0
        phase_name = f"val_epoch{epoch+1}"
        valid_result_file = f"{result_dir}valid_result_run{run_id+1}_epoch{epoch+1}_{safe_model_name}_{method}.csv"
        model.train()  # 에폭 시작 시 그래디언트 초기화

        train_dataloader_current = wrong_first_dataloader(
            model,
            train_dataloader.dataset,
            batch_size=train_dataloader.batch_size,
            num_cls=num_cls,
            current_epoch=epoch+1,
            device=device
        )

        for step, batch in enumerate(train_dataloader_current) :
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss / accumulation_steps
            total_train_loss += loss.item()
            total_count += 1

            loss.backward()

            if (step + 1) % accumulation_steps == 0  or (step + 1) == len(train_dataloader) :
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()


        avg_train_loss = total_train_loss / total_count
        train_loss_log.append(avg_train_loss)
        if val_data is None:
            raise ValueError("Validation data is None.")
        if test_data is None:
            raise ValueError("Test data is None.")

        print(f'Run {run_id+1}, Epoch {epoch + 1} 성능 평가:')
        # 현재 에폭에서 검증 데이터 평가 및 틀린 예측 저장
        phase_name = f"val_epoch{epoch+1}"

        avg_valid_loss, valid_acc, valid_f1, valid_precision, valid_recall, valid_auc, valid_tpr, _ = eval_model(
            model, valid_dataloader, device, num_cls,
            data_source=val_data,
            phase=phase_name,
            result_file=valid_result_file
        )
        valid_loss_log.append(avg_valid_loss)



       # AUROC와 TPR@FPR=0.01%를 결합한 점수 계산
        combined_score = auroc_weight * valid_auc + (1 - auroc_weight) * valid_tpr

        # Save the best model based on combined score
        if combined_score > best_combined_score:
            best_combined_score = combined_score
            best_valid_auroc = valid_auc  # 최고 성능일 때 AUROC 값 저장
            best_valid_tpr = valid_tpr    # 최고 성능일 때 TPR 값 저장
            best_model_state = model.state_dict().copy()
            early_stop_counter = 0
            print(f"Run {run_id+1}: New best model found with combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {valid_auc:.4f} (weight: {auroc_weight:.2f})")
            print(f"  - TPR @ FPR=0.01%: {valid_tpr:.4f} (weight: {(1-auroc_weight):.2f})")
        else:
            early_stop_counter += 1
            print(f"Run {run_id+1}: No improvement. Early stop counter = {early_stop_counter}/{early_stopping_patience}")

        # Early stopping 조건 만족 시 중단
        if early_stop_counter >= early_stopping_patience:
            print(f"Run {run_id+1}: Early stopping triggered at epoch {epoch + 1}")
            break

    # 최고 성능 모델로 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Run {run_id+1}: Restored best model for final evaluation")


    print(f'----- Run {run_id+1}: Final Evaluation on Test Data -----')
    test_result_file = f"{result_dir}test_result_run{run_id+1}_{safe_model_name}_{method}.csv"
    test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, _ = eval_model(
        model, test_dataloader, device, num_cls,
        data_source=test_data,
        phase="test",
        result_file=test_result_file
    )
    loss_log_df = pd.DataFrame({'train_loss_log': train_loss_log,'valid_loss_log': valid_loss_log})
    loss_log_df.to_csv(f"{result_dir}loss_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8')


    return train_loss_log, valid_loss_log, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score

# seed 추가
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True) # 추가


def run_multiple_experiments(
    model_class,
    model_name,
    train_data,
    val_data,
    test_data,
    tokenizer,
    device,
    num_runs=3,
    learning_rate=3e-5,
    num_epochs=5,
    batch_size=16,
    accumulation_steps=2,
    num_cls=2,
    seed=42,
    auroc_weight=0.0  # AUROC 가중치 추가
):
    # seed=42 설정
    set_seed(seed)

    # 결과 저장을 위한 리스트
    all_test_metrics = []
    best_overall_score = 0.0  # 전체 실험에서 최고 결합 점수
    best_overall_auroc = 0.0  # 최고 모델의 AUROC
    best_overall_tpr = 0.0    # 최고 모델의 TPR
    best_overall_model_state = None
    best_run_id = -1

    # 데이터로더 생성
    train_dataset = data_preparing(train_data, tokenizer, batch_size=batch_size, shuffle=True, return_dataset=True)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    valid_dataloader = data_preparing(val_data, tokenizer, batch_size=batch_size, shuffle=False)
    test_dataloader = data_preparing(test_data, tokenizer, batch_size=batch_size, shuffle=False)

    print(f"\nUsing combined metric with AUROC weight: {auroc_weight:.2f}, TPR@FPR=0.01% weight: {(1-auroc_weight):.2f}")

    for run in range(num_runs):
        print(f"\n===== Starting Run {run+1}/{num_runs} =====")

        # 새로운 모델 인스턴스 생성
        model = model_class.from_pretrained(model_name, num_labels=num_cls)

        # 모델 학습 - 원본 데이터 전달
        _, _, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score= train_model(
            model=model,
            train_dataloader=train_dataloader,
            valid_dataloader=valid_dataloader,
            test_dataloader=test_dataloader,
            train_data=train_data,
            val_data=val_data,
            test_data=test_data,
            device=device,
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            accumulation_steps=accumulation_steps,
            num_cls=num_cls,
            run_id=run
        )

        # 테스트 메트릭 저장
        run_metrics = {
            'loss': test_loss,
            'accuracy': test_acc,
            'f1': test_f1,
            'precision': test_precision,
            'recall': test_recall,
            'auc': test_auc,
            'tpr_at_fpr_0001': test_tpr,
            'valid_auroc': best_valid_auroc,
            'valid_tpr': best_valid_tpr,
            'combined_score': best_combined_score
        }
        all_test_metrics.append(run_metrics)

        # Update best overall model based on combined score
        if best_combined_score > best_overall_score:
            best_overall_score = best_combined_score
            best_overall_auroc = best_valid_auroc
            best_overall_tpr = best_valid_tpr
            best_overall_model_state = best_model_state
            best_run_id = run

            # best model 저장
            torch.save(best_model_state, f"{result_dir}best_model_run_{run+1}_score_{best_combined_score:.4f}_auroc_{best_valid_auroc:.4f}_tpr_{best_valid_tpr:.4f}_{safe_model_name}_{method}.pt")
            print(f"New overall best model saved from run {run+1}")
            print(f"  - Combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {best_valid_auroc:.4f}")
            print(f"  - TPR @ FPR=0.01%: {best_valid_tpr:.4f}")

    # 평균 성능 계산
    avg_metrics = {metric: np.mean([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}
    std_metrics = {metric: np.std([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}

    print("\n===== Average Test Results Across All Runs =====")
    print(f"Avg Test Loss: {avg_metrics['loss']:.4f} ± {std_metrics['loss']:.4f}")
    print(f"Avg Test Accuracy: {avg_metrics['accuracy']:.4f} ± {std_metrics['accuracy']:.4f}")
    print(f"Avg Test F1 Score: {avg_metrics['f1']:.4f} ± {std_metrics['f1']:.4f}")
    print(f"Avg Test Precision: {avg_metrics['precision']:.4f} ± {std_metrics['precision']:.4f}")
    print(f"Avg Test Recall: {avg_metrics['recall']:.4f} ± {std_metrics['recall']:.4f}")
    print(f"Avg Test AUROC: {avg_metrics['auc']:.4f} ± {std_metrics['auc']:.4f}")
    print(f"Avg Test TPR @ FPR=0.01%: {avg_metrics['tpr_at_fpr_0001']:.4f} ± {std_metrics['tpr_at_fpr_0001']:.4f}")
    print(f"Avg Validation AUROC: {avg_metrics['valid_auroc']:.4f} ± {std_metrics['valid_auroc']:.4f}")
    print(f"Avg Validation TPR @ FPR=0.01%: {avg_metrics['valid_tpr']:.4f} ± {std_metrics['valid_tpr']:.4f}")
    print(f"Avg Combined Score: {avg_metrics['combined_score']:.4f} ± {std_metrics['combined_score']:.4f}")

    print(f"\n===== Best Model from Run {best_run_id+1} =====")
    print(f"Combined Score: {best_overall_score:.4f}")
    print(f"Best Validation AUROC: {best_overall_auroc:.4f}")
    print(f"Best Validation TPR: {best_overall_tpr:.4f}")
    print(f"Saved as: best_model_run_{best_run_id+1}_score_{best_overall_score:.4f}_auroc_{best_overall_auroc:.4f}_tpr_{best_overall_tpr:.4f}_{safe_model_name}_{method}.pt")

    # 평균 성능 저장
    summary_df = pd.DataFrame({
      'metric': list(avg_metrics.keys()),
      'mean': [avg_metrics[m] for m in avg_metrics],
      'std': [std_metrics[m] for m in std_metrics]
    })
    summary_file_path = os.path.join(result_dir, f"summary_metrics_{safe_model_name}_{method}.csv")
    summary_df.to_csv(summary_file_path, index=False, encoding='utf-8')

    # best overall model 저장
    torch.save(best_overall_model_state, f"{result_dir}best_model_overall_{safe_model_name}_{method}.pt")
    print(f"Best overall model saved as: best_model_overall_{safe_model_name}_{method}.pt")

    return all_test_metrics, avg_metrics, std_metrics, best_overall_model_state, best_run_id


def main():
   # 데이터 분할 (7:1.5:1.5)

    prompt_list = rs_sample['info.essay_prompt'].unique().tolist()
    train_prompts, temp_prompts = train_test_split(prompt_list,
                                                   test_size=0.3,
                                                   random_state=42,
                                                   )
    val_prompts, test_prompts = train_test_split(temp_prompts,
                                                 test_size=0.5,
                                                 random_state=42,
                                                 )
    train_data = rs_sample[rs_sample['info.essay_prompt'].isin(train_prompts)].reset_index(drop=True)
    val_data = rs_sample[rs_sample['info.essay_prompt'].isin(val_prompts)].reset_index(drop=True)
    test_data = rs_sample[rs_sample['info.essay_prompt'].isin(test_prompts)].reset_index(drop=True)



    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    auroc_weight = 0.0  # 이 값을 조정하여 AUROC의 중요도 설정 (0~1 사이 값)

    all_test_metrics, avg_metrics, std_metrics, best_model_state, best_run_id = run_multiple_experiments(
        model_class=model_class,
        model_name=model_name,
        train_data=train_data,
        val_data=val_data,
        test_data=test_data,
        tokenizer=tokenizer,
        device=device,
        num_runs=3,
        learning_rate=3e-5,
        num_epochs=5,
        batch_size=16,
        accumulation_steps=2,
        seed=42,
        auroc_weight=auroc_weight  # AUROC 가중치 전달
    )

    # Final evaluation with best model
    print("\n===== Final Evaluation with Best Overall Model =====")
    final_model = model_class.from_pretrained(model_name, num_labels=2)
    final_model.load_state_dict(best_model_state)
    final_model.to(device)

    test_dataloader = data_preparing(test_data, tokenizer, batch_size=16, shuffle=False)
    _, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr,_ = eval_model(final_model, test_dataloader, device)


start_time = time.time()

if __name__ == "__main__":
    main()

elapsed_time_sec = time.time() - start_time
elapsed_time_min = elapsed_time_sec / 60
print(f"소요 시간: {elapsed_time_min:.2f}분")


Using combined metric with AUROC weight: 0.00, TPR@FPR=0.01% weight: 1.00

===== Starting Run 1/3 =====


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 1: Initial Validation Metrics +-+-+-
Loss: 0.6854 | Acc: 0.6372 | F1: 0.5879 | Precision: 0.5291 | Recall: 0.6614 | AUROC: 0.6909 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 1, Epoch 1 성능 평가:
Loss: 0.0133 | Acc: 0.9940 | F1: 0.9923 | Precision: 0.9993 | Recall: 0.9855 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9557
Run 1: New best model found with combined score: 0.9557
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9557 (weight: 1.00)
Run 1, Epoch 2 성능 평가:
Loss: 0.0088 | Acc: 0.9976 | F1: 0.9969 | Precision: 0.9979 | Recall: 0.9958 | AUROC: 0.9999 | TPR@FPR=0.01%: 0.9037
Run 1: No improvement. Early stop counter = 1/2
Run 1, Epoch 3 성능 평가:
Loss: 0.0058 | Acc: 0.9981 | F1: 0.9976 | Precision: 0.9986 | Recall: 0.9965 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9543
Run 1: No improvement. Early stop counter = 2/2
Run 1: Early stopping triggered at epoch 3
Run 1: Restored best model for final evaluation
----- Run 1: Final Evaluation on Test Data -----
Loss: 0.00

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 2: Initial Validation Metrics +-+-+-
Loss: 0.6815 | Acc: 0.6088 | F1: 0.0000 | Precision: 0.0000 | Recall: 0.0000 | AUROC: 0.2768 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 2, Epoch 1 성능 평가:
Loss: 0.0084 | Acc: 0.9984 | F1: 0.9979 | Precision: 0.9993 | Recall: 0.9965 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9945
Run 2: New best model found with combined score: 0.9945
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9945 (weight: 1.00)
Run 2, Epoch 2 성능 평가:
Loss: 0.0024 | Acc: 0.9992 | F1: 0.9990 | Precision: 0.9986 | Recall: 0.9993 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9979
Run 2: New best model found with combined score: 0.9979
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9979 (weight: 1.00)
Run 2, Epoch 3 성능 평가:
Loss: 0.0023 | Acc: 0.9995 | F1: 0.9993 | Precision: 0.9993 | Recall: 0.9993 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9986
Run 2: New best model found with combined score: 0.9986
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.99

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 3: Initial Validation Metrics +-+-+-
Loss: 0.7120 | Acc: 0.3888 | F1: 0.5477 | Precision: 0.3854 | Recall: 0.9460 | AUROC: 0.4415 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 3, Epoch 1 성능 평가:
Loss: 0.0112 | Acc: 0.9965 | F1: 0.9955 | Precision: 0.9918 | Recall: 0.9993 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9945
Run 3: New best model found with combined score: 0.9945
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9945 (weight: 1.00)
Run 3, Epoch 2 성능 평가:
Loss: 0.0030 | Acc: 0.9995 | F1: 0.9993 | Precision: 0.9986 | Recall: 1.0000 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9931
Run 3: No improvement. Early stop counter = 1/2
Run 3, Epoch 3 성능 평가:
Loss: 0.0042 | Acc: 0.9984 | F1: 0.9979 | Precision: 0.9993 | Recall: 0.9965 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9910
Run 3: No improvement. Early stop counter = 2/2
Run 3: Early stopping triggered at epoch 3
Run 3: Restored best model for final evaluation
----- Run 3: Final Evaluation on Test Data -----
Loss: 0.01

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loss: 0.0189 | Acc: 0.9945 | F1: 0.9932 | Precision: 0.9986 | Recall: 0.9879 | AUROC: 0.9999 | TPR@FPR=0.01%: 0.9866
소요 시간: 101.98분


In [ ]:
# baseline

result_dir = f"/content/drive/My Drive/cdf/"

safe_model_name = model_name.split('/')[-1] if '/' in model_name else model_name
method = "pure"

class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.texts = dataframe['txt'].tolist()
        self.labels = dataframe['target'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': tokenized['input_ids'].squeeze(0),  # 차원 축소 유지
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)  # 데이터 타입 명확히 설정
        }


def data_preparing(data_src, tokenizer, batch_size=16, shuffle=True, return_dataset=False):
    dataset = TextDataset(data_src, tokenizer)

    if return_dataset:
        return dataset

    dataloader = DataLoader(
        dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )
    return dataloader


def eval_model(
    model,
    eval_dataloader,
    device,
    num_cls=2,
    data_source=None,
    phase=None,
    wrong_predictions_list=None,
    result_file=None  # 추가
):
    total_loss = 0.0
    total_count = 0
    all_preds, all_labels, all_probs = [], [], []

    model.eval()
    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss
            total_loss += loss.item() * batch['labels'].size(0)
            total_count += batch['labels'].size(0)

            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = torch.argmax(logits, dim=-1)
            labels = batch['labels']

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = total_loss / total_count
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs) if not np.all(all_probs == all_probs[0]) else 0.0

    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    tpr_at_fpr_0001 = np.interp(0.01 / 100, fpr, tpr) if len(fpr) > 0 else 0.0

    print(f"Loss: {avg_loss:.4f} | Acc: {accuracy:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | AUROC: {auc:.4f} | TPR@FPR=0.01%: {tpr_at_fpr_0001:.4f}")

    if data_source is not None and 'txt' in data_source:
        txt_data = [text for text in data_source['txt']]
    else:
        txt_data = []

    # 결과 저장
    if result_file is not None:
        df_result = pd.DataFrame({
            'label': all_labels,
            'txt' : txt_data ,
            'pred': all_preds,
            'prob': all_probs
        })
        df_result.to_csv(result_file, index=False, encoding='utf-8-sig')

    return avg_loss, accuracy, f1, precision, recall, auc, tpr_at_fpr_0001, (fpr, tpr)



def train_model(
    model,
    train_dataloader,
    valid_dataloader,
    test_dataloader,
    device,
    learning_rate=3e-5,
    num_epochs=5,
    accumulation_steps=2,
    num_cls=2,
    run_id=0,
    early_stopping_patience=2,
    train_data=None,
    val_data=None,
    test_data=None,
    auroc_weight=0.0,
    result_file=None
):
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    model.to(device)

    print(f'-+-+-+ Run {run_id+1}: Initial Validation Metrics +-+-+-')
    eval_model(model, valid_dataloader, device, num_cls, data_source=val_data, phase="initial_val")
    print('-+-+-+ END OF INITIAL EVALUATION +-+-+-')

    train_loss_log = []
    valid_loss_log = []
    best_combined_score = 0.0  # 최고 결합 점수 초기화
    best_model_state = None
    early_stop_counter = 0
    all_wrong_predictions = []

    # 최적 성능 기록용
    best_valid_auroc = 0.0
    best_valid_tpr = 0.0

    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_count = 0
        phase_name = f"val_epoch{epoch+1}"
        valid_result_file = f"{result_dir}valid_result_run{run_id+1}_epoch{epoch+1}_{safe_model_name}_{method}.csv"
        model.train()  # 에폭 시작 시 그래디언트 초기화

        for step, batch in enumerate(train_dataloader):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss / accumulation_steps
            total_train_loss += loss.item()
            total_count += 1

            loss.backward()

            if (step + 1) % accumulation_steps == 0  or (step + 1) == len(train_dataloader) :
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()


        avg_train_loss = total_train_loss / total_count
        train_loss_log.append(avg_train_loss)
        if val_data is None:
            raise ValueError("Validation data is None.")
        if test_data is None:
            raise ValueError("Test data is None.")

        print(f'Run {run_id+1}, Epoch {epoch + 1} 성능 평가:')
        # 현재 에폭에서 검증 데이터 평가 및 틀린 예측 저장
        phase_name = f"val_epoch{epoch+1}"

        avg_valid_loss, valid_acc, valid_f1, valid_precision, valid_recall, valid_auc, valid_tpr, _ = eval_model(
            model, valid_dataloader, device, num_cls,
            data_source=val_data,
            phase=phase_name,
            result_file=valid_result_file
        )
        valid_loss_log.append(avg_valid_loss)


       # AUROC와 TPR@FPR=0.01%를 결합한 점수 계산
        combined_score = auroc_weight * valid_auc + (1 - auroc_weight) * valid_tpr

        # Save the best model based on combined score
        if combined_score > best_combined_score:
            best_combined_score = combined_score
            best_valid_auroc = valid_auc  # 최고 성능일 때 AUROC 값 저장
            best_valid_tpr = valid_tpr    # 최고 성능일 때 TPR 값 저장
            best_model_state = model.state_dict().copy()
            early_stop_counter = 0
            print(f"Run {run_id+1}: New best model found with combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {valid_auc:.4f} (weight: {auroc_weight:.2f})")
            print(f"  - TPR @ FPR=0.01%: {valid_tpr:.4f} (weight: {(1-auroc_weight):.2f})")
        else:
            early_stop_counter += 1
            print(f"Run {run_id+1}: No improvement. Early stop counter = {early_stop_counter}/{early_stopping_patience}")

        # Early stopping 조건 만족 시 중단
        if early_stop_counter >= early_stopping_patience:
            print(f"Run {run_id+1}: Early stopping triggered at epoch {epoch + 1}")
            break

    # 최고 성능 모델로 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Run {run_id+1}: Restored best model for final evaluation")


    print(f'----- Run {run_id+1}: Final Evaluation on Test Data -----')
    test_result_file = f"{result_dir}test_result_run{run_id+1}_{safe_model_name}_{method}.csv"
    test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, _ = eval_model(
        model, test_dataloader, device, num_cls,
        data_source=test_data,
        phase="test",
        result_file=test_result_file
    )
    loss_log_df = pd.DataFrame({'train_loss_log': train_loss_log,'valid_loss_log': valid_loss_log})
    loss_log_df.to_csv(f"{result_dir}loss_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8')


    return train_loss_log, valid_loss_log, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score

# seed 추가
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True) # 추가


def run_multiple_experiments(
    model_class,
    model_name,
    train_data,
    val_data,
    test_data,
    tokenizer,
    device,
    num_runs=3,
    learning_rate=3e-5,
    num_epochs=5,
    batch_size=16,
    accumulation_steps=2,
    num_cls=2,
    seed=42,
    auroc_weight=0.0  # AUROC 가중치 추가
):
    # seed=42 설정
    set_seed(seed)

    # 결과 저장을 위한 리스트
    all_test_metrics = []
    best_overall_score = 0.0  # 전체 실험에서 최고 결합 점수
    best_overall_auroc = 0.0  # 최고 모델의 AUROC
    best_overall_tpr = 0.0    # 최고 모델의 TPR
    best_overall_model_state = None
    best_run_id = -1

    # 데이터로더 생성
    train_dataset = data_preparing(train_data, tokenizer, batch_size=batch_size, shuffle=True, return_dataset=True)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    valid_dataloader = data_preparing(val_data, tokenizer, batch_size=batch_size, shuffle=False)
    test_dataloader = data_preparing(test_data, tokenizer, batch_size=batch_size, shuffle=False)

    print(f"\nUsing combined metric with AUROC weight: {auroc_weight:.2f}, TPR@FPR=0.01% weight: {(1-auroc_weight):.2f}")

    for run in range(num_runs):
        print(f"\n===== Starting Run {run+1}/{num_runs} =====")

        # 새로운 모델 인스턴스 생성
        model = model_class.from_pretrained(model_name, num_labels=num_cls)

        # 모델 학습 - 원본 데이터 전달
        _, _, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score= train_model(
            model=model,
            train_dataloader=train_dataloader,
            valid_dataloader=valid_dataloader,
            test_dataloader=test_dataloader,
            train_data=train_data,
            val_data=val_data,
            test_data=test_data,
            device=device,
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            accumulation_steps=accumulation_steps,
            num_cls=num_cls,
            run_id=run
        )

        # 테스트 메트릭 저장
        run_metrics = {
            'loss': test_loss,
            'accuracy': test_acc,
            'f1': test_f1,
            'precision': test_precision,
            'recall': test_recall,
            'auc': test_auc,
            'tpr_at_fpr_0001': test_tpr,
            'valid_auroc': best_valid_auroc,
            'valid_tpr': best_valid_tpr,
            'combined_score': best_combined_score
        }
        all_test_metrics.append(run_metrics)

        # Update best overall model based on combined score
        if best_combined_score > best_overall_score:
            best_overall_score = best_combined_score
            best_overall_auroc = best_valid_auroc
            best_overall_tpr = best_valid_tpr
            best_overall_model_state = best_model_state
            best_run_id = run

            # best model 저장
            torch.save(best_model_state, f"{result_dir}best_model_run_{run+1}_score_{best_combined_score:.4f}_auroc_{best_valid_auroc:.4f}_tpr_{best_valid_tpr:.4f}_{safe_model_name}_{method}.pt")
            print(f"New overall best model saved from run {run+1}")
            print(f"  - Combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {best_valid_auroc:.4f}")
            print(f"  - TPR @ FPR=0.01%: {best_valid_tpr:.4f}")

    # 평균 성능 계산
    avg_metrics = {metric: np.mean([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}
    std_metrics = {metric: np.std([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}

    print("\n===== Average Test Results Across All Runs =====")
    print(f"Avg Test Loss: {avg_metrics['loss']:.4f} ± {std_metrics['loss']:.4f}")
    print(f"Avg Test Accuracy: {avg_metrics['accuracy']:.4f} ± {std_metrics['accuracy']:.4f}")
    print(f"Avg Test F1 Score: {avg_metrics['f1']:.4f} ± {std_metrics['f1']:.4f}")
    print(f"Avg Test Precision: {avg_metrics['precision']:.4f} ± {std_metrics['precision']:.4f}")
    print(f"Avg Test Recall: {avg_metrics['recall']:.4f} ± {std_metrics['recall']:.4f}")
    print(f"Avg Test AUROC: {avg_metrics['auc']:.4f} ± {std_metrics['auc']:.4f}")
    print(f"Avg Test TPR @ FPR=0.01%: {avg_metrics['tpr_at_fpr_0001']:.4f} ± {std_metrics['tpr_at_fpr_0001']:.4f}")
    print(f"Avg Validation AUROC: {avg_metrics['valid_auroc']:.4f} ± {std_metrics['valid_auroc']:.4f}")
    print(f"Avg Validation TPR @ FPR=0.01%: {avg_metrics['valid_tpr']:.4f} ± {std_metrics['valid_tpr']:.4f}")
    print(f"Avg Combined Score: {avg_metrics['combined_score']:.4f} ± {std_metrics['combined_score']:.4f}")

    print(f"\n===== Best Model from Run {best_run_id+1} =====")
    print(f"Combined Score: {best_overall_score:.4f}")
    print(f"Best Validation AUROC: {best_overall_auroc:.4f}")
    print(f"Best Validation TPR: {best_overall_tpr:.4f}")
    print(f"Saved as: best_model_run_{best_run_id+1}_score_{best_overall_score:.4f}_auroc_{best_overall_auroc:.4f}_tpr_{best_overall_tpr:.4f}_{safe_model_name}_{method}.pt")

    # 평균 성능 저장
    summary_df = pd.DataFrame({
      'metric': list(avg_metrics.keys()),
      'mean': [avg_metrics[m] for m in avg_metrics],
      'std': [std_metrics[m] for m in std_metrics]
    })
    summary_file_path = os.path.join(result_dir, f"summary_metrics_{safe_model_name}_{method}.csv")
    summary_df.to_csv(summary_file_path, index=False, encoding='utf-8')

    # best overall model 저장
    torch.save(best_overall_model_state, f"{result_dir}best_model_overall_{safe_model_name}_{method}.pt")
    print(f"Best overall model saved as: best_model_overall_{safe_model_name}_{method}.pt")

    return all_test_metrics, avg_metrics, std_metrics, best_overall_model_state, best_run_id


def main():
   # 데이터 분할 (7:1.5:1.5)

    prompt_list = rs_sample['info.essay_prompt'].unique().tolist()
    train_prompts, temp_prompts = train_test_split(prompt_list,
                                                   test_size=0.3,
                                                   random_state=42,
                                                   )
    val_prompts, test_prompts = train_test_split(temp_prompts,
                                                 test_size=0.5,
                                                 random_state=42,
                                                 )
    train_data = rs_sample[rs_sample['info.essay_prompt'].isin(train_prompts)].reset_index(drop=True)
    val_data = rs_sample[rs_sample['info.essay_prompt'].isin(val_prompts)].reset_index(drop=True)
    test_data = rs_sample[rs_sample['info.essay_prompt'].isin(test_prompts)].reset_index(drop=True)



    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    auroc_weight = 0.0  # 이 값을 조정하여 AUROC의 중요도 설정 (0~1 사이 값)

    all_test_metrics, avg_metrics, std_metrics, best_model_state, best_run_id = run_multiple_experiments(
        model_class=model_class,
        model_name=model_name,
        train_data=train_data,
        val_data=val_data,
        test_data=test_data,
        tokenizer=tokenizer,
        device=device,
        num_runs=3,
        learning_rate=3e-5,
        num_epochs=5,
        batch_size=16,
        accumulation_steps=2,
        seed=42,
        auroc_weight=auroc_weight  # AUROC 가중치 전달
    )

    # Final evaluation with best model
    print("\n===== Final Evaluation with Best Overall Model =====")
    final_model = model_class.from_pretrained(model_name, num_labels=2)
    final_model.load_state_dict(best_model_state)
    final_model.to(device)

    test_dataloader = data_preparing(test_data, tokenizer, batch_size=16, shuffle=False)
    _, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr,_ = eval_model(final_model, test_dataloader, device)


start_time = time.time()

if __name__ == "__main__":
    main()

elapsed_time_sec = time.time() - start_time
elapsed_time_min = elapsed_time_sec / 60
print(f"소요 시간: {elapsed_time_min:.2f}분")


Using combined metric with AUROC weight: 0.00, TPR@FPR=0.01% weight: 1.00

===== Starting Run 1/3 =====


config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 1: Initial Validation Metrics +-+-+-
Loss: 0.7471 | Acc: 0.3912 | F1: 0.5622 | Precision: 0.3912 | Recall: 0.9993 | AUROC: 0.5788 | TPR@FPR=0.01%: 0.0007
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 1, Epoch 1 성능 평가:
Loss: 0.0197 | Acc: 0.9935 | F1: 0.9917 | Precision: 0.9910 | Recall: 0.9924 | AUROC: 0.9997 | TPR@FPR=0.01%: 0.8940
Run 1: New best model found with combined score: 0.8940
  - AUROC: 0.9997 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.8940 (weight: 1.00)
Run 1, Epoch 2 성능 평가:
Loss: 0.0274 | Acc: 0.9946 | F1: 0.9930 | Precision: 0.9979 | Recall: 0.9882 | AUROC: 0.9999 | TPR@FPR=0.01%: 0.9529
Run 1: New best model found with combined score: 0.9529
  - AUROC: 0.9999 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9529 (weight: 1.00)
Run 1, Epoch 3 성능 평가:
Loss: 0.0275 | Acc: 0.9913 | F1: 0.9890 | Precision: 0.9790 | Recall: 0.9993 | AUROC: 0.9997 | TPR@FPR=0.01%: 0.5636
Run 1: No improvement. Early stop counter = 1/2
Run 1, Epoch 4 성능 평가:
Loss: 0.0423 | Acc: 0.9878 | F1: 0.9842 | Pr

...

-+-+-+ Run 2: Initial Validation Metrics +-+-+-
Loss: 0.7041 | Acc: 0.4191 | F1: 0.5635 | Precision: 0.3991 | Recall: 0.9584 | AUROC: 0.5678 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 2, Epoch 1 성능 평가:
Loss: 0.0293 | Acc: 0.9905 | F1: 0.9877 | Precision: 0.9986 | Recall: 0.9771 | AUROC: 0.9998 | TPR@FPR=0.01%: 0.7535
Run 2: New best model found with combined score: 0.7535
  - AUROC: 0.9998 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.7535 (weight: 1.00)
Run 2, Epoch 2 성능 평가:
Loss: 0.0301 | Acc: 0.9916 | F1: 0.9893 | Precision: 0.9829 | Recall: 0.9958 | AUROC: 0.9995 | TPR@FPR=0.01%: 0.6579
Run 2: No improvement. Early stop counter = 1/2
Run 2, Epoch 3 성능 평가:
Loss: 0.0154 | Acc: 0.9957 | F1: 0.9945 | Precision: 0.9911 | Recall: 0.9979 | AUROC: 0.9999 | TPR@FPR=0.01%: 0.9176
Run 2: New best model found with combined score: 0.9176
  - AUROC: 0.9999 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9176 (weight: 1.00)
Run 2, Epoch 4 성능 평가:
Loss: 0.0180 | Acc: 0.9962 | F1: 0.9952 | Pr

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 3: Initial Validation Metrics +-+-+-
Loss: 0.6855 | Acc: 0.5901 | F1: 0.5477 | Precision: 0.4819 | Recall: 0.6343 | AUROC: 0.6425 | TPR@FPR=0.01%: 0.0007
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 3, Epoch 1 성능 평가:
Loss: 0.0354 | Acc: 0.9881 | F1: 0.9850 | Precision: 0.9704 | Recall: 1.0000 | AUROC: 0.9999 | TPR@FPR=0.01%: 0.9432
Run 3: New best model found with combined score: 0.9432
  - AUROC: 0.9999 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9432 (weight: 1.00)
Run 3, Epoch 2 성능 평가:
Loss: 0.0098 | Acc: 0.9959 | F1: 0.9948 | Precision: 0.9904 | Recall: 0.9993 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9765
Run 3: New best model found with combined score: 0.9765
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9765 (weight: 1.00)
Run 3, Epoch 3 성능 평가:
Loss: 0.0152 | Acc: 0.9943 | F1: 0.9927 | Precision: 0.9951 | Recall: 0.9903 | AUROC: 0.9999 | TPR@FPR=0.01%: 0.9370
Run 3: No improvement. Early stop counter = 1/2
Run 3, Epoch 4 성능 평가:
Loss: 0.0237 | Acc: 0.9916 | F1: 0.9894 | Pr

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loss: 0.0902 | Acc: 0.9839 | F1: 0.9800 | Precision: 0.9904 | Recall: 0.9698 | AUROC: 0.9992 | TPR@FPR=0.01%: 0.9093
소요 시간: 172.05분


커리큘럼 방식 실험

In [ ]:
# predefined1


result_dir = f"/content/drive/My Drive/cdf/"
safe_model_name = model_name.split('/')[-1] if '/' in model_name else model_name
method = "predefined1"



class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.texts = dataframe['txt'].tolist()
        self.difficulty_score = dataframe['difficulty_score1'].tolist()  #difficulty_score1 difficulty_score2 difficulty_score3
        self.labels = dataframe['target'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': tokenized['input_ids'].squeeze(0),  # 차원 축소 유지
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)  # 데이터 타입 명확히 설정
        }

def re_weighting_sequence(target_series, current_epoch, total_epoch=5):

    easy_threshold = min(1.0, current_epoch*1.0/total_epoch)

    hard_weight = 0.2 + (current_epoch / total_epoch) * 0.8
    res_series = [1 if e <= easy_threshold else hard_weight for e in target_series] # 점진적으로 어려운 샘플에 더 높은 가중치 부여

    return res_series


def data_preparing(data_src, tokenizer, batch_size=16, shuffle=True, return_dataset=False):
    dataset = TextDataset(data_src, tokenizer)

    if return_dataset:
        return dataset

    dataloader = DataLoader(
        dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )
    return dataloader

def loader_return(dataframe, tokenizer, current_epoch, total_epoch, batch_size=16):
    target_col = dataframe['difficulty_score1']
    weights = re_weighting_sequence(target_col, current_epoch=current_epoch, total_epoch=total_epoch)

    dataset = TextDataset(dataframe, tokenizer)
    sampler = WeightedRandomSampler(weights=weights, num_samples=len(dataset), replacement=True)

    dataloader = DataLoader(dataset, batch_size=batch_size, sampler=sampler)

    return dataloader


def eval_model(
    model,
    eval_dataloader,
    device,
    num_cls=2,
    data_source=None,
    phase=None,
    wrong_predictions_list=None,
    result_file=None  # 추가
):
    total_loss = 0.0
    total_count = 0
    all_preds, all_labels, all_probs = [], [], []

    model.eval()
    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss
            total_loss += loss.item() * batch['labels'].size(0)
            total_count += batch['labels'].size(0)

            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = torch.argmax(logits, dim=-1)
            labels = batch['labels']

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = total_loss / total_count
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs) if not np.all(all_probs == all_probs[0]) else 0.0

    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    tpr_at_fpr_0001 = np.interp(0.01 / 100, fpr, tpr) if len(fpr) > 0 else 0.0

    print(f"Loss: {avg_loss:.4f} | Acc: {accuracy:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | AUROC: {auc:.4f} | TPR@FPR=0.01%: {tpr_at_fpr_0001:.4f}")

    if data_source is not None and 'txt' in data_source:
        txt_data = [text for text in data_source['txt']]
    else:
        txt_data = []

    # 결과 저장
    if result_file is not None:
        df_result = pd.DataFrame({
            'label': all_labels,
            'txt' : txt_data ,
            'pred': all_preds,
            'prob': all_probs
        })
        df_result.to_csv(result_file, index=False, encoding='utf-8-sig')

    return avg_loss, accuracy, f1, precision, recall, auc, tpr_at_fpr_0001, (fpr, tpr)



def train_model(
    model,
    train_dataloader,
    valid_dataloader,
    test_dataloader,
    device,
    learning_rate=3e-5,
    num_epochs=5,
    accumulation_steps=2,
    num_cls=2,
    run_id=0,
    early_stopping_patience=2,
    train_data=None,
    val_data=None,
    test_data=None,
    auroc_weight=0.0,
    result_file=None
):
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    model.to(device)

    print(f'-+-+-+ Run {run_id+1}: Initial Validation Metrics +-+-+-')
    eval_model(model, valid_dataloader, device, num_cls, data_source=val_data, phase="initial_val")
    print('-+-+-+ END OF INITIAL EVALUATION +-+-+-')

    train_loss_log = []
    valid_loss_log = []
    best_combined_score = 0.0  # 최고 결합 점수 초기화
    best_model_state = None
    early_stop_counter = 0
    all_wrong_predictions = []

    # 최적 성능 기록용
    best_valid_auroc = 0.0
    best_valid_tpr = 0.0

    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_count = 0
        phase_name = f"val_epoch{epoch+1}"
        valid_result_file = f"{result_dir}valid_result_run{run_id+1}_epoch{epoch+1}_{safe_model_name}_{method}.csv"
        model.train()  # 에폭 시작 시 그래디언트 초기화

        train_dataloader_current = loader_return(train_data, tokenizer, epoch + 1, num_epochs)

        for step, batch in enumerate(train_dataloader_current):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss / accumulation_steps
            total_train_loss += loss.item()
            total_count += 1

            loss.backward()

            if (step + 1) % accumulation_steps == 0  or (step + 1) == len(train_dataloader) :
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()


        avg_train_loss = total_train_loss / total_count
        train_loss_log.append(avg_train_loss)
        if val_data is None:
            raise ValueError("Validation data is None.")
        if test_data is None:
            raise ValueError("Test data is None.")

        print(f'Run {run_id+1}, Epoch {epoch + 1} 성능 평가:')
        # 현재 에폭에서 검증 데이터 평가 및 틀린 예측 저장
        phase_name = f"val_epoch{epoch+1}"

        avg_valid_loss, valid_acc, valid_f1, valid_precision, valid_recall, valid_auc, valid_tpr, _ = eval_model(
            model, valid_dataloader, device, num_cls,
            data_source=val_data,
            phase=phase_name,
            result_file=valid_result_file
        )
        valid_loss_log.append(avg_valid_loss)


       # AUROC와 TPR@FPR=0.01%를 결합한 점수 계산
        combined_score = auroc_weight * valid_auc + (1 - auroc_weight) * valid_tpr

        # Save the best model based on combined score
        if combined_score > best_combined_score:
            best_combined_score = combined_score
            best_valid_auroc = valid_auc  # 최고 성능일 때 AUROC 값 저장
            best_valid_tpr = valid_tpr    # 최고 성능일 때 TPR 값 저장
            best_model_state = model.state_dict().copy()
            early_stop_counter = 0
            print(f"Run {run_id+1}: New best model found with combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {valid_auc:.4f} (weight: {auroc_weight:.2f})")
            print(f"  - TPR @ FPR=0.01%: {valid_tpr:.4f} (weight: {(1-auroc_weight):.2f})")
        else:
            early_stop_counter += 1
            print(f"Run {run_id+1}: No improvement. Early stop counter = {early_stop_counter}/{early_stopping_patience}")

        # Early stopping 조건 만족 시 중단
        if early_stop_counter >= early_stopping_patience:
            print(f"Run {run_id+1}: Early stopping triggered at epoch {epoch + 1}")
            break

    # 최고 성능 모델로 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Run {run_id+1}: Restored best model for final evaluation")


    print(f'----- Run {run_id+1}: Final Evaluation on Test Data -----')
    test_result_file = f"{result_dir}test_result_run{run_id+1}_{safe_model_name}_{method}.csv"
    test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, _ = eval_model(
        model, test_dataloader, device, num_cls,
        data_source=test_data,
        phase="test",
        result_file=test_result_file
    )
    loss_log_df = pd.DataFrame({'train_loss_log': train_loss_log,'valid_loss_log': valid_loss_log})
    loss_log_df.to_csv(f"{result_dir}loss_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8')


    return train_loss_log, valid_loss_log, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score

# seed 추가
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True) # 추가


def run_multiple_experiments(
    model_class,
    model_name,
    train_data,
    val_data,
    test_data,
    tokenizer,
    device,
    num_runs=3,
    learning_rate=3e-5,
    num_epochs=5,
    batch_size=16,
    accumulation_steps=2,
    num_cls=2,
    seed=42,
    auroc_weight=0.0  # AUROC 가중치 추가
):
    # seed=42 설정
    set_seed(seed)

    # 결과 저장을 위한 리스트
    all_test_metrics = []
    best_overall_score = 0.0  # 전체 실험에서 최고 결합 점수
    best_overall_auroc = 0.0  # 최고 모델의 AUROC
    best_overall_tpr = 0.0    # 최고 모델의 TPR
    best_overall_model_state = None
    best_run_id = -1

    # 데이터로더 생성
    train_dataset = data_preparing(train_data, tokenizer, batch_size=batch_size, shuffle=True, return_dataset=True)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    valid_dataloader = data_preparing(val_data, tokenizer, batch_size=batch_size, shuffle=False)
    test_dataloader = data_preparing(test_data, tokenizer, batch_size=batch_size, shuffle=False)

    print(f"\nUsing combined metric with AUROC weight: {auroc_weight:.2f}, TPR@FPR=0.01% weight: {(1-auroc_weight):.2f}")

    for run in range(num_runs):
        print(f"\n===== Starting Run {run+1}/{num_runs} =====")

        # 새로운 모델 인스턴스 생성
        model = model_class.from_pretrained(model_name, num_labels=num_cls)

        # 모델 학습 - 원본 데이터 전달
        _, _, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score= train_model(
            model=model,
            train_dataloader=train_dataloader,
            valid_dataloader=valid_dataloader,
            test_dataloader=test_dataloader,
            train_data=train_data,
            val_data=val_data,
            test_data=test_data,
            device=device,
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            accumulation_steps=accumulation_steps,
            num_cls=num_cls,
            run_id=run
        )

        # 테스트 메트릭 저장
        run_metrics = {
            'loss': test_loss,
            'accuracy': test_acc,
            'f1': test_f1,
            'precision': test_precision,
            'recall': test_recall,
            'auc': test_auc,
            'tpr_at_fpr_0001': test_tpr,
            'valid_auroc': best_valid_auroc,
            'valid_tpr': best_valid_tpr,
            'combined_score': best_combined_score
        }
        all_test_metrics.append(run_metrics)

        # Update best overall model based on combined score
        if best_combined_score > best_overall_score:
            best_overall_score = best_combined_score
            best_overall_auroc = best_valid_auroc
            best_overall_tpr = best_valid_tpr
            best_overall_model_state = best_model_state
            best_run_id = run

            # best model 저장
            torch.save(best_model_state, f"{result_dir}best_model_run_{run+1}_score_{best_combined_score:.4f}_auroc_{best_valid_auroc:.4f}_tpr_{best_valid_tpr:.4f}_{safe_model_name}_{method}.pt")
            print(f"New overall best model saved from run {run+1}")
            print(f"  - Combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {best_valid_auroc:.4f}")
            print(f"  - TPR @ FPR=0.01%: {best_valid_tpr:.4f}")

    # 평균 성능 계산
    avg_metrics = {metric: np.mean([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}
    std_metrics = {metric: np.std([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}

    print("\n===== Average Test Results Across All Runs =====")
    print(f"Avg Test Loss: {avg_metrics['loss']:.4f} ± {std_metrics['loss']:.4f}")
    print(f"Avg Test Accuracy: {avg_metrics['accuracy']:.4f} ± {std_metrics['accuracy']:.4f}")
    print(f"Avg Test F1 Score: {avg_metrics['f1']:.4f} ± {std_metrics['f1']:.4f}")
    print(f"Avg Test Precision: {avg_metrics['precision']:.4f} ± {std_metrics['precision']:.4f}")
    print(f"Avg Test Recall: {avg_metrics['recall']:.4f} ± {std_metrics['recall']:.4f}")
    print(f"Avg Test AUROC: {avg_metrics['auc']:.4f} ± {std_metrics['auc']:.4f}")
    print(f"Avg Test TPR @ FPR=0.01%: {avg_metrics['tpr_at_fpr_0001']:.4f} ± {std_metrics['tpr_at_fpr_0001']:.4f}")
    print(f"Avg Validation AUROC: {avg_metrics['valid_auroc']:.4f} ± {std_metrics['valid_auroc']:.4f}")
    print(f"Avg Validation TPR @ FPR=0.01%: {avg_metrics['valid_tpr']:.4f} ± {std_metrics['valid_tpr']:.4f}")
    print(f"Avg Combined Score: {avg_metrics['combined_score']:.4f} ± {std_metrics['combined_score']:.4f}")

    print(f"\n===== Best Model from Run {best_run_id+1} =====")
    print(f"Combined Score: {best_overall_score:.4f}")
    print(f"Best Validation AUROC: {best_overall_auroc:.4f}")
    print(f"Best Validation TPR: {best_overall_tpr:.4f}")
    print(f"Saved as: best_model_run_{best_run_id+1}_score_{best_overall_score:.4f}_auroc_{best_overall_auroc:.4f}_tpr_{best_overall_tpr:.4f}_{safe_model_name}_{method}.pt")

    # 평균 성능 저장
    summary_df = pd.DataFrame({
      'metric': list(avg_metrics.keys()),
      'mean': [avg_metrics[m] for m in avg_metrics],
      'std': [std_metrics[m] for m in std_metrics]
    })
    summary_file_path = os.path.join(result_dir, f"summary_metrics_{safe_model_name}_{method}.csv")
    summary_df.to_csv(summary_file_path, index=False, encoding='utf-8')

    # best overall model 저장
    torch.save(best_overall_model_state, f"{result_dir}best_model_overall_{safe_model_name}_{method}.pt")
    print(f"Best overall model saved as: best_model_overall_{safe_model_name}_{method}.pt")

    return all_test_metrics, avg_metrics, std_metrics, best_overall_model_state, best_run_id


def main():
   # 데이터 분할 (7:1.5:1.5)

    prompt_list = rs_sample['info.essay_prompt'].unique().tolist()
    train_prompts, temp_prompts = train_test_split(prompt_list,
                                                   test_size=0.3,
                                                   random_state=42,
                                                   )
    val_prompts, test_prompts = train_test_split(temp_prompts,
                                                 test_size=0.5,
                                                 random_state=42,
                                                 )
    train_data = rs_sample[rs_sample['info.essay_prompt'].isin(train_prompts)].reset_index(drop=True)
    val_data = rs_sample[rs_sample['info.essay_prompt'].isin(val_prompts)].reset_index(drop=True)
    test_data = rs_sample[rs_sample['info.essay_prompt'].isin(test_prompts)].reset_index(drop=True)



    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    auroc_weight = 0.0  # 이 값을 조정하여 AUROC의 중요도 설정 (0~1 사이 값)

    all_test_metrics, avg_metrics, std_metrics, best_model_state, best_run_id = run_multiple_experiments(
        model_class=model_class,
        model_name=model_name,
        train_data=train_data,
        val_data=val_data,
        test_data=test_data,
        tokenizer=tokenizer,
        device=device,
        num_runs=3,
        learning_rate=3e-5,
        num_epochs=5,
        batch_size=16,
        accumulation_steps=2,
        seed=42,
        auroc_weight=auroc_weight  # AUROC 가중치 전달
    )

    # Final evaluation with best model
    print("\n===== Final Evaluation with Best Overall Model =====")
    final_model = model_class.from_pretrained(model_name, num_labels=2)
    final_model.load_state_dict(best_model_state)
    final_model.to(device)

    test_dataloader = data_preparing(test_data, tokenizer, batch_size=16, shuffle=False)
    _, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr,_ = eval_model(final_model, test_dataloader, device)


start_time = time.time()

if __name__ == "__main__":
    main()

elapsed_time_sec = time.time() - start_time
elapsed_time_min = elapsed_time_sec / 60
print(f"소요 시간: {elapsed_time_min:.2f}분")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Using combined metric with AUROC weight: 0.00, TPR@FPR=0.01% weight: 1.00

===== Starting Run 1/3 =====
-+-+-+ Run 1: Initial Validation Metrics +-+-+-
Loss: 0.7471 | Acc: 0.3912 | F1: 0.5622 | Precision: 0.3912 | Recall: 0.9993 | AUROC: 0.5788 | TPR@FPR=0.01%: 0.0007
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 1, Epoch 1 성능 평가:
Loss: 0.1801 | Acc: 0.9363 | F1: 0.9118 | Precision: 0.9959 | Recall: 0.8407 | AUROC: 0.9950 | TPR@FPR=0.01%: 0.7860
Run 1: New best model found with combined score: 0.7860
  - AUROC: 0.9950 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.7860 (weight: 1.00)
Run 1, Epoch 2 성능 평가:
Loss: 0.0154 | Acc: 0.9954 | F1: 0.9941 | Precision: 0.9979 | Recall: 0.9903 | AUROC: 0.9998 | TPR@FPR=0.01%: 0.9591
Run 1: New best model found with combined score: 0.9591
  - AUROC: 0.9998 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9591 (weight: 1.00)
Run 1, Epoch 3 성능 평가:
Loss: 0.0138 | Acc: 0.9957 | F1: 0.9945 | Precision: 0.9945 | Recall: 0.9945 | AUROC: 0.9999 | TPR@FPR=0.01%: 0.9411
Run 1: No

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 2: Initial Validation Metrics +-+-+-
Loss: 0.6649 | Acc: 0.6090 | F1: 0.0014 | Precision: 1.0000 | Recall: 0.0007 | AUROC: 0.6893 | TPR@FPR=0.01%: 0.0007
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 2, Epoch 1 성능 평가:
Loss: 0.1399 | Acc: 0.9447 | F1: 0.9241 | Precision: 0.9984 | Recall: 0.8601 | AUROC: 0.9994 | TPR@FPR=0.01%: 0.8262
Run 2: New best model found with combined score: 0.8262
  - AUROC: 0.9994 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.8262 (weight: 1.00)
Run 2, Epoch 2 성능 평가:
Loss: 0.0383 | Acc: 0.9851 | F1: 0.9807 | Precision: 0.9950 | Recall: 0.9668 | AUROC: 0.9997 | TPR@FPR=0.01%: 0.9245
Run 2: New best model found with combined score: 0.9245
  - AUROC: 0.9997 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9245 (weight: 1.00)
Run 2, Epoch 3 성능 평가:
Loss: 0.0166 | Acc: 0.9946 | F1: 0.9931 | Precision: 0.9965 | Recall: 0.9896 | AUROC: 0.9999 | TPR@FPR=0.01%: 0.9792
Run 2: New best model found with combined score: 0.9792
  - AUROC: 0.9999 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.97

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 3: Initial Validation Metrics +-+-+-
Loss: 0.6664 | Acc: 0.6090 | F1: 0.0014 | Precision: 1.0000 | Recall: 0.0007 | AUROC: 0.5530 | TPR@FPR=0.01%: 0.0007
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 3, Epoch 1 성능 평가:
Loss: 0.0187 | Acc: 0.9921 | F1: 0.9900 | Precision: 0.9829 | Recall: 0.9972 | AUROC: 0.9998 | TPR@FPR=0.01%: 0.9051
Run 3: New best model found with combined score: 0.9051
  - AUROC: 0.9998 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9051 (weight: 1.00)
Run 3, Epoch 2 성능 평가:
Loss: 0.0259 | Acc: 0.9930 | F1: 0.9910 | Precision: 0.9876 | Recall: 0.9945 | AUROC: 0.9996 | TPR@FPR=0.01%: 0.8331
Run 3: No improvement. Early stop counter = 1/2
Run 3, Epoch 3 성능 평가:
Loss: 0.0180 | Acc: 0.9938 | F1: 0.9920 | Precision: 0.9951 | Recall: 0.9889 | AUROC: 0.9998 | TPR@FPR=0.01%: 0.9114
Run 3: New best model found with combined score: 0.9114
  - AUROC: 0.9998 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9114 (weight: 1.00)
Run 3, Epoch 4 성능 평가:
Loss: 0.0271 | Acc: 0.9897 | F1: 0.9867 | Pr

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loss: 0.0938 | Acc: 0.9828 | F1: 0.9792 | Precision: 0.9630 | Recall: 0.9960 | AUROC: 0.9978 | TPR@FPR=0.01%: 0.4241
소요 시간: 197.07분


In [ ]:
# predefined2

result_dir = f"/content/drive/My Drive/cdf/"
safe_model_name = model_name.split('/')[-1] if '/' in model_name else model_name
method = "predefined2"



class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.texts = dataframe['txt'].tolist()
        self.difficulty_score = dataframe['difficulty_score2'].tolist()  #difficulty_score1 difficulty_score2 difficulty_score3
        self.labels = dataframe['target'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': tokenized['input_ids'].squeeze(0),  # 차원 축소 유지
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)  # 데이터 타입 명확히 설정
        }

def re_weighting_sequence(target_series, current_epoch, total_epoch=5):

    easy_threshold = min(1.0, current_epoch*1.0/total_epoch)

    hard_weight = 0.2 + (current_epoch / total_epoch) * 0.8
    res_series = [1 if e <= easy_threshold else hard_weight for e in target_series] # 점진적으로 어려운 샘플에 더 높은 가중치 부여

    return res_series


def data_preparing(data_src, tokenizer, batch_size=16, shuffle=True, return_dataset=False):
    dataset = TextDataset(data_src, tokenizer)

    if return_dataset:
        return dataset

    dataloader = DataLoader(
        dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )
    return dataloader

def loader_return(dataframe, tokenizer, current_epoch, total_epoch, batch_size=16):
    target_col = dataframe['difficulty_score2']
    weights = re_weighting_sequence(target_col, current_epoch=current_epoch, total_epoch=total_epoch)

    dataset = TextDataset(dataframe, tokenizer)
    sampler = WeightedRandomSampler(weights=weights, num_samples=len(dataset), replacement=True)

    dataloader = DataLoader(dataset, batch_size=batch_size, sampler=sampler)

    return dataloader


def eval_model(
    model,
    eval_dataloader,
    device,
    num_cls=2,
    data_source=None,
    phase=None,
    wrong_predictions_list=None,
    result_file=None  # 추가
):
    total_loss = 0.0
    total_count = 0
    all_preds, all_labels, all_probs = [], [], []

    model.eval()
    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss
            total_loss += loss.item() * batch['labels'].size(0)
            total_count += batch['labels'].size(0)

            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = torch.argmax(logits, dim=-1)
            labels = batch['labels']

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = total_loss / total_count
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs) if not np.all(all_probs == all_probs[0]) else 0.0

    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    tpr_at_fpr_0001 = np.interp(0.01 / 100, fpr, tpr) if len(fpr) > 0 else 0.0

    print(f"Loss: {avg_loss:.4f} | Acc: {accuracy:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | AUROC: {auc:.4f} | TPR@FPR=0.01%: {tpr_at_fpr_0001:.4f}")

    if data_source is not None and 'txt' in data_source:
        txt_data = [text for text in data_source['txt']]
    else:
        txt_data = []

    # 결과 저장
    if result_file is not None:
        df_result = pd.DataFrame({
            'label': all_labels,
            'txt' : txt_data ,
            'pred': all_preds,
            'prob': all_probs
        })
        df_result.to_csv(result_file, index=False, encoding='utf-8-sig')

    return avg_loss, accuracy, f1, precision, recall, auc, tpr_at_fpr_0001, (fpr, tpr)



def train_model(
    model,
    train_dataloader,
    valid_dataloader,
    test_dataloader,
    device,
    learning_rate=3e-5,
    num_epochs=5,
    accumulation_steps=2,
    num_cls=2,
    run_id=0,
    early_stopping_patience=2,
    train_data=None,
    val_data=None,
    test_data=None,
    auroc_weight=0.0,
    result_file=None
):
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    model.to(device)

    print(f'-+-+-+ Run {run_id+1}: Initial Validation Metrics +-+-+-')
    eval_model(model, valid_dataloader, device, num_cls, data_source=val_data, phase="initial_val")
    print('-+-+-+ END OF INITIAL EVALUATION +-+-+-')

    train_loss_log = []
    valid_loss_log = []
    best_combined_score = 0.0  # 최고 결합 점수 초기화
    best_model_state = None
    early_stop_counter = 0
    all_wrong_predictions = []

    # 최적 성능 기록용
    best_valid_auroc = 0.0
    best_valid_tpr = 0.0

    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_count = 0
        phase_name = f"val_epoch{epoch+1}"
        valid_result_file = f"{result_dir}valid_result_run{run_id+1}_epoch{epoch+1}_{safe_model_name}_{method}.csv"
        model.train()  # 에폭 시작 시 그래디언트 초기화

        train_dataloader_current = loader_return(train_data, tokenizer, epoch + 1, num_epochs)

        for step, batch in enumerate(train_dataloader_current):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss / accumulation_steps
            total_train_loss += loss.item()
            total_count += 1

            loss.backward()

            if (step + 1) % accumulation_steps == 0  or (step + 1) == len(train_dataloader) :
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()


        avg_train_loss = total_train_loss / total_count
        train_loss_log.append(avg_train_loss)
        if val_data is None:
            raise ValueError("Validation data is None.")
        if test_data is None:
            raise ValueError("Test data is None.")

        print(f'Run {run_id+1}, Epoch {epoch + 1} 성능 평가:')
        # 현재 에폭에서 검증 데이터 평가 및 틀린 예측 저장
        phase_name = f"val_epoch{epoch+1}"

        avg_valid_loss, valid_acc, valid_f1, valid_precision, valid_recall, valid_auc, valid_tpr, _ = eval_model(
            model, valid_dataloader, device, num_cls,
            data_source=val_data,
            phase=phase_name,
            result_file=valid_result_file
        )
        valid_loss_log.append(avg_valid_loss)


       # AUROC와 TPR@FPR=0.01%를 결합한 점수 계산
        combined_score = auroc_weight * valid_auc + (1 - auroc_weight) * valid_tpr

        # Save the best model based on combined score
        if combined_score > best_combined_score:
            best_combined_score = combined_score
            best_valid_auroc = valid_auc  # 최고 성능일 때 AUROC 값 저장
            best_valid_tpr = valid_tpr    # 최고 성능일 때 TPR 값 저장
            best_model_state = model.state_dict().copy()
            early_stop_counter = 0
            print(f"Run {run_id+1}: New best model found with combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {valid_auc:.4f} (weight: {auroc_weight:.2f})")
            print(f"  - TPR @ FPR=0.01%: {valid_tpr:.4f} (weight: {(1-auroc_weight):.2f})")
        else:
            early_stop_counter += 1
            print(f"Run {run_id+1}: No improvement. Early stop counter = {early_stop_counter}/{early_stopping_patience}")

        # Early stopping 조건 만족 시 중단
        if early_stop_counter >= early_stopping_patience:
            print(f"Run {run_id+1}: Early stopping triggered at epoch {epoch + 1}")
            break

    # 최고 성능 모델로 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Run {run_id+1}: Restored best model for final evaluation")


    print(f'----- Run {run_id+1}: Final Evaluation on Test Data -----')
    test_result_file = f"{result_dir}test_result_run{run_id+1}_{safe_model_name}_{method}.csv"
    test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, _ = eval_model(
        model, test_dataloader, device, num_cls,
        data_source=test_data,
        phase="test",
        result_file=test_result_file
    )
    loss_log_df = pd.DataFrame({'train_loss_log': train_loss_log,'valid_loss_log': valid_loss_log})
    loss_log_df.to_csv(f"{result_dir}loss_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8')


    return train_loss_log, valid_loss_log, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score

# seed 추가
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True) # 추가


def run_multiple_experiments(
    model_class,
    model_name,
    train_data,
    val_data,
    test_data,
    tokenizer,
    device,
    num_runs=3,
    learning_rate=3e-5,
    num_epochs=5,
    batch_size=16,
    accumulation_steps=2,
    num_cls=2,
    seed=42,
    auroc_weight=0.0  # AUROC 가중치 추가
):
    # seed=42 설정
    set_seed(seed)

    # 결과 저장을 위한 리스트
    all_test_metrics = []
    best_overall_score = 0.0  # 전체 실험에서 최고 결합 점수
    best_overall_auroc = 0.0  # 최고 모델의 AUROC
    best_overall_tpr = 0.0    # 최고 모델의 TPR
    best_overall_model_state = None
    best_run_id = -1

    # 데이터로더 생성
    train_dataset = data_preparing(train_data, tokenizer, batch_size=batch_size, shuffle=True, return_dataset=True)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    valid_dataloader = data_preparing(val_data, tokenizer, batch_size=batch_size, shuffle=False)
    test_dataloader = data_preparing(test_data, tokenizer, batch_size=batch_size, shuffle=False)

    print(f"\nUsing combined metric with AUROC weight: {auroc_weight:.2f}, TPR@FPR=0.01% weight: {(1-auroc_weight):.2f}")

    for run in range(num_runs):
        print(f"\n===== Starting Run {run+1}/{num_runs} =====")

        # 새로운 모델 인스턴스 생성
        model = model_class.from_pretrained(model_name, num_labels=num_cls)

        # 모델 학습 - 원본 데이터 전달
        _, _, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score= train_model(
            model=model,
            train_dataloader=train_dataloader,
            valid_dataloader=valid_dataloader,
            test_dataloader=test_dataloader,
            train_data=train_data,
            val_data=val_data,
            test_data=test_data,
            device=device,
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            accumulation_steps=accumulation_steps,
            num_cls=num_cls,
            run_id=run
        )

        # 테스트 메트릭 저장
        run_metrics = {
            'loss': test_loss,
            'accuracy': test_acc,
            'f1': test_f1,
            'precision': test_precision,
            'recall': test_recall,
            'auc': test_auc,
            'tpr_at_fpr_0001': test_tpr,
            'valid_auroc': best_valid_auroc,
            'valid_tpr': best_valid_tpr,
            'combined_score': best_combined_score
        }
        all_test_metrics.append(run_metrics)

        # Update best overall model based on combined score
        if best_combined_score > best_overall_score:
            best_overall_score = best_combined_score
            best_overall_auroc = best_valid_auroc
            best_overall_tpr = best_valid_tpr
            best_overall_model_state = best_model_state
            best_run_id = run

            # best model 저장
            torch.save(best_model_state, f"{result_dir}best_model_run_{run+1}_score_{best_combined_score:.4f}_auroc_{best_valid_auroc:.4f}_tpr_{best_valid_tpr:.4f}_{safe_model_name}_{method}.pt")
            print(f"New overall best model saved from run {run+1}")
            print(f"  - Combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {best_valid_auroc:.4f}")
            print(f"  - TPR @ FPR=0.01%: {best_valid_tpr:.4f}")

    # 평균 성능 계산
    avg_metrics = {metric: np.mean([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}
    std_metrics = {metric: np.std([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}

    print("\n===== Average Test Results Across All Runs =====")
    print(f"Avg Test Loss: {avg_metrics['loss']:.4f} ± {std_metrics['loss']:.4f}")
    print(f"Avg Test Accuracy: {avg_metrics['accuracy']:.4f} ± {std_metrics['accuracy']:.4f}")
    print(f"Avg Test F1 Score: {avg_metrics['f1']:.4f} ± {std_metrics['f1']:.4f}")
    print(f"Avg Test Precision: {avg_metrics['precision']:.4f} ± {std_metrics['precision']:.4f}")
    print(f"Avg Test Recall: {avg_metrics['recall']:.4f} ± {std_metrics['recall']:.4f}")
    print(f"Avg Test AUROC: {avg_metrics['auc']:.4f} ± {std_metrics['auc']:.4f}")
    print(f"Avg Test TPR @ FPR=0.01%: {avg_metrics['tpr_at_fpr_0001']:.4f} ± {std_metrics['tpr_at_fpr_0001']:.4f}")
    print(f"Avg Validation AUROC: {avg_metrics['valid_auroc']:.4f} ± {std_metrics['valid_auroc']:.4f}")
    print(f"Avg Validation TPR @ FPR=0.01%: {avg_metrics['valid_tpr']:.4f} ± {std_metrics['valid_tpr']:.4f}")
    print(f"Avg Combined Score: {avg_metrics['combined_score']:.4f} ± {std_metrics['combined_score']:.4f}")

    print(f"\n===== Best Model from Run {best_run_id+1} =====")
    print(f"Combined Score: {best_overall_score:.4f}")
    print(f"Best Validation AUROC: {best_overall_auroc:.4f}")
    print(f"Best Validation TPR: {best_overall_tpr:.4f}")
    print(f"Saved as: best_model_run_{best_run_id+1}_score_{best_overall_score:.4f}_auroc_{best_overall_auroc:.4f}_tpr_{best_overall_tpr:.4f}_{safe_model_name}_{method}.pt")

    # 평균 성능 저장
    summary_df = pd.DataFrame({
      'metric': list(avg_metrics.keys()),
      'mean': [avg_metrics[m] for m in avg_metrics],
      'std': [std_metrics[m] for m in std_metrics]
    })
    summary_file_path = os.path.join(result_dir, f"summary_metrics_{safe_model_name}_{method}.csv")
    summary_df.to_csv(summary_file_path, index=False, encoding='utf-8')

    # best overall model 저장
    torch.save(best_overall_model_state, f"{result_dir}best_model_overall_{safe_model_name}_{method}.pt")
    print(f"Best overall model saved as: best_model_overall_{safe_model_name}_{method}.pt")

    return all_test_metrics, avg_metrics, std_metrics, best_overall_model_state, best_run_id


def main():
   # 데이터 분할 (7:1.5:1.5)

    prompt_list = rs_sample['info.essay_prompt'].unique().tolist()
    train_prompts, temp_prompts = train_test_split(prompt_list,
                                                   test_size=0.3,
                                                   random_state=42,
                                                   )
    val_prompts, test_prompts = train_test_split(temp_prompts,
                                                 test_size=0.5,
                                                 random_state=42,
                                                 )
    train_data = rs_sample[rs_sample['info.essay_prompt'].isin(train_prompts)].reset_index(drop=True)
    val_data = rs_sample[rs_sample['info.essay_prompt'].isin(val_prompts)].reset_index(drop=True)
    test_data = rs_sample[rs_sample['info.essay_prompt'].isin(test_prompts)].reset_index(drop=True)



    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    auroc_weight = 0.0  # 이 값을 조정하여 AUROC의 중요도 설정 (0~1 사이 값)

    all_test_metrics, avg_metrics, std_metrics, best_model_state, best_run_id = run_multiple_experiments(
        model_class=model_class,
        model_name=model_name,
        train_data=train_data,
        val_data=val_data,
        test_data=test_data,
        tokenizer=tokenizer,
        device=device,
        num_runs=3,
        learning_rate=3e-5,
        num_epochs=5,
        batch_size=16,
        accumulation_steps=2,
        seed=42,
        auroc_weight=auroc_weight  # AUROC 가중치 전달
    )

    # Final evaluation with best model
    print("\n===== Final Evaluation with Best Overall Model =====")
    final_model = model_class.from_pretrained(model_name, num_labels=2)
    final_model.load_state_dict(best_model_state)
    final_model.to(device)

    test_dataloader = data_preparing(test_data, tokenizer, batch_size=16, shuffle=False)
    _, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr,_ = eval_model(final_model, test_dataloader, device)


start_time = time.time()

if __name__ == "__main__":
    main()

elapsed_time_sec = time.time() - start_time
elapsed_time_min = elapsed_time_sec / 60
print(f"소요 시간: {elapsed_time_min:.2f}분")


Using combined metric with AUROC weight: 0.00, TPR@FPR=0.01% weight: 1.00

===== Starting Run 1/3 =====


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 1: Initial Validation Metrics +-+-+-
Loss: 0.7471 | Acc: 0.3912 | F1: 0.5622 | Precision: 0.3912 | Recall: 0.9993 | AUROC: 0.5788 | TPR@FPR=0.01%: 0.0007
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 1, Epoch 1 성능 평가:
Loss: 0.1020 | Acc: 0.9520 | F1: 0.9347 | Precision: 1.0000 | Recall: 0.8774 | AUROC: 0.9999 | TPR@FPR=0.01%: 0.9564
Run 1: New best model found with combined score: 0.9564
  - AUROC: 0.9999 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9564 (weight: 1.00)
Run 1, Epoch 2 성능 평가:
Loss: 0.0296 | Acc: 0.9919 | F1: 0.9896 | Precision: 0.9937 | Recall: 0.9855 | AUROC: 0.9997 | TPR@FPR=0.01%: 0.8054
Run 1: No improvement. Early stop counter = 1/2
Run 1, Epoch 3 성능 평가:
Loss: 0.0107 | Acc: 0.9967 | F1: 0.9959 | Precision: 0.9931 | Recall: 0.9986 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9723
Run 1: New best model found with combined score: 0.9723
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9723 (weight: 1.00)
Run 1, Epoch 4 성능 평가:
Loss: 0.0091 | Acc: 0.9970 | F1: 0.9962 | Pr

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 2: Initial Validation Metrics +-+-+-
Loss: 0.6649 | Acc: 0.6090 | F1: 0.0014 | Precision: 1.0000 | Recall: 0.0007 | AUROC: 0.6893 | TPR@FPR=0.01%: 0.0007
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 2, Epoch 1 성능 평가:
Loss: 0.0236 | Acc: 0.9930 | F1: 0.9910 | Precision: 0.9944 | Recall: 0.9875 | AUROC: 0.9998 | TPR@FPR=0.01%: 0.9072
Run 2: New best model found with combined score: 0.9072
  - AUROC: 0.9998 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9072 (weight: 1.00)
Run 2, Epoch 2 성능 평가:
Loss: 0.0108 | Acc: 0.9959 | F1: 0.9948 | Precision: 0.9945 | Recall: 0.9952 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9806
Run 2: New best model found with combined score: 0.9806
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9806 (weight: 1.00)
Run 2, Epoch 3 성능 평가:
Loss: 0.0351 | Acc: 0.9851 | F1: 0.9812 | Precision: 0.9664 | Recall: 0.9965 | AUROC: 0.9998 | TPR@FPR=0.01%: 0.9806
Run 2: No improvement. Early stop counter = 1/2
Run 2, Epoch 4 성능 평가:
Loss: 0.0210 | Acc: 0.9916 | F1: 0.9894 | Pr

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 3: Initial Validation Metrics +-+-+-
Loss: 0.6871 | Acc: 0.5904 | F1: 0.1168 | Precision: 0.3731 | Recall: 0.0693 | AUROC: 0.4514 | TPR@FPR=0.01%: 0.0007
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 3, Epoch 1 성능 평가:
Loss: 0.0306 | Acc: 0.9894 | F1: 0.9864 | Precision: 0.9958 | Recall: 0.9771 | AUROC: 0.9997 | TPR@FPR=0.01%: 0.9245
Run 3: New best model found with combined score: 0.9245
  - AUROC: 0.9997 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9245 (weight: 1.00)
Run 3, Epoch 2 성능 평가:
Loss: 0.1209 | Acc: 0.9626 | F1: 0.9498 | Precision: 1.0000 | Recall: 0.9044 | AUROC: 0.9996 | TPR@FPR=0.01%: 0.9273
Run 3: New best model found with combined score: 0.9273
  - AUROC: 0.9996 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9273 (weight: 1.00)
Run 3, Epoch 3 성능 평가:
Loss: 0.0095 | Acc: 0.9970 | F1: 0.9962 | Precision: 0.9931 | Recall: 0.9993 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9688
Run 3: New best model found with combined score: 0.9688
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.96

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loss: 0.0339 | Acc: 0.9902 | F1: 0.9880 | Precision: 0.9802 | Recall: 0.9960 | AUROC: 0.9997 | TPR@FPR=0.01%: 0.9476
소요 시간: 184.74분


In [ ]:
# predefined2

tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")
model_class = AutoModelForSequenceClassification
model_name = "monologg/kobert"

result_dir = f"/content/drive/My Drive/cdf/"
safe_model_name = model_name.split('/')[-1] if '/' in model_name else model_name
method = "predefined2"



class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.texts = dataframe['txt'].tolist()
        self.difficulty_score = dataframe['difficulty_score2'].tolist()  #difficulty_score1 difficulty_score2 difficulty_score3
        self.labels = dataframe['target'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': tokenized['input_ids'].squeeze(0),  # 차원 축소 유지
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)  # 데이터 타입 명확히 설정
        }

def re_weighting_sequence(target_series, current_epoch, total_epoch=5):

    easy_threshold = min(1.0, current_epoch*1.0/total_epoch)

    hard_weight = 0.2 + (current_epoch / total_epoch) * 0.8
    res_series = [1 if e <= easy_threshold else hard_weight for e in target_series] # 점진적으로 어려운 샘플에 더 높은 가중치 부여

    return res_series


def data_preparing(data_src, tokenizer, batch_size=16, shuffle=True, return_dataset=False):
    dataset = TextDataset(data_src, tokenizer)

    if return_dataset:
        return dataset

    dataloader = DataLoader(
        dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )
    return dataloader

def loader_return(dataframe, tokenizer, current_epoch, total_epoch, batch_size=16):
    target_col = dataframe['difficulty_score2']
    weights = re_weighting_sequence(target_col, current_epoch=current_epoch, total_epoch=total_epoch)

    dataset = TextDataset(dataframe, tokenizer)
    sampler = WeightedRandomSampler(weights=weights, num_samples=len(dataset), replacement=True)

    dataloader = DataLoader(dataset, batch_size=batch_size, sampler=sampler)

    return dataloader


def eval_model(
    model,
    eval_dataloader,
    device,
    num_cls=2,
    data_source=None,
    phase=None,
    wrong_predictions_list=None,
    result_file=None  # 추가
):
    total_loss = 0.0
    total_count = 0
    all_preds, all_labels, all_probs = [], [], []

    model.eval()
    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss
            total_loss += loss.item() * batch['labels'].size(0)
            total_count += batch['labels'].size(0)

            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = torch.argmax(logits, dim=-1)
            labels = batch['labels']

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = total_loss / total_count
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs) if not np.all(all_probs == all_probs[0]) else 0.0

    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    tpr_at_fpr_0001 = np.interp(0.01 / 100, fpr, tpr) if len(fpr) > 0 else 0.0

    print(f"Loss: {avg_loss:.4f} | Acc: {accuracy:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | AUROC: {auc:.4f} | TPR@FPR=0.01%: {tpr_at_fpr_0001:.4f}")

    if data_source is not None and 'txt' in data_source:
        txt_data = [text for text in data_source['txt']]
    else:
        txt_data = []

    # 결과 저장
    if result_file is not None:
        df_result = pd.DataFrame({
            'label': all_labels,
            'txt' : txt_data ,
            'pred': all_preds,
            'prob': all_probs
        })
        df_result.to_csv(result_file, index=False, encoding='utf-8-sig')

    return avg_loss, accuracy, f1, precision, recall, auc, tpr_at_fpr_0001, (fpr, tpr)



def train_model(
    model,
    train_dataloader,
    valid_dataloader,
    test_dataloader,
    device,
    learning_rate=3e-5,
    num_epochs=5,
    accumulation_steps=2,
    num_cls=2,
    run_id=0,
    early_stopping_patience=2,
    train_data=None,
    val_data=None,
    test_data=None,
    auroc_weight=0.0,
    result_file=None
):
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    model.to(device)

    print(f'-+-+-+ Run {run_id+1}: Initial Validation Metrics +-+-+-')
    eval_model(model, valid_dataloader, device, num_cls, data_source=val_data, phase="initial_val")
    print('-+-+-+ END OF INITIAL EVALUATION +-+-+-')

    train_loss_log = []
    valid_loss_log = []
    best_combined_score = 0.0  # 최고 결합 점수 초기화
    best_model_state = None
    early_stop_counter = 0
    all_wrong_predictions = []

    # 최적 성능 기록용
    best_valid_auroc = 0.0
    best_valid_tpr = 0.0

    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_count = 0
        phase_name = f"val_epoch{epoch+1}"
        valid_result_file = f"{result_dir}valid_result_run{run_id+1}_epoch{epoch+1}_{safe_model_name}_{method}.csv"
        model.train()  # 에폭 시작 시 그래디언트 초기화

        train_dataloader_current = loader_return(train_data, tokenizer, epoch + 1, num_epochs)

        for step, batch in enumerate(train_dataloader_current):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss / accumulation_steps
            total_train_loss += loss.item()
            total_count += 1

            loss.backward()

            if (step + 1) % accumulation_steps == 0  or (step + 1) == len(train_dataloader) :
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()


        avg_train_loss = total_train_loss / total_count
        train_loss_log.append(avg_train_loss)
        if val_data is None:
            raise ValueError("Validation data is None.")
        if test_data is None:
            raise ValueError("Test data is None.")

        print(f'Run {run_id+1}, Epoch {epoch + 1} 성능 평가:')
        # 현재 에폭에서 검증 데이터 평가 및 틀린 예측 저장
        phase_name = f"val_epoch{epoch+1}"

        avg_valid_loss, valid_acc, valid_f1, valid_precision, valid_recall, valid_auc, valid_tpr, _ = eval_model(
            model, valid_dataloader, device, num_cls,
            data_source=val_data,
            phase=phase_name,
            result_file=valid_result_file
        )
        valid_loss_log.append(avg_valid_loss)


       # AUROC와 TPR@FPR=0.01%를 결합한 점수 계산
        combined_score = auroc_weight * valid_auc + (1 - auroc_weight) * valid_tpr

        # Save the best model based on combined score
        if combined_score > best_combined_score:
            best_combined_score = combined_score
            best_valid_auroc = valid_auc  # 최고 성능일 때 AUROC 값 저장
            best_valid_tpr = valid_tpr    # 최고 성능일 때 TPR 값 저장
            best_model_state = model.state_dict().copy()
            early_stop_counter = 0
            print(f"Run {run_id+1}: New best model found with combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {valid_auc:.4f} (weight: {auroc_weight:.2f})")
            print(f"  - TPR @ FPR=0.01%: {valid_tpr:.4f} (weight: {(1-auroc_weight):.2f})")
        else:
            early_stop_counter += 1
            print(f"Run {run_id+1}: No improvement. Early stop counter = {early_stop_counter}/{early_stopping_patience}")

        # Early stopping 조건 만족 시 중단
        if early_stop_counter >= early_stopping_patience:
            print(f"Run {run_id+1}: Early stopping triggered at epoch {epoch + 1}")
            break

    # 최고 성능 모델로 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Run {run_id+1}: Restored best model for final evaluation")


    print(f'----- Run {run_id+1}: Final Evaluation on Test Data -----')
    test_result_file = f"{result_dir}test_result_run{run_id+1}_{safe_model_name}_{method}.csv"
    test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, _ = eval_model(
        model, test_dataloader, device, num_cls,
        data_source=test_data,
        phase="test",
        result_file=test_result_file
    )
    loss_log_df = pd.DataFrame({'train_loss_log': train_loss_log,'valid_loss_log': valid_loss_log})
    loss_log_df.to_csv(f"{result_dir}loss_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8')


    return train_loss_log, valid_loss_log, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score

# seed 추가
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True) # 추가


def run_multiple_experiments(
    model_class,
    model_name,
    train_data,
    val_data,
    test_data,
    tokenizer,
    device,
    num_runs=3,
    learning_rate=3e-5,
    num_epochs=5,
    batch_size=16,
    accumulation_steps=2,
    num_cls=2,
    seed=42,
    auroc_weight=0.0  # AUROC 가중치 추가
):
    # seed=42 설정
    set_seed(seed)

    # 결과 저장을 위한 리스트
    all_test_metrics = []
    best_overall_score = 0.0  # 전체 실험에서 최고 결합 점수
    best_overall_auroc = 0.0  # 최고 모델의 AUROC
    best_overall_tpr = 0.0    # 최고 모델의 TPR
    best_overall_model_state = None
    best_run_id = -1

    # 데이터로더 생성
    train_dataset = data_preparing(train_data, tokenizer, batch_size=batch_size, shuffle=True, return_dataset=True)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    valid_dataloader = data_preparing(val_data, tokenizer, batch_size=batch_size, shuffle=False)
    test_dataloader = data_preparing(test_data, tokenizer, batch_size=batch_size, shuffle=False)

    print(f"\nUsing combined metric with AUROC weight: {auroc_weight:.2f}, TPR@FPR=0.01% weight: {(1-auroc_weight):.2f}")

    for run in range(num_runs):
        print(f"\n===== Starting Run {run+1}/{num_runs} =====")

        # 새로운 모델 인스턴스 생성
        model = model_class.from_pretrained(model_name, num_labels=num_cls)

        # 모델 학습 - 원본 데이터 전달
        _, _, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score= train_model(
            model=model,
            train_dataloader=train_dataloader,
            valid_dataloader=valid_dataloader,
            test_dataloader=test_dataloader,
            train_data=train_data,
            val_data=val_data,
            test_data=test_data,
            device=device,
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            accumulation_steps=accumulation_steps,
            num_cls=num_cls,
            run_id=run
        )

        # 테스트 메트릭 저장
        run_metrics = {
            'loss': test_loss,
            'accuracy': test_acc,
            'f1': test_f1,
            'precision': test_precision,
            'recall': test_recall,
            'auc': test_auc,
            'tpr_at_fpr_0001': test_tpr,
            'valid_auroc': best_valid_auroc,
            'valid_tpr': best_valid_tpr,
            'combined_score': best_combined_score
        }
        all_test_metrics.append(run_metrics)

        # Update best overall model based on combined score
        if best_combined_score > best_overall_score:
            best_overall_score = best_combined_score
            best_overall_auroc = best_valid_auroc
            best_overall_tpr = best_valid_tpr
            best_overall_model_state = best_model_state
            best_run_id = run

            # best model 저장
            torch.save(best_model_state, f"{result_dir}best_model_run_{run+1}_score_{best_combined_score:.4f}_auroc_{best_valid_auroc:.4f}_tpr_{best_valid_tpr:.4f}_{safe_model_name}_{method}.pt")
            print(f"New overall best model saved from run {run+1}")
            print(f"  - Combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {best_valid_auroc:.4f}")
            print(f"  - TPR @ FPR=0.01%: {best_valid_tpr:.4f}")

    # 평균 성능 계산
    avg_metrics = {metric: np.mean([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}
    std_metrics = {metric: np.std([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}

    print("\n===== Average Test Results Across All Runs =====")
    print(f"Avg Test Loss: {avg_metrics['loss']:.4f} ± {std_metrics['loss']:.4f}")
    print(f"Avg Test Accuracy: {avg_metrics['accuracy']:.4f} ± {std_metrics['accuracy']:.4f}")
    print(f"Avg Test F1 Score: {avg_metrics['f1']:.4f} ± {std_metrics['f1']:.4f}")
    print(f"Avg Test Precision: {avg_metrics['precision']:.4f} ± {std_metrics['precision']:.4f}")
    print(f"Avg Test Recall: {avg_metrics['recall']:.4f} ± {std_metrics['recall']:.4f}")
    print(f"Avg Test AUROC: {avg_metrics['auc']:.4f} ± {std_metrics['auc']:.4f}")
    print(f"Avg Test TPR @ FPR=0.01%: {avg_metrics['tpr_at_fpr_0001']:.4f} ± {std_metrics['tpr_at_fpr_0001']:.4f}")
    print(f"Avg Validation AUROC: {avg_metrics['valid_auroc']:.4f} ± {std_metrics['valid_auroc']:.4f}")
    print(f"Avg Validation TPR @ FPR=0.01%: {avg_metrics['valid_tpr']:.4f} ± {std_metrics['valid_tpr']:.4f}")
    print(f"Avg Combined Score: {avg_metrics['combined_score']:.4f} ± {std_metrics['combined_score']:.4f}")

    print(f"\n===== Best Model from Run {best_run_id+1} =====")
    print(f"Combined Score: {best_overall_score:.4f}")
    print(f"Best Validation AUROC: {best_overall_auroc:.4f}")
    print(f"Best Validation TPR: {best_overall_tpr:.4f}")
    print(f"Saved as: best_model_run_{best_run_id+1}_score_{best_overall_score:.4f}_auroc_{best_overall_auroc:.4f}_tpr_{best_overall_tpr:.4f}_{safe_model_name}_{method}.pt")

    # 평균 성능 저장
    summary_df = pd.DataFrame({
      'metric': list(avg_metrics.keys()),
      'mean': [avg_metrics[m] for m in avg_metrics],
      'std': [std_metrics[m] for m in std_metrics]
    })
    summary_file_path = os.path.join(result_dir, f"summary_metrics_{safe_model_name}_{method}.csv")
    summary_df.to_csv(summary_file_path, index=False, encoding='utf-8')

    # best overall model 저장
    torch.save(best_overall_model_state, f"{result_dir}best_model_overall_{safe_model_name}_{method}.pt")
    print(f"Best overall model saved as: best_model_overall_{safe_model_name}_{method}.pt")

    return all_test_metrics, avg_metrics, std_metrics, best_overall_model_state, best_run_id


def main():
   # 데이터 분할 (7:1.5:1.5)

    prompt_list = rs_sample['info.essay_prompt'].unique().tolist()
    train_prompts, temp_prompts = train_test_split(prompt_list,
                                                   test_size=0.3,
                                                   random_state=42,
                                                   )
    val_prompts, test_prompts = train_test_split(temp_prompts,
                                                 test_size=0.5,
                                                 random_state=42,
                                                 )
    train_data = rs_sample[rs_sample['info.essay_prompt'].isin(train_prompts)].reset_index(drop=True)
    val_data = rs_sample[rs_sample['info.essay_prompt'].isin(val_prompts)].reset_index(drop=True)
    test_data = rs_sample[rs_sample['info.essay_prompt'].isin(test_prompts)].reset_index(drop=True)



    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    auroc_weight = 0.0  # 이 값을 조정하여 AUROC의 중요도 설정 (0~1 사이 값)

    all_test_metrics, avg_metrics, std_metrics, best_model_state, best_run_id = run_multiple_experiments(
        model_class=model_class,
        model_name=model_name,
        train_data=train_data,
        val_data=val_data,
        test_data=test_data,
        tokenizer=tokenizer,
        device=device,
        num_runs=3,
        learning_rate=3e-5,
        num_epochs=5,
        batch_size=16,
        accumulation_steps=2,
        seed=42,
        auroc_weight=auroc_weight  # AUROC 가중치 전달
    )

    # Final evaluation with best model
    print("\n===== Final Evaluation with Best Overall Model =====")
    final_model = model_class.from_pretrained(model_name, num_labels=2)
    final_model.load_state_dict(best_model_state)
    final_model.to(device)

    test_dataloader = data_preparing(test_data, tokenizer, batch_size=16, shuffle=False)
    _, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr,_ = eval_model(final_model, test_dataloader, device)


start_time = time.time()

if __name__ == "__main__":
    main()

elapsed_time_sec = time.time() - start_time
elapsed_time_min = elapsed_time_sec / 60
print(f"소요 시간: {elapsed_time_min:.2f}분")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Using combined metric with AUROC weight: 0.00, TPR@FPR=0.01% weight: 1.00

===== Starting Run 1/3 =====
-+-+-+ Run 1: Initial Validation Metrics +-+-+-
Loss: 0.7471 | Acc: 0.3912 | F1: 0.5622 | Precision: 0.3912 | Recall: 0.9993 | AUROC: 0.5788 | TPR@FPR=0.01%: 0.0007
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 1, Epoch 1 성능 평가:
Loss: 0.0225 | Acc: 0.9940 | F1: 0.9924 | Precision: 0.9890 | Recall: 0.9958 | AUROC: 0.9996 | TPR@FPR=0.01%: 0.7966
Run 1: New best model found with combined score: 0.7966
  - AUROC: 0.9996 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.7966 (weight: 1.00)
Run 1, Epoch 2 성능 평가:
Loss: 0.0303 | Acc: 0.9900 | F1: 0.9873 | Precision: 0.9763 | Recall: 0.9986 | AUROC: 0.9994 | TPR@FPR=0.01%: 0.1326
Run 1: No improvement. Early stop counter = 1/2
Run 1, Epoch 3 성능 평가:
Loss: 0.0153 | Acc: 0.9949 | F1: 0.9934 | Precision: 0.9993 | Recall: 0.9875 | AUROC: 0.9999 | TPR@FPR=0.01%: 0.9778
Run 1: New best model found with combined score: 0.9778
  - AUROC: 0.9999 (weight: 0.00)
  - 

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 2: Initial Validation Metrics +-+-+-
Loss: 0.6649 | Acc: 0.6090 | F1: 0.0014 | Precision: 1.0000 | Recall: 0.0007 | AUROC: 0.6893 | TPR@FPR=0.01%: 0.0007
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 2, Epoch 1 성능 평가:
Loss: 0.0250 | Acc: 0.9935 | F1: 0.9916 | Precision: 0.9965 | Recall: 0.9868 | AUROC: 0.9996 | TPR@FPR=0.01%: 0.8435
Run 2: New best model found with combined score: 0.8435
  - AUROC: 0.9996 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.8435 (weight: 1.00)
Run 2, Epoch 2 성능 평가:
Loss: 0.0352 | Acc: 0.9892 | F1: 0.9863 | Precision: 0.9743 | Recall: 0.9986 | AUROC: 0.9998 | TPR@FPR=0.01%: 0.8809
Run 2: New best model found with combined score: 0.8809
  - AUROC: 0.9998 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.8809 (weight: 1.00)
Run 2, Epoch 3 성능 평가:
Loss: 0.0377 | Acc: 0.9919 | F1: 0.9897 | Precision: 0.9810 | Recall: 0.9986 | AUROC: 0.9990 | TPR@FPR=0.01%: 0.0205
Run 2: No improvement. Early stop counter = 1/2
Run 2, Epoch 4 성능 평가:
Loss: 0.0087 | Acc: 0.9967 | F1: 0.9959 | Pr

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 3: Initial Validation Metrics +-+-+-
Loss: 0.6664 | Acc: 0.6090 | F1: 0.0014 | Precision: 1.0000 | Recall: 0.0007 | AUROC: 0.5530 | TPR@FPR=0.01%: 0.0007
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 3, Epoch 1 성능 평가:
Loss: 0.0322 | Acc: 0.9897 | F1: 0.9869 | Precision: 0.9828 | Recall: 0.9910 | AUROC: 0.9984 | TPR@FPR=0.01%: 0.0284
Run 3: New best model found with combined score: 0.0284
  - AUROC: 0.9984 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.0284 (weight: 1.00)
Run 3, Epoch 2 성능 평가:
Loss: 0.0254 | Acc: 0.9919 | F1: 0.9897 | Precision: 0.9823 | Recall: 0.9972 | AUROC: 0.9997 | TPR@FPR=0.01%: 0.8920
Run 3: New best model found with combined score: 0.8920
  - AUROC: 0.9997 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.8920 (weight: 1.00)
Run 3, Epoch 3 성능 평가:
Loss: 0.0147 | Acc: 0.9938 | F1: 0.9921 | Precision: 0.9876 | Recall: 0.9965 | AUROC: 0.9999 | TPR@FPR=0.01%: 0.9418
Run 3: New best model found with combined score: 0.9418
  - AUROC: 0.9999 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.94

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loss: 0.0723 | Acc: 0.9809 | F1: 0.9759 | Precision: 0.9993 | Recall: 0.9537 | AUROC: 0.9997 | TPR@FPR=0.01%: 0.9100
소요 시간: 197.41분


In [ ]:
# predefined3


result_dir = f"/content/drive/My Drive/cdf/"
safe_model_name = model_name.split('/')[-1] if '/' in model_name else model_name
method = "predefined3"



class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.texts = dataframe['txt'].tolist()
        self.difficulty_score = dataframe['difficulty_score3'].tolist()  #difficulty_score1 difficulty_score3 difficulty_score3
        self.labels = dataframe['target'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': tokenized['input_ids'].squeeze(0),  # 차원 축소 유지
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)  # 데이터 타입 명확히 설정
        }

def re_weighting_sequence(target_series, current_epoch, total_epoch=5):

    easy_threshold = min(1.0, current_epoch*1.0/total_epoch)

    hard_weight = 0.2 + (current_epoch / total_epoch) * 0.8
    res_series = [1 if e <= easy_threshold else hard_weight for e in target_series] # 점진적으로 어려운 샘플에 더 높은 가중치 부여

    return res_series


def data_preparing(data_src, tokenizer, batch_size=16, shuffle=True, return_dataset=False):
    dataset = TextDataset(data_src, tokenizer)

    if return_dataset:
        return dataset

    dataloader = DataLoader(
        dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )
    return dataloader

def loader_return(dataframe, tokenizer, current_epoch, total_epoch, batch_size=16):
    target_col = dataframe['difficulty_score3']
    weights = re_weighting_sequence(target_col, current_epoch=current_epoch, total_epoch=total_epoch)

    dataset = TextDataset(dataframe, tokenizer)
    sampler = WeightedRandomSampler(weights=weights, num_samples=len(dataset), replacement=True)

    dataloader = DataLoader(dataset, batch_size=batch_size, sampler=sampler)

    return dataloader


def eval_model(
    model,
    eval_dataloader,
    device,
    num_cls=2,
    data_source=None,
    phase=None,
    wrong_predictions_list=None,
    result_file=None  # 추가
):
    total_loss = 0.0
    total_count = 0
    all_preds, all_labels, all_probs = [], [], []

    model.eval()
    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss
            total_loss += loss.item() * batch['labels'].size(0)
            total_count += batch['labels'].size(0)

            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = torch.argmax(logits, dim=-1)
            labels = batch['labels']

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = total_loss / total_count
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs) if not np.all(all_probs == all_probs[0]) else 0.0

    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    tpr_at_fpr_0001 = np.interp(0.01 / 100, fpr, tpr) if len(fpr) > 0 else 0.0

    print(f"Loss: {avg_loss:.4f} | Acc: {accuracy:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | AUROC: {auc:.4f} | TPR@FPR=0.01%: {tpr_at_fpr_0001:.4f}")

    if data_source is not None and 'txt' in data_source:
        txt_data = [text for text in data_source['txt']]
    else:
        txt_data = []

    # 결과 저장
    if result_file is not None:
        df_result = pd.DataFrame({
            'label': all_labels,
            'txt' : txt_data ,
            'pred': all_preds,
            'prob': all_probs
        })
        df_result.to_csv(result_file, index=False, encoding='utf-8-sig')

    return avg_loss, accuracy, f1, precision, recall, auc, tpr_at_fpr_0001, (fpr, tpr)



def train_model(
    model,
    train_dataloader,
    valid_dataloader,
    test_dataloader,
    device,
    learning_rate=3e-5,
    num_epochs=5,
    accumulation_steps=2,
    num_cls=2,
    run_id=0,
    early_stopping_patience=2,
    train_data=None,
    val_data=None,
    test_data=None,
    auroc_weight=0.0,
    result_file=None
):
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    model.to(device)

    print(f'-+-+-+ Run {run_id+1}: Initial Validation Metrics +-+-+-')
    eval_model(model, valid_dataloader, device, num_cls, data_source=val_data, phase="initial_val")
    print('-+-+-+ END OF INITIAL EVALUATION +-+-+-')

    train_loss_log = []
    valid_loss_log = []
    best_combined_score = 0.0  # 최고 결합 점수 초기화
    best_model_state = None
    early_stop_counter = 0
    all_wrong_predictions = []

    # 최적 성능 기록용
    best_valid_auroc = 0.0
    best_valid_tpr = 0.0

    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_count = 0
        phase_name = f"val_epoch{epoch+1}"
        valid_result_file = f"{result_dir}valid_result_run{run_id+1}_epoch{epoch+1}_{safe_model_name}_{method}.csv"
        model.train()  # 에폭 시작 시 그래디언트 초기화

        train_dataloader_current = loader_return(train_data, tokenizer, epoch + 1, num_epochs)

        for step, batch in enumerate(train_dataloader_current):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss / accumulation_steps
            total_train_loss += loss.item()
            total_count += 1

            loss.backward()

            if (step + 1) % accumulation_steps == 0  or (step + 1) == len(train_dataloader) :
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()


        avg_train_loss = total_train_loss / total_count
        train_loss_log.append(avg_train_loss)
        if val_data is None:
            raise ValueError("Validation data is None.")
        if test_data is None:
            raise ValueError("Test data is None.")

        print(f'Run {run_id+1}, Epoch {epoch + 1} 성능 평가:')
        # 현재 에폭에서 검증 데이터 평가 및 틀린 예측 저장
        phase_name = f"val_epoch{epoch+1}"

        avg_valid_loss, valid_acc, valid_f1, valid_precision, valid_recall, valid_auc, valid_tpr, _ = eval_model(
            model, valid_dataloader, device, num_cls,
            data_source=val_data,
            phase=phase_name,
            result_file=valid_result_file
        )
        valid_loss_log.append(avg_valid_loss)


       # AUROC와 TPR@FPR=0.01%를 결합한 점수 계산
        combined_score = auroc_weight * valid_auc + (1 - auroc_weight) * valid_tpr

        # Save the best model based on combined score
        if combined_score > best_combined_score:
            best_combined_score = combined_score
            best_valid_auroc = valid_auc  # 최고 성능일 때 AUROC 값 저장
            best_valid_tpr = valid_tpr    # 최고 성능일 때 TPR 값 저장
            best_model_state = model.state_dict().copy()
            early_stop_counter = 0
            print(f"Run {run_id+1}: New best model found with combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {valid_auc:.4f} (weight: {auroc_weight:.2f})")
            print(f"  - TPR @ FPR=0.01%: {valid_tpr:.4f} (weight: {(1-auroc_weight):.2f})")
        else:
            early_stop_counter += 1
            print(f"Run {run_id+1}: No improvement. Early stop counter = {early_stop_counter}/{early_stopping_patience}")

        # Early stopping 조건 만족 시 중단
        if early_stop_counter >= early_stopping_patience:
            print(f"Run {run_id+1}: Early stopping triggered at epoch {epoch + 1}")
            break

    # 최고 성능 모델로 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Run {run_id+1}: Restored best model for final evaluation")


    print(f'----- Run {run_id+1}: Final Evaluation on Test Data -----')
    test_result_file = f"{result_dir}test_result_run{run_id+1}_{safe_model_name}_{method}.csv"
    test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, _ = eval_model(
        model, test_dataloader, device, num_cls,
        data_source=test_data,
        phase="test",
        result_file=test_result_file
    )
    loss_log_df = pd.DataFrame({'train_loss_log': train_loss_log,'valid_loss_log': valid_loss_log})
    loss_log_df.to_csv(f"{result_dir}loss_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8')


    return train_loss_log, valid_loss_log, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score

# seed 추가
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True) # 추가


def run_multiple_experiments(
    model_class,
    model_name,
    train_data,
    val_data,
    test_data,
    tokenizer,
    device,
    num_runs=3,
    learning_rate=3e-5,
    num_epochs=5,
    batch_size=16,
    accumulation_steps=2,
    num_cls=2,
    seed=42,
    auroc_weight=0.0  # AUROC 가중치 추가
):
    # seed=42 설정
    set_seed(seed)

    # 결과 저장을 위한 리스트
    all_test_metrics = []
    best_overall_score = 0.0  # 전체 실험에서 최고 결합 점수
    best_overall_auroc = 0.0  # 최고 모델의 AUROC
    best_overall_tpr = 0.0    # 최고 모델의 TPR
    best_overall_model_state = None
    best_run_id = -1

    # 데이터로더 생성
    train_dataset = data_preparing(train_data, tokenizer, batch_size=batch_size, shuffle=True, return_dataset=True)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    valid_dataloader = data_preparing(val_data, tokenizer, batch_size=batch_size, shuffle=False)
    test_dataloader = data_preparing(test_data, tokenizer, batch_size=batch_size, shuffle=False)

    print(f"\nUsing combined metric with AUROC weight: {auroc_weight:.2f}, TPR@FPR=0.01% weight: {(1-auroc_weight):.2f}")

    for run in range(num_runs):
        print(f"\n===== Starting Run {run+1}/{num_runs} =====")

        # 새로운 모델 인스턴스 생성
        model = model_class.from_pretrained(model_name, num_labels=num_cls)

        # 모델 학습 - 원본 데이터 전달
        _, _, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score= train_model(
            model=model,
            train_dataloader=train_dataloader,
            valid_dataloader=valid_dataloader,
            test_dataloader=test_dataloader,
            train_data=train_data,
            val_data=val_data,
            test_data=test_data,
            device=device,
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            accumulation_steps=accumulation_steps,
            num_cls=num_cls,
            run_id=run
        )

        # 테스트 메트릭 저장
        run_metrics = {
            'loss': test_loss,
            'accuracy': test_acc,
            'f1': test_f1,
            'precision': test_precision,
            'recall': test_recall,
            'auc': test_auc,
            'tpr_at_fpr_0001': test_tpr,
            'valid_auroc': best_valid_auroc,
            'valid_tpr': best_valid_tpr,
            'combined_score': best_combined_score
        }
        all_test_metrics.append(run_metrics)

        # Update best overall model based on combined score
        if best_combined_score > best_overall_score:
            best_overall_score = best_combined_score
            best_overall_auroc = best_valid_auroc
            best_overall_tpr = best_valid_tpr
            best_overall_model_state = best_model_state
            best_run_id = run

            # best model 저장
            torch.save(best_model_state, f"{result_dir}best_model_run_{run+1}_score_{best_combined_score:.4f}_auroc_{best_valid_auroc:.4f}_tpr_{best_valid_tpr:.4f}_{safe_model_name}_{method}.pt")
            print(f"New overall best model saved from run {run+1}")
            print(f"  - Combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {best_valid_auroc:.4f}")
            print(f"  - TPR @ FPR=0.01%: {best_valid_tpr:.4f}")

    # 평균 성능 계산
    avg_metrics = {metric: np.mean([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}
    std_metrics = {metric: np.std([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}

    print("\n===== Average Test Results Across All Runs =====")
    print(f"Avg Test Loss: {avg_metrics['loss']:.4f} ± {std_metrics['loss']:.4f}")
    print(f"Avg Test Accuracy: {avg_metrics['accuracy']:.4f} ± {std_metrics['accuracy']:.4f}")
    print(f"Avg Test F1 Score: {avg_metrics['f1']:.4f} ± {std_metrics['f1']:.4f}")
    print(f"Avg Test Precision: {avg_metrics['precision']:.4f} ± {std_metrics['precision']:.4f}")
    print(f"Avg Test Recall: {avg_metrics['recall']:.4f} ± {std_metrics['recall']:.4f}")
    print(f"Avg Test AUROC: {avg_metrics['auc']:.4f} ± {std_metrics['auc']:.4f}")
    print(f"Avg Test TPR @ FPR=0.01%: {avg_metrics['tpr_at_fpr_0001']:.4f} ± {std_metrics['tpr_at_fpr_0001']:.4f}")
    print(f"Avg Validation AUROC: {avg_metrics['valid_auroc']:.4f} ± {std_metrics['valid_auroc']:.4f}")
    print(f"Avg Validation TPR @ FPR=0.01%: {avg_metrics['valid_tpr']:.4f} ± {std_metrics['valid_tpr']:.4f}")
    print(f"Avg Combined Score: {avg_metrics['combined_score']:.4f} ± {std_metrics['combined_score']:.4f}")

    print(f"\n===== Best Model from Run {best_run_id+1} =====")
    print(f"Combined Score: {best_overall_score:.4f}")
    print(f"Best Validation AUROC: {best_overall_auroc:.4f}")
    print(f"Best Validation TPR: {best_overall_tpr:.4f}")
    print(f"Saved as: best_model_run_{best_run_id+1}_score_{best_overall_score:.4f}_auroc_{best_overall_auroc:.4f}_tpr_{best_overall_tpr:.4f}_{safe_model_name}_{method}.pt")

    # 평균 성능 저장
    summary_df = pd.DataFrame({
      'metric': list(avg_metrics.keys()),
      'mean': [avg_metrics[m] for m in avg_metrics],
      'std': [std_metrics[m] for m in std_metrics]
    })
    summary_file_path = os.path.join(result_dir, f"summary_metrics_{safe_model_name}_{method}.csv")
    summary_df.to_csv(summary_file_path, index=False, encoding='utf-8')

    # best overall model 저장
    torch.save(best_overall_model_state, f"{result_dir}best_model_overall_{safe_model_name}_{method}.pt")
    print(f"Best overall model saved as: best_model_overall_{safe_model_name}_{method}.pt")

    return all_test_metrics, avg_metrics, std_metrics, best_overall_model_state, best_run_id


def main():
   # 데이터 분할 (7:1.5:1.5)

    prompt_list = rs_sample['info.essay_prompt'].unique().tolist()
    train_prompts, temp_prompts = train_test_split(prompt_list,
                                                   test_size=0.3,
                                                   random_state=42,
                                                   )
    val_prompts, test_prompts = train_test_split(temp_prompts,
                                                 test_size=0.5,
                                                 random_state=42,
                                                 )
    train_data = rs_sample[rs_sample['info.essay_prompt'].isin(train_prompts)].reset_index(drop=True)
    val_data = rs_sample[rs_sample['info.essay_prompt'].isin(val_prompts)].reset_index(drop=True)
    test_data = rs_sample[rs_sample['info.essay_prompt'].isin(test_prompts)].reset_index(drop=True)



    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    auroc_weight = 0.0  # 이 값을 조정하여 AUROC의 중요도 설정 (0~1 사이 값)

    all_test_metrics, avg_metrics, std_metrics, best_model_state, best_run_id = run_multiple_experiments(
        model_class=model_class,
        model_name=model_name,
        train_data=train_data,
        val_data=val_data,
        test_data=test_data,
        tokenizer=tokenizer,
        device=device,
        num_runs=3,
        learning_rate=3e-5,
        num_epochs=5,
        batch_size=16,
        accumulation_steps=2,
        seed=42,
        auroc_weight=auroc_weight  # AUROC 가중치 전달
    )

    # Final evaluation with best model
    print("\n===== Final Evaluation with Best Overall Model =====")
    final_model = model_class.from_pretrained(model_name, num_labels=2)
    final_model.load_state_dict(best_model_state)
    final_model.to(device)

    test_dataloader = data_preparing(test_data, tokenizer, batch_size=16, shuffle=False)
    _, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr,_ = eval_model(final_model, test_dataloader, device)


start_time = time.time()

if __name__ == "__main__":
    main()

elapsed_time_sec = time.time() - start_time
elapsed_time_min = elapsed_time_sec / 60
print(f"소요 시간: {elapsed_time_min:.2f}분")


Using combined metric with AUROC weight: 0.00, TPR@FPR=0.01% weight: 1.00

===== Starting Run 1/3 =====


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 1: Initial Validation Metrics +-+-+-


In [ ]:
# kobert 출력

tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")
model_class = AutoModelForSequenceClassification
model_name = "monologg/kobert"

tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

The repository for monologg/kobert contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/monologg/kobert.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


tokenization_kobert.py:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/monologg/kobert:
- tokenization_kobert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_78b3253a26.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

In [ ]:
result_dir = f"/content/drive/My Drive/cdf/"

safe_model_name = model_name.split('/')[-1] if '/' in model_name else model_name
method = "adaptive"

class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.texts = dataframe['txt'].tolist()
        self.labels = dataframe['target'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': tokenized['input_ids'].squeeze(0),  # 차원 축소 유지
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)  # 데이터 타입 명확히 설정
        }

def re_weighting_sequence(target_series, mode='adaptive'):
    wrong_cnt = np.sum(target_series)*1.0
    right_cnt = len(target_series)*1.0 - wrong_cnt
    if wrong_cnt == 0:
        wrong_cnt = 1
    if mode == 'adaptive':
        res_series = [right_cnt*1.0 / wrong_cnt if e == 1 else 1 for e in target_series]
    elif mode == 'static':
        res_series = [2 if e == 1 else 1 for e in target_series]

    return res_series

def wrong_first_dataloader(model, tokenized_datasets, batch_size=16, num_cls=2, current_epoch=1, device=torch.device("cpu")):
    if current_epoch == 1:
        return DataLoader(tokenized_datasets, shuffle=True, batch_size=batch_size)

    eval_dataloader = DataLoader(tokenized_datasets, shuffle=False, batch_size=1)

    all_preds = []
    all_labels = []
    right_wrong_list = []

    model.eval()
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )


        predictions = torch.argmax(outputs.logits, dim=-1)
        all_preds.append(predictions.item())
        all_labels.append(batch["labels"].item())

        right_wrong_list.append(1 if predictions.item() != batch["labels"].item() else 0)

    # 샘플링 가중치 설정
    weights = re_weighting_sequence(right_wrong_list)

    # 가중치 길이가 맞는지 확인
    if len(weights) != len(tokenized_datasets):
        print(f"Warning: weights length {len(weights)} does not match dataset size {len(tokenized_datasets)}")

    sampler = WeightedRandomSampler(weights=weights, num_samples=len(tokenized_datasets), replacement=True)

    return DataLoader(tokenized_datasets, batch_size=batch_size, sampler=sampler)




def data_preparing(data_src, tokenizer, batch_size=16, shuffle=True, return_dataset=False):
    dataset = TextDataset(data_src, tokenizer)

    if return_dataset:
        return dataset

    dataloader = DataLoader(
        dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )
    return dataloader


def eval_model(
    model,
    eval_dataloader,
    device,
    num_cls=2,
    data_source=None,
    phase=None,
    wrong_predictions_list=None,
    result_file=None  # 추가
):
    total_loss = 0.0
    total_count = 0
    all_preds, all_labels, all_probs = [], [], []

    model.eval()
    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss
            total_loss += loss.item() * batch['labels'].size(0)
            total_count += batch['labels'].size(0)

            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = torch.argmax(logits, dim=-1)
            labels = batch['labels']

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = total_loss / total_count
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs) if not np.all(all_probs == all_probs[0]) else 0.0

    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    tpr_at_fpr_0001 = np.interp(0.01 / 100, fpr, tpr) if len(fpr) > 0 else 0.0

    print(f"Loss: {avg_loss:.4f} | Acc: {accuracy:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | AUROC: {auc:.4f} | TPR@FPR=0.01%: {tpr_at_fpr_0001:.4f}")

    if data_source is not None and 'txt' in data_source:
        txt_data = [text for text in data_source['txt']]
    else:
        txt_data = []

    # 결과 저장
    if result_file is not None:
        df_result = pd.DataFrame({
            'label': all_labels,
            'txt' : txt_data ,
            'pred': all_preds,
            'prob': all_probs
        })
        df_result.to_csv(result_file, index=False, encoding='utf-8-sig')

    return avg_loss, accuracy, f1, precision, recall, auc, tpr_at_fpr_0001, (fpr, tpr)


def train_model(
    model,
    train_dataloader,
    valid_dataloader,
    test_dataloader,
    device,
    learning_rate=3e-5,
    num_epochs=5,
    accumulation_steps=2,
    num_cls=2,
    run_id=0,
    early_stopping_patience=2,
    train_data=None,
    val_data=None,
    test_data=None,
    auroc_weight=0.0,
    result_file=None
):
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    model.to(device)
    print(f'-+-+-+ Run {run_id+1}: Initial Validation Metrics +-+-+-')
    eval_model(model, valid_dataloader, device, num_cls)
    print('-+-+-+ END OF INITIAL EVALUATION +-+-+-')


    train_loss_log = []
    valid_loss_log = []
    best_combined_score = 0.0  # 최고 결합 점수 초기화
    best_model_state = None
    early_stop_counter = 0
    all_wrong_predictions = []

    # 최적 성능 기록용
    best_valid_auroc = 0.0
    best_valid_tpr = 0.0

    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_count = 0
        phase_name = f"val_epoch{epoch+1}"
        valid_result_file = f"{result_dir}valid_result_run{run_id+1}_epoch{epoch+1}_{safe_model_name}_{method}.csv"
        model.train()  # 에폭 시작 시 그래디언트 초기화

        train_dataloader_current = wrong_first_dataloader(
            model,
            train_dataloader.dataset,
            batch_size=train_dataloader.batch_size,
            num_cls=num_cls,
            current_epoch=epoch+1,
            device=device
        )

        for step, batch in enumerate(train_dataloader_current) :
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss / accumulation_steps
            total_train_loss += loss.item()
            total_count += 1

            loss.backward()

            if (step + 1) % accumulation_steps == 0  or (step + 1) == len(train_dataloader) :
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()


        avg_train_loss = total_train_loss / total_count
        train_loss_log.append(avg_train_loss)
        if val_data is None:
            raise ValueError("Validation data is None.")
        if test_data is None:
            raise ValueError("Test data is None.")

        print(f'Run {run_id+1}, Epoch {epoch + 1} 성능 평가:')
        # 현재 에폭에서 검증 데이터 평가 및 틀린 예측 저장
        phase_name = f"val_epoch{epoch+1}"

        avg_valid_loss, valid_acc, valid_f1, valid_precision, valid_recall, valid_auc, valid_tpr, _ = eval_model(
            model, valid_dataloader, device, num_cls,
            data_source=val_data,
            phase=phase_name,
            result_file=valid_result_file
        )
        valid_loss_log.append(avg_valid_loss)



       # AUROC와 TPR@FPR=0.01%를 결합한 점수 계산
        combined_score = auroc_weight * valid_auc + (1 - auroc_weight) * valid_tpr

        # Save the best model based on combined score
        if combined_score > best_combined_score:
            best_combined_score = combined_score
            best_valid_auroc = valid_auc  # 최고 성능일 때 AUROC 값 저장
            best_valid_tpr = valid_tpr    # 최고 성능일 때 TPR 값 저장
            best_model_state = model.state_dict().copy()
            early_stop_counter = 0
            print(f"Run {run_id+1}: New best model found with combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {valid_auc:.4f} (weight: {auroc_weight:.2f})")
            print(f"  - TPR @ FPR=0.01%: {valid_tpr:.4f} (weight: {(1-auroc_weight):.2f})")
        else:
            early_stop_counter += 1
            print(f"Run {run_id+1}: No improvement. Early stop counter = {early_stop_counter}/{early_stopping_patience}")

        # Early stopping 조건 만족 시 중단
        if early_stop_counter >= early_stopping_patience:
            print(f"Run {run_id+1}: Early stopping triggered at epoch {epoch + 1}")
            break

    # 최고 성능 모델로 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Run {run_id+1}: Restored best model for final evaluation")


    print(f'----- Run {run_id+1}: Final Evaluation on Test Data -----')
    test_result_file = f"{result_dir}test_result_run{run_id+1}_{safe_model_name}_{method}.csv"
    test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, _ = eval_model(
        model, test_dataloader, device, num_cls,
        data_source=test_data,
        phase="test",
        result_file=test_result_file
    )
    loss_log_df = pd.DataFrame({'train_loss_log': train_loss_log,'valid_loss_log': valid_loss_log})
    loss_log_df.to_csv(f"{result_dir}loss_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8')


    return train_loss_log, valid_loss_log, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score

# seed 추가
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True) # 추가


def run_multiple_experiments(
    model_class,
    model_name,
    train_data,
    val_data,
    test_data,
    tokenizer,
    device,
    num_runs=3,
    learning_rate=3e-5,
    num_epochs=5,
    batch_size=16,
    accumulation_steps=2,
    num_cls=2,
    seed=42,
    auroc_weight=0.0  # AUROC 가중치 추가
):
    # seed=42 설정
    set_seed(seed)

    # 결과 저장을 위한 리스트
    all_test_metrics = []
    best_overall_score = 0.0  # 전체 실험에서 최고 결합 점수
    best_overall_auroc = 0.0  # 최고 모델의 AUROC
    best_overall_tpr = 0.0    # 최고 모델의 TPR
    best_overall_model_state = None
    best_run_id = -1

    # 데이터로더 생성
    train_dataset = data_preparing(train_data, tokenizer, batch_size=batch_size, shuffle=True, return_dataset=True)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    valid_dataloader = data_preparing(val_data, tokenizer, batch_size=batch_size, shuffle=False)
    test_dataloader = data_preparing(test_data, tokenizer, batch_size=batch_size, shuffle=False)

    print(f"\nUsing combined metric with AUROC weight: {auroc_weight:.2f}, TPR@FPR=0.01% weight: {(1-auroc_weight):.2f}")

    for run in range(num_runs):
        print(f"\n===== Starting Run {run+1}/{num_runs} =====")

        # 새로운 모델 인스턴스 생성
        model = model_class.from_pretrained(model_name, num_labels=num_cls)

        # 모델 학습 - 원본 데이터 전달
        _, _, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score= train_model(
            model=model,
            train_dataloader=train_dataloader,
            valid_dataloader=valid_dataloader,
            test_dataloader=test_dataloader,
            train_data=train_data,
            val_data=val_data,
            test_data=test_data,
            device=device,
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            accumulation_steps=accumulation_steps,
            num_cls=num_cls,
            run_id=run
        )

        # 테스트 메트릭 저장
        run_metrics = {
            'loss': test_loss,
            'accuracy': test_acc,
            'f1': test_f1,
            'precision': test_precision,
            'recall': test_recall,
            'auc': test_auc,
            'tpr_at_fpr_0001': test_tpr,
            'valid_auroc': best_valid_auroc,
            'valid_tpr': best_valid_tpr,
            'combined_score': best_combined_score
        }
        all_test_metrics.append(run_metrics)

        # Update best overall model based on combined score
        if best_combined_score > best_overall_score:
            best_overall_score = best_combined_score
            best_overall_auroc = best_valid_auroc
            best_overall_tpr = best_valid_tpr
            best_overall_model_state = best_model_state
            best_run_id = run

            # best model 저장
            torch.save(best_model_state, f"{result_dir}best_model_run_{run+1}_score_{best_combined_score:.4f}_auroc_{best_valid_auroc:.4f}_tpr_{best_valid_tpr:.4f}_{safe_model_name}_{method}.pt")
            print(f"New overall best model saved from run {run+1}")
            print(f"  - Combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {best_valid_auroc:.4f}")
            print(f"  - TPR @ FPR=0.01%: {best_valid_tpr:.4f}")

    # 평균 성능 계산
    avg_metrics = {metric: np.mean([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}
    std_metrics = {metric: np.std([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}

    print("\n===== Average Test Results Across All Runs =====")
    print(f"Avg Test Loss: {avg_metrics['loss']:.4f} ± {std_metrics['loss']:.4f}")
    print(f"Avg Test Accuracy: {avg_metrics['accuracy']:.4f} ± {std_metrics['accuracy']:.4f}")
    print(f"Avg Test F1 Score: {avg_metrics['f1']:.4f} ± {std_metrics['f1']:.4f}")
    print(f"Avg Test Precision: {avg_metrics['precision']:.4f} ± {std_metrics['precision']:.4f}")
    print(f"Avg Test Recall: {avg_metrics['recall']:.4f} ± {std_metrics['recall']:.4f}")
    print(f"Avg Test AUROC: {avg_metrics['auc']:.4f} ± {std_metrics['auc']:.4f}")
    print(f"Avg Test TPR @ FPR=0.01%: {avg_metrics['tpr_at_fpr_0001']:.4f} ± {std_metrics['tpr_at_fpr_0001']:.4f}")
    print(f"Avg Validation AUROC: {avg_metrics['valid_auroc']:.4f} ± {std_metrics['valid_auroc']:.4f}")
    print(f"Avg Validation TPR @ FPR=0.01%: {avg_metrics['valid_tpr']:.4f} ± {std_metrics['valid_tpr']:.4f}")
    print(f"Avg Combined Score: {avg_metrics['combined_score']:.4f} ± {std_metrics['combined_score']:.4f}")

    print(f"\n===== Best Model from Run {best_run_id+1} =====")
    print(f"Combined Score: {best_overall_score:.4f}")
    print(f"Best Validation AUROC: {best_overall_auroc:.4f}")
    print(f"Best Validation TPR: {best_overall_tpr:.4f}")
    print(f"Saved as: best_model_run_{best_run_id+1}_score_{best_overall_score:.4f}_auroc_{best_overall_auroc:.4f}_tpr_{best_overall_tpr:.4f}_{safe_model_name}_{method}.pt")

    # 평균 성능 저장
    summary_df = pd.DataFrame({
      'metric': list(avg_metrics.keys()),
      'mean': [avg_metrics[m] for m in avg_metrics],
      'std': [std_metrics[m] for m in std_metrics]
    })
    summary_file_path = os.path.join(result_dir, f"summary_metrics_{safe_model_name}_{method}.csv")
    summary_df.to_csv(summary_file_path, index=False, encoding='utf-8')

    # best overall model 저장
    torch.save(best_overall_model_state, f"{result_dir}best_model_overall_{safe_model_name}_{method}.pt")
    print(f"Best overall model saved as: best_model_overall_{safe_model_name}_{method}.pt")

    return all_test_metrics, avg_metrics, std_metrics, best_overall_model_state, best_run_id


def main():
   # 데이터 분할 (7:1.5:1.5)

    prompt_list = rs_sample['info.essay_prompt'].unique().tolist()
    train_prompts, temp_prompts = train_test_split(prompt_list,
                                                   test_size=0.3,
                                                   random_state=42,
                                                   )
    val_prompts, test_prompts = train_test_split(temp_prompts,
                                                 test_size=0.5,
                                                 random_state=42,
                                                 )
    train_data = rs_sample[rs_sample['info.essay_prompt'].isin(train_prompts)].reset_index(drop=True)
    val_data = rs_sample[rs_sample['info.essay_prompt'].isin(val_prompts)].reset_index(drop=True)
    test_data = rs_sample[rs_sample['info.essay_prompt'].isin(test_prompts)].reset_index(drop=True)



    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    auroc_weight = 0.0  # 이 값을 조정하여 AUROC의 중요도 설정 (0~1 사이 값)

    all_test_metrics, avg_metrics, std_metrics, best_model_state, best_run_id = run_multiple_experiments(
        model_class=model_class,
        model_name=model_name,
        train_data=train_data,
        val_data=val_data,
        test_data=test_data,
        tokenizer=tokenizer,
        device=device,
        num_runs=3,
        learning_rate=3e-5,
        num_epochs=5,
        batch_size=16,
        accumulation_steps=2,
        seed=42,
        auroc_weight=auroc_weight  # AUROC 가중치 전달
    )

    # Final evaluation with best model
    print("\n===== Final Evaluation with Best Overall Model =====")
    final_model = model_class.from_pretrained(model_name, num_labels=2)
    final_model.load_state_dict(best_model_state)
    final_model.to(device)

    test_dataloader = data_preparing(test_data, tokenizer, batch_size=16, shuffle=False)
    _, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr,_ = eval_model(final_model, test_dataloader, device)


start_time = time.time()

if __name__ == "__main__":
    main()

elapsed_time_sec = time.time() - start_time
elapsed_time_min = elapsed_time_sec / 60
print(f"소요 시간: {elapsed_time_min:.2f}분")


Using combined metric with AUROC weight: 0.00, TPR@FPR=0.01% weight: 1.00

===== Starting Run 1/3 =====


config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 1: Initial Validation Metrics +-+-+-
Loss: 0.7471 | Acc: 0.3912 | F1: 0.5622 | Precision: 0.3912 | Recall: 0.9993 | AUROC: 0.5788 | TPR@FPR=0.01%: 0.0007
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 1, Epoch 1 성능 평가:
Loss: 0.0185 | Acc: 0.9943 | F1: 0.9927 | Precision: 0.9904 | Recall: 0.9952 | AUROC: 0.9998 | TPR@FPR=0.01%: 0.8518
Run 1: New best model found with combined score: 0.8518
  - AUROC: 0.9998 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.8518 (weight: 1.00)
Run 1, Epoch 2 성능 평가:
Loss: 0.0277 | Acc: 0.9924 | F1: 0.9903 | Precision: 0.9930 | Recall: 0.9875 | AUROC: 0.9997 | TPR@FPR=0.01%: 0.9342
Run 1: New best model found with combined score: 0.9342
  - AUROC: 0.9997 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9342 (weight: 1.00)
Run 1, Epoch 3 성능 평가:
Loss: 0.0140 | Acc: 0.9954 | F1: 0.9941 | Precision: 0.9924 | Recall: 0.9958 | AUROC: 0.9999 | TPR@FPR=0.01%: 0.9564
Run 1: New best model found with combined score: 0.9564
  - AUROC: 0.9999 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.95

...

-+-+-+ Run 2: Initial Validation Metrics +-+-+-
Loss: 0.6694 | Acc: 0.6077 | F1: 0.0028 | Precision: 0.2500 | Recall: 0.0014 | AUROC: 0.5496 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 2, Epoch 1 성능 평가:
Loss: 0.0600 | Acc: 0.9791 | F1: 0.9740 | Precision: 0.9494 | Recall: 1.0000 | AUROC: 0.9998 | TPR@FPR=0.01%: 0.9280
Run 2: New best model found with combined score: 0.9280
  - AUROC: 0.9998 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9280 (weight: 1.00)
Run 2, Epoch 2 성능 평가:
Loss: 0.0345 | Acc: 0.9905 | F1: 0.9877 | Precision: 1.0000 | Recall: 0.9758 | AUROC: 0.9999 | TPR@FPR=0.01%: 0.9834
Run 2: New best model found with combined score: 0.9834
  - AUROC: 0.9999 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9834 (weight: 1.00)
Run 2, Epoch 3 성능 평가:
Loss: 0.0309 | Acc: 0.9900 | F1: 0.9871 | Precision: 0.9944 | Recall: 0.9799 | AUROC: 0.9997 | TPR@FPR=0.01%: 0.9287
Run 2: No improvement. Early stop counter = 1/2
Run 2, Epoch 4 성능 평가:
Loss: 0.0359 | Acc: 0.9884 | F1: 0.9850 | Pr

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 3: Initial Validation Metrics +-+-+-
Loss: 0.7151 | Acc: 0.3766 | F1: 0.5373 | Precision: 0.3786 | Recall: 0.9252 | AUROC: 0.4800 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 3, Epoch 1 성능 평가:
Loss: 0.0504 | Acc: 0.9816 | F1: 0.9759 | Precision: 0.9993 | Recall: 0.9536 | AUROC: 0.9998 | TPR@FPR=0.01%: 0.8206
Run 3: New best model found with combined score: 0.8206
  - AUROC: 0.9998 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.8206 (weight: 1.00)
Run 3, Epoch 2 성능 평가:
Loss: 0.0256 | Acc: 0.9921 | F1: 0.9900 | Precision: 0.9876 | Recall: 0.9924 | AUROC: 0.9996 | TPR@FPR=0.01%: 0.8920
Run 3: New best model found with combined score: 0.8920
  - AUROC: 0.9996 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.8920 (weight: 1.00)
Run 3, Epoch 3 성능 평가:
Loss: 0.0095 | Acc: 0.9967 | F1: 0.9958 | Precision: 0.9972 | Recall: 0.9945 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9924
Run 3: New best model found with combined score: 0.9924
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.99

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loss: 0.1410 | Acc: 0.9703 | F1: 0.9621 | Precision: 0.9964 | Recall: 0.9302 | AUROC: 0.9977 | TPR@FPR=0.01%: 0.9208
소요 시간: 230.28분


In [ ]:
# static
result_dir = f"/content/drive/My Drive/cdf/"

safe_model_name = model_name.split('/')[-1] if '/' in model_name else model_name
method = "static_rerun" #static

class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.texts = dataframe['txt'].tolist()
        self.labels = dataframe['target'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': tokenized['input_ids'].squeeze(0),  # 차원 축소 유지
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)  # 데이터 타입 명확히 설정
        }

def re_weighting_sequence(target_series, mode='static'):
    wrong_cnt = np.sum(target_series)*1.0
    right_cnt = len(target_series)*1.0 - wrong_cnt
    if wrong_cnt == 0:
        wrong_cnt = 1
    if mode == 'adaptive':
        res_series = [right_cnt*1.0 / wrong_cnt if e == 1 else 1 for e in target_series]
    elif mode == 'static':
        res_series = [2 if e == 1 else 1 for e in target_series]

    return res_series

def wrong_first_dataloader(model, tokenized_datasets, batch_size=16, num_cls=2, current_epoch=1, device=torch.device("cpu")):
    if current_epoch == 1:
        return DataLoader(tokenized_datasets, shuffle=True, batch_size=batch_size)

    eval_dataloader = DataLoader(tokenized_datasets, shuffle=False, batch_size=1)

    all_preds = []
    all_labels = []
    right_wrong_list = []

    model.eval()
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )


        predictions = torch.argmax(outputs.logits, dim=-1)
        all_preds.append(predictions.item())
        all_labels.append(batch["labels"].item())

        right_wrong_list.append(1 if predictions.item() != batch["labels"].item() else 0)

    # 샘플링 가중치 설정
    weights = re_weighting_sequence(right_wrong_list)

    # 가중치 길이가 맞는지 확인
    if len(weights) != len(tokenized_datasets):
        print(f"Warning: weights length {len(weights)} does not match dataset size {len(tokenized_datasets)}")

    sampler = WeightedRandomSampler(weights=weights, num_samples=len(tokenized_datasets), replacement=True)

    return DataLoader(tokenized_datasets, batch_size=batch_size, sampler=sampler)




def data_preparing(data_src, tokenizer, batch_size=16, shuffle=True, return_dataset=False):
    dataset = TextDataset(data_src, tokenizer)

    if return_dataset:
        return dataset

    dataloader = DataLoader(
        dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )
    return dataloader


def eval_model(
    model,
    eval_dataloader,
    device,
    num_cls=2,
    data_source=None,
    phase=None,
    wrong_predictions_list=None,
    result_file=None  # 추가
):
    total_loss = 0.0
    total_count = 0
    all_preds, all_labels, all_probs = [], [], []

    model.eval()
    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss
            total_loss += loss.item() * batch['labels'].size(0)
            total_count += batch['labels'].size(0)

            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = torch.argmax(logits, dim=-1)
            labels = batch['labels']

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = total_loss / total_count
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs) if not np.all(all_probs == all_probs[0]) else 0.0

    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    tpr_at_fpr_0001 = np.interp(0.01 / 100, fpr, tpr) if len(fpr) > 0 else 0.0

    print(f"Loss: {avg_loss:.4f} | Acc: {accuracy:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | AUROC: {auc:.4f} | TPR@FPR=0.01%: {tpr_at_fpr_0001:.4f}")

    if data_source is not None and 'txt' in data_source:
        txt_data = [text for text in data_source['txt']]
    else:
        txt_data = []

    # 결과 저장
    if result_file is not None:
        df_result = pd.DataFrame({
            'label': all_labels,
            'txt' : txt_data ,
            'pred': all_preds,
            'prob': all_probs
        })
        df_result.to_csv(result_file, index=False, encoding='utf-8-sig')

    return avg_loss, accuracy, f1, precision, recall, auc, tpr_at_fpr_0001, (fpr, tpr)


def train_model(
    model,
    train_dataloader,
    valid_dataloader,
    test_dataloader,
    device,
    learning_rate=3e-5,
    num_epochs=5,
    accumulation_steps=2,
    num_cls=2,
    run_id=0,
    early_stopping_patience=2,
    train_data=None,
    val_data=None,
    test_data=None,
    auroc_weight=0.0,
    result_file=None
):
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    model.to(device)
    print(f'-+-+-+ Run {run_id+1}: Initial Validation Metrics +-+-+-')
    eval_model(model, valid_dataloader, device, num_cls)
    print('-+-+-+ END OF INITIAL EVALUATION +-+-+-')


    train_loss_log = []
    valid_loss_log = []
    best_combined_score = 0.0  # 최고 결합 점수 초기화
    best_model_state = None
    early_stop_counter = 0
    all_wrong_predictions = []

    # 최적 성능 기록용
    best_valid_auroc = 0.0
    best_valid_tpr = 0.0

    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_count = 0
        phase_name = f"val_epoch{epoch+1}"
        valid_result_file = f"{result_dir}valid_result_run{run_id+1}_epoch{epoch+1}_{safe_model_name}_{method}.csv"
        model.train()  # 에폭 시작 시 그래디언트 초기화

        train_dataloader_current = wrong_first_dataloader(
            model,
            train_dataloader.dataset,
            batch_size=train_dataloader.batch_size,
            num_cls=num_cls,
            current_epoch=epoch+1,
            device=device
        )

        for step, batch in enumerate(train_dataloader_current) :
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss / accumulation_steps
            total_train_loss += loss.item()
            total_count += 1

            loss.backward()

            if (step + 1) % accumulation_steps == 0  or (step + 1) == len(train_dataloader) :
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()


        avg_train_loss = total_train_loss / total_count
        train_loss_log.append(avg_train_loss)
        if val_data is None:
            raise ValueError("Validation data is None.")
        if test_data is None:
            raise ValueError("Test data is None.")

        print(f'Run {run_id+1}, Epoch {epoch + 1} 성능 평가:')
        # 현재 에폭에서 검증 데이터 평가 및 틀린 예측 저장
        phase_name = f"val_epoch{epoch+1}"

        avg_valid_loss, valid_acc, valid_f1, valid_precision, valid_recall, valid_auc, valid_tpr, _ = eval_model(
            model, valid_dataloader, device, num_cls,
            data_source=val_data,
            phase=phase_name,
            result_file=valid_result_file
        )
        valid_loss_log.append(avg_valid_loss)



       # AUROC와 TPR@FPR=0.01%를 결합한 점수 계산
        combined_score = auroc_weight * valid_auc + (1 - auroc_weight) * valid_tpr

        # Save the best model based on combined score
        if combined_score > best_combined_score:
            best_combined_score = combined_score
            best_valid_auroc = valid_auc  # 최고 성능일 때 AUROC 값 저장
            best_valid_tpr = valid_tpr    # 최고 성능일 때 TPR 값 저장
            best_model_state = model.state_dict().copy()
            early_stop_counter = 0
            print(f"Run {run_id+1}: New best model found with combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {valid_auc:.4f} (weight: {auroc_weight:.2f})")
            print(f"  - TPR @ FPR=0.01%: {valid_tpr:.4f} (weight: {(1-auroc_weight):.2f})")
        else:
            early_stop_counter += 1
            print(f"Run {run_id+1}: No improvement. Early stop counter = {early_stop_counter}/{early_stopping_patience}")

        # Early stopping 조건 만족 시 중단
        if early_stop_counter >= early_stopping_patience:
            print(f"Run {run_id+1}: Early stopping triggered at epoch {epoch + 1}")
            break

    # 최고 성능 모델로 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Run {run_id+1}: Restored best model for final evaluation")


    print(f'----- Run {run_id+1}: Final Evaluation on Test Data -----')
    test_result_file = f"{result_dir}test_result_run{run_id+1}_{safe_model_name}_{method}.csv"
    test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, _ = eval_model(
        model, test_dataloader, device, num_cls,
        data_source=test_data,
        phase="test",
        result_file=test_result_file
    )
    loss_log_df = pd.DataFrame({'train_loss_log': train_loss_log,'valid_loss_log': valid_loss_log})
    loss_log_df.to_csv(f"{result_dir}loss_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8')


    return train_loss_log, valid_loss_log, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score

# seed 추가
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True) # 추가


def run_multiple_experiments(
    model_class,
    model_name,
    train_data,
    val_data,
    test_data,
    tokenizer,
    device,
    num_runs=3,
    learning_rate=3e-5,
    num_epochs=5,
    batch_size=16,
    accumulation_steps=2,
    num_cls=2,
    seed=42,
    auroc_weight=0.0  # AUROC 가중치 추가
):
    # seed=42 설정
    set_seed(seed)

    # 결과 저장을 위한 리스트
    all_test_metrics = []
    best_overall_score = 0.0  # 전체 실험에서 최고 결합 점수
    best_overall_auroc = 0.0  # 최고 모델의 AUROC
    best_overall_tpr = 0.0    # 최고 모델의 TPR
    best_overall_model_state = None
    best_run_id = -1

    # 데이터로더 생성
    train_dataset = data_preparing(train_data, tokenizer, batch_size=batch_size, shuffle=True, return_dataset=True)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    valid_dataloader = data_preparing(val_data, tokenizer, batch_size=batch_size, shuffle=False)
    test_dataloader = data_preparing(test_data, tokenizer, batch_size=batch_size, shuffle=False)

    print(f"\nUsing combined metric with AUROC weight: {auroc_weight:.2f}, TPR@FPR=0.01% weight: {(1-auroc_weight):.2f}")

    for run in range(num_runs):
        print(f"\n===== Starting Run {run+1}/{num_runs} =====")

        # 새로운 모델 인스턴스 생성
        model = model_class.from_pretrained(model_name, num_labels=num_cls)

        # 모델 학습 - 원본 데이터 전달
        _, _, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score= train_model(
            model=model,
            train_dataloader=train_dataloader,
            valid_dataloader=valid_dataloader,
            test_dataloader=test_dataloader,
            train_data=train_data,
            val_data=val_data,
            test_data=test_data,
            device=device,
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            accumulation_steps=accumulation_steps,
            num_cls=num_cls,
            run_id=run
        )

        # 테스트 메트릭 저장
        run_metrics = {
            'loss': test_loss,
            'accuracy': test_acc,
            'f1': test_f1,
            'precision': test_precision,
            'recall': test_recall,
            'auc': test_auc,
            'tpr_at_fpr_0001': test_tpr,
            'valid_auroc': best_valid_auroc,
            'valid_tpr': best_valid_tpr,
            'combined_score': best_combined_score
        }
        all_test_metrics.append(run_metrics)

        # Update best overall model based on combined score
        if best_combined_score > best_overall_score:
            best_overall_score = best_combined_score
            best_overall_auroc = best_valid_auroc
            best_overall_tpr = best_valid_tpr
            best_overall_model_state = best_model_state
            best_run_id = run

            # best model 저장
            torch.save(best_model_state, f"{result_dir}best_model_run_{run+1}_score_{best_combined_score:.4f}_auroc_{best_valid_auroc:.4f}_tpr_{best_valid_tpr:.4f}_{safe_model_name}_{method}.pt")
            print(f"New overall best model saved from run {run+1}")
            print(f"  - Combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {best_valid_auroc:.4f}")
            print(f"  - TPR @ FPR=0.01%: {best_valid_tpr:.4f}")

    # 평균 성능 계산
    avg_metrics = {metric: np.mean([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}
    std_metrics = {metric: np.std([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}

    print("\n===== Average Test Results Across All Runs =====")
    print(f"Avg Test Loss: {avg_metrics['loss']:.4f} ± {std_metrics['loss']:.4f}")
    print(f"Avg Test Accuracy: {avg_metrics['accuracy']:.4f} ± {std_metrics['accuracy']:.4f}")
    print(f"Avg Test F1 Score: {avg_metrics['f1']:.4f} ± {std_metrics['f1']:.4f}")
    print(f"Avg Test Precision: {avg_metrics['precision']:.4f} ± {std_metrics['precision']:.4f}")
    print(f"Avg Test Recall: {avg_metrics['recall']:.4f} ± {std_metrics['recall']:.4f}")
    print(f"Avg Test AUROC: {avg_metrics['auc']:.4f} ± {std_metrics['auc']:.4f}")
    print(f"Avg Test TPR @ FPR=0.01%: {avg_metrics['tpr_at_fpr_0001']:.4f} ± {std_metrics['tpr_at_fpr_0001']:.4f}")
    print(f"Avg Validation AUROC: {avg_metrics['valid_auroc']:.4f} ± {std_metrics['valid_auroc']:.4f}")
    print(f"Avg Validation TPR @ FPR=0.01%: {avg_metrics['valid_tpr']:.4f} ± {std_metrics['valid_tpr']:.4f}")
    print(f"Avg Combined Score: {avg_metrics['combined_score']:.4f} ± {std_metrics['combined_score']:.4f}")

    print(f"\n===== Best Model from Run {best_run_id+1} =====")
    print(f"Combined Score: {best_overall_score:.4f}")
    print(f"Best Validation AUROC: {best_overall_auroc:.4f}")
    print(f"Best Validation TPR: {best_overall_tpr:.4f}")
    print(f"Saved as: best_model_run_{best_run_id+1}_score_{best_overall_score:.4f}_auroc_{best_overall_auroc:.4f}_tpr_{best_overall_tpr:.4f}_{safe_model_name}_{method}.pt")

    # 평균 성능 저장
    summary_df = pd.DataFrame({
      'metric': list(avg_metrics.keys()),
      'mean': [avg_metrics[m] for m in avg_metrics],
      'std': [std_metrics[m] for m in std_metrics]
    })
    summary_file_path = os.path.join(result_dir, f"summary_metrics_{safe_model_name}_{method}.csv")
    summary_df.to_csv(summary_file_path, index=False, encoding='utf-8')

    # best overall model 저장
    torch.save(best_overall_model_state, f"{result_dir}best_model_overall_{safe_model_name}_{method}.pt")
    print(f"Best overall model saved as: best_model_overall_{safe_model_name}_{method}.pt")

    return all_test_metrics, avg_metrics, std_metrics, best_overall_model_state, best_run_id


def main():
   # 데이터 분할 (7:1.5:1.5)

    prompt_list = rs_sample['info.essay_prompt'].unique().tolist()
    train_prompts, temp_prompts = train_test_split(prompt_list,
                                                   test_size=0.3,
                                                   random_state=42,
                                                   )
    val_prompts, test_prompts = train_test_split(temp_prompts,
                                                 test_size=0.5,
                                                 random_state=42,
                                                 )
    train_data = rs_sample[rs_sample['info.essay_prompt'].isin(train_prompts)].reset_index(drop=True)
    val_data = rs_sample[rs_sample['info.essay_prompt'].isin(val_prompts)].reset_index(drop=True)
    test_data = rs_sample[rs_sample['info.essay_prompt'].isin(test_prompts)].reset_index(drop=True)



    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    auroc_weight = 0.0  # 이 값을 조정하여 AUROC의 중요도 설정 (0~1 사이 값)

    all_test_metrics, avg_metrics, std_metrics, best_model_state, best_run_id = run_multiple_experiments(
        model_class=model_class,
        model_name=model_name,
        train_data=train_data,
        val_data=val_data,
        test_data=test_data,
        tokenizer=tokenizer,
        device=device,
        num_runs=3,
        learning_rate=3e-5,
        num_epochs=5,
        batch_size=16,
        accumulation_steps=2,
        seed=42,
        auroc_weight=auroc_weight  # AUROC 가중치 전달
    )

    # Final evaluation with best model
    print("\n===== Final Evaluation with Best Overall Model =====")
    final_model = model_class.from_pretrained(model_name, num_labels=2)
    final_model.load_state_dict(best_model_state)
    final_model.to(device)

    test_dataloader = data_preparing(test_data, tokenizer, batch_size=16, shuffle=False)
    _, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr,_ = eval_model(final_model, test_dataloader, device)


start_time = time.time()

if __name__ == "__main__":
    main()

elapsed_time_sec = time.time() - start_time
elapsed_time_min = elapsed_time_sec / 60
print(f"소요 시간: {elapsed_time_min:.2f}분")


Using combined metric with AUROC weight: 0.00, TPR@FPR=0.01% weight: 1.00

===== Starting Run 1/3 =====


config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 1: Initial Validation Metrics +-+-+-
Loss: 0.7471 | Acc: 0.3912 | F1: 0.5622 | Precision: 0.3912 | Recall: 0.9993 | AUROC: 0.5788 | TPR@FPR=0.01%: 0.0007
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 1, Epoch 1 성능 평가:
Loss: 0.0160 | Acc: 0.9967 | F1: 0.9959 | Precision: 0.9945 | Recall: 0.9972 | AUROC: 0.9999 | TPR@FPR=0.01%: 0.9584
Run 1: New best model found with combined score: 0.9584
  - AUROC: 0.9999 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9584 (weight: 1.00)
Run 1, Epoch 2 성능 평가:
Loss: 0.0184 | Acc: 0.9930 | F1: 0.9909 | Precision: 0.9993 | Recall: 0.9827 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9778
Run 1: New best model found with combined score: 0.9778
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9778 (weight: 1.00)
Run 1, Epoch 3 성능 평가:
Loss: 0.0030 | Acc: 0.9989 | F1: 0.9986 | Precision: 0.9986 | Recall: 0.9986 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9945
Run 1: New best model found with combined score: 0.9945
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.99

...

-+-+-+ Run 2: Initial Validation Metrics +-+-+-
Loss: 0.6694 | Acc: 0.6077 | F1: 0.0028 | Precision: 0.2500 | Recall: 0.0014 | AUROC: 0.5496 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 2, Epoch 1 성능 평가:
Loss: 0.0468 | Acc: 0.9856 | F1: 0.9819 | Precision: 0.9684 | Recall: 0.9958 | AUROC: 0.9993 | TPR@FPR=0.01%: 0.8476
Run 2: New best model found with combined score: 0.8476
  - AUROC: 0.9993 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.8476 (weight: 1.00)
Run 2, Epoch 2 성능 평가:
Loss: 0.0817 | Acc: 0.9602 | F1: 0.9464 | Precision: 0.9992 | Recall: 0.8989 | AUROC: 0.9998 | TPR@FPR=0.01%: 0.8747
Run 2: New best model found with combined score: 0.8747
  - AUROC: 0.9998 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.8747 (weight: 1.00)
Run 2, Epoch 3 성능 평가:
Loss: 0.0719 | Acc: 0.9840 | F1: 0.9792 | Precision: 0.9978 | Recall: 0.9612 | AUROC: 0.9995 | TPR@FPR=0.01%: 0.8449
Run 2: No improvement. Early stop counter = 1/2
Run 2, Epoch 4 성능 평가:
Loss: 0.0421 | Acc: 0.9884 | F1: 0.9850 | Pr

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 3: Initial Validation Metrics +-+-+-
Loss: 0.8675 | Acc: 0.3915 | F1: 0.5624 | Precision: 0.3913 | Recall: 0.9993 | AUROC: 0.2884 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 3, Epoch 1 성능 평가:
Loss: 0.0245 | Acc: 0.9916 | F1: 0.9893 | Precision: 0.9829 | Recall: 0.9958 | AUROC: 0.9993 | TPR@FPR=0.01%: 0.0048
Run 3: New best model found with combined score: 0.0048
  - AUROC: 0.9993 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.0048 (weight: 1.00)
Run 3, Epoch 2 성능 평가:
Loss: 0.0385 | Acc: 0.9848 | F1: 0.9803 | Precision: 0.9950 | Recall: 0.9661 | AUROC: 0.9995 | TPR@FPR=0.01%: 0.8470
Run 3: New best model found with combined score: 0.8470
  - AUROC: 0.9995 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.8470 (weight: 1.00)
Run 3, Epoch 3 성능 평가:
Loss: 0.0245 | Acc: 0.9924 | F1: 0.9902 | Precision: 0.9993 | Recall: 0.9813 | AUROC: 0.9998 | TPR@FPR=0.01%: 0.6913
Run 3: No improvement. Early stop counter = 1/2
Run 3, Epoch 4 성능 평가:
Loss: 0.0175 | Acc: 0.9932 | F1: 0.9913 | Pr

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loss: 0.0786 | Acc: 0.9853 | F1: 0.9816 | Precision: 0.9959 | Recall: 0.9678 | AUROC: 0.9991 | TPR@FPR=0.01%: 0.8805
소요 시간: 247.48분


규제방식

In [ ]:
## kobert w dropout
tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")

# 모델 config 가져오기
config = AutoConfig.from_pretrained("monologg/kobert")
config.hidden_dropout_prob = 0.3  # Dropout 비율 설정
config.attention_probs_dropout_prob = 0.1  # Attention Dropout 비율 설정
config.num_labels = 2

In [ ]:
## kobert w dropout

safe_model_name = model_name.split('/')[-1] if '/' in model_name else model_name
method = "do"

class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.texts = dataframe['txt'].tolist()
        self.labels = dataframe['target'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': tokenized['input_ids'].squeeze(0),  # 차원 축소 유지
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)  # 데이터 타입 명확히 설정
        }


def data_preparing(data_src, tokenizer, batch_size=16, shuffle=True, return_dataset=False):
    dataset = TextDataset(data_src, tokenizer)

    if return_dataset:
        return dataset

    dataloader = DataLoader(
        dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )
    return dataloader


def eval_model(
    model,
    eval_dataloader,
    device,
    num_cls=2,
    data_source=None,
    phase=None,
    wrong_predictions_list=None,
    result_file=None  # 추가
):
    total_loss = 0.0
    total_count = 0
    all_preds, all_labels, all_probs = [], [], []

    model.eval()
    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss
            total_loss += loss.item() * batch['labels'].size(0)
            total_count += batch['labels'].size(0)

            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = torch.argmax(logits, dim=-1)
            labels = batch['labels']

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = total_loss / total_count
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs) if not np.all(all_probs == all_probs[0]) else 0.0

    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    tpr_at_fpr_0001 = np.interp(0.01 / 100, fpr, tpr) if len(fpr) > 0 else 0.0

    print(f"Loss: {avg_loss:.4f} | Acc: {accuracy:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | AUROC: {auc:.4f} | TPR@FPR=0.01%: {tpr_at_fpr_0001:.4f}")

    if data_source is not None and 'txt' in data_source:
        txt_data = [text for text in data_source['txt']]
    else:
        txt_data = []

    # 결과 저장
    if result_file is not None:
        df_result = pd.DataFrame({
            'label': all_labels,
            'txt' : txt_data ,
            'pred': all_preds,
            'prob': all_probs
        })
        df_result.to_csv(result_file, index=False, encoding='utf-8-sig')

    return avg_loss, accuracy, f1, precision, recall, auc, tpr_at_fpr_0001, (fpr, tpr)



def train_model(
    model,
    train_dataloader,
    valid_dataloader,
    test_dataloader,
    device,
    learning_rate=3e-5,
    num_epochs=5,
    accumulation_steps=2,
    num_cls=2,
    run_id=0,
    early_stopping_patience=2,
    train_data=None,
    val_data=None,
    test_data=None,
    auroc_weight=0.0,
    result_file=None
):
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    model.to(device)

    print(f'-+-+-+ Run {run_id+1}: Initial Validation Metrics +-+-+-')
    eval_model(model, valid_dataloader, device, num_cls, data_source=val_data, phase="initial_val")
    print('-+-+-+ END OF INITIAL EVALUATION +-+-+-')

    train_loss_log = []
    valid_loss_log = []
    best_combined_score = 0.0  # 최고 결합 점수 초기화
    best_model_state = None
    early_stop_counter = 0
    all_wrong_predictions = []

    # 최적 성능 기록용
    best_valid_auroc = 0.0
    best_valid_tpr = 0.0

    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_count = 0
        phase_name = f"val_epoch{epoch+1}"
        valid_result_file = f"{result_dir}valid_result_run{run_id+1}_epoch{epoch+1}_{safe_model_name}_{method}.csv"
        model.train()  # 에폭 시작 시 그래디언트 초기화

        for step, batch in enumerate(train_dataloader):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss / accumulation_steps
            total_train_loss += loss.item()
            total_count += 1

            loss.backward()

            if (step + 1) % accumulation_steps == 0  or (step + 1) == len(train_dataloader) :
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()


        avg_train_loss = total_train_loss / total_count
        train_loss_log.append(avg_train_loss)
        if val_data is None:
            raise ValueError("Validation data is None.")
        if test_data is None:
            raise ValueError("Test data is None.")

        print(f'Run {run_id+1}, Epoch {epoch + 1} 성능 평가:')
        # 현재 에폭에서 검증 데이터 평가 및 틀린 예측 저장
        phase_name = f"val_epoch{epoch+1}"

        avg_valid_loss, valid_acc, valid_f1, valid_precision, valid_recall, valid_auc, valid_tpr, _ = eval_model(
            model, valid_dataloader, device, num_cls,
            data_source=val_data,
            phase=phase_name,
            result_file=valid_result_file
        )
        valid_loss_log.append(avg_valid_loss)


       # AUROC와 TPR@FPR=0.01%를 결합한 점수 계산
        combined_score = auroc_weight * valid_auc + (1 - auroc_weight) * valid_tpr

        # Save the best model based on combined score
        if combined_score > best_combined_score:
            best_combined_score = combined_score
            best_valid_auroc = valid_auc  # 최고 성능일 때 AUROC 값 저장
            best_valid_tpr = valid_tpr    # 최고 성능일 때 TPR 값 저장
            best_model_state = model.state_dict().copy()
            early_stop_counter = 0
            print(f"Run {run_id+1}: New best model found with combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {valid_auc:.4f} (weight: {auroc_weight:.2f})")
            print(f"  - TPR @ FPR=0.01%: {valid_tpr:.4f} (weight: {(1-auroc_weight):.2f})")
        else:
            early_stop_counter += 1
            print(f"Run {run_id+1}: No improvement. Early stop counter = {early_stop_counter}/{early_stopping_patience}")

        # Early stopping 조건 만족 시 중단
        if early_stop_counter >= early_stopping_patience:
            print(f"Run {run_id+1}: Early stopping triggered at epoch {epoch + 1}")
            break

    # 최고 성능 모델로 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Run {run_id+1}: Restored best model for final evaluation")


    print(f'----- Run {run_id+1}: Final Evaluation on Test Data -----')
    test_result_file = f"{result_dir}test_result_run{run_id+1}_{safe_model_name}_{method}.csv"
    test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, _ = eval_model(
        model, test_dataloader, device, num_cls,
        data_source=test_data,
        phase="test",
        result_file=test_result_file
    )
    loss_log_df = pd.DataFrame({'train_loss_log': train_loss_log,'valid_loss_log': valid_loss_log})
    loss_log_df.to_csv(f"{result_dir}loss_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8')


    return train_loss_log, valid_loss_log, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score

# seed 추가
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True) # 추가



def run_multiple_experiments(
    model_class,
    model_name,
    train_data,
    val_data,
    test_data,
    tokenizer,
    device,
    num_runs=3,
    learning_rate=3e-5,
    num_epochs=5,
    batch_size=16,
    accumulation_steps=2,
    num_cls=2,
    seed=42,
    auroc_weight=0.0  # AUROC 가중치 추가
):
    # seed=42 설정
    set_seed(seed)

    # 결과 저장을 위한 리스트
    all_test_metrics = []
    best_overall_score = 0.0  # 전체 실험에서 최고 결합 점수
    best_overall_auroc = 0.0  # 최고 모델의 AUROC
    best_overall_tpr = 0.0    # 최고 모델의 TPR
    best_overall_model_state = None
    best_run_id = -1

    # 데이터로더 생성
    train_dataset = data_preparing(train_data, tokenizer, batch_size=batch_size, shuffle=True, return_dataset=True)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    valid_dataloader = data_preparing(val_data, tokenizer, batch_size=batch_size, shuffle=False)
    test_dataloader = data_preparing(test_data, tokenizer, batch_size=batch_size, shuffle=False)

    print(f"\nUsing combined metric with AUROC weight: {auroc_weight:.2f}, TPR@FPR=0.01% weight: {(1-auroc_weight):.2f}")

    for run in range(num_runs):
        print(f"\n===== Starting Run {run+1}/{num_runs} =====")

        # 새로운 모델 인스턴스 생성
        model = AutoModelForSequenceClassification.from_pretrained("monologg/kobert", config=config)

        # 새로운 모델 인스턴스 생성
        model = model_class.from_pretrained(model_name, num_labels=num_cls)

        # 모델 학습 - 원본 데이터 전달
        _, _, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score= train_model(
            model=model,
            train_dataloader=train_dataloader,
            valid_dataloader=valid_dataloader,
            test_dataloader=test_dataloader,
            train_data=train_data,
            val_data=val_data,
            test_data=test_data,
            device=device,
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            accumulation_steps=accumulation_steps,
            num_cls=num_cls,
            run_id=run
        )

        # 테스트 메트릭 저장
        run_metrics = {
            'loss': test_loss,
            'accuracy': test_acc,
            'f1': test_f1,
            'precision': test_precision,
            'recall': test_recall,
            'auc': test_auc,
            'tpr_at_fpr_0001': test_tpr,
            'valid_auroc': best_valid_auroc,
            'valid_tpr': best_valid_tpr,
            'combined_score': best_combined_score
        }
        all_test_metrics.append(run_metrics)

        # Update best overall model based on combined score
        if best_combined_score > best_overall_score:
            best_overall_score = best_combined_score
            best_overall_auroc = best_valid_auroc
            best_overall_tpr = best_valid_tpr
            best_overall_model_state = best_model_state
            best_run_id = run

            # best model 저장
            torch.save(best_model_state, f"{result_dir}best_model_run_{run+1}_score_{best_combined_score:.4f}_auroc_{best_valid_auroc:.4f}_tpr_{best_valid_tpr:.4f}_{safe_model_name}_{method}.pt")
            print(f"New overall best model saved from run {run+1}")
            print(f"  - Combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {best_valid_auroc:.4f}")
            print(f"  - TPR @ FPR=0.01%: {best_valid_tpr:.4f}")

    # 평균 성능 계산
    avg_metrics = {metric: np.mean([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}
    std_metrics = {metric: np.std([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}

    print("\n===== Average Test Results Across All Runs =====")
    print(f"Avg Test Loss: {avg_metrics['loss']:.4f} ± {std_metrics['loss']:.4f}")
    print(f"Avg Test Accuracy: {avg_metrics['accuracy']:.4f} ± {std_metrics['accuracy']:.4f}")
    print(f"Avg Test F1 Score: {avg_metrics['f1']:.4f} ± {std_metrics['f1']:.4f}")
    print(f"Avg Test Precision: {avg_metrics['precision']:.4f} ± {std_metrics['precision']:.4f}")
    print(f"Avg Test Recall: {avg_metrics['recall']:.4f} ± {std_metrics['recall']:.4f}")
    print(f"Avg Test AUROC: {avg_metrics['auc']:.4f} ± {std_metrics['auc']:.4f}")
    print(f"Avg Test TPR @ FPR=0.01%: {avg_metrics['tpr_at_fpr_0001']:.4f} ± {std_metrics['tpr_at_fpr_0001']:.4f}")
    print(f"Avg Validation AUROC: {avg_metrics['valid_auroc']:.4f} ± {std_metrics['valid_auroc']:.4f}")
    print(f"Avg Validation TPR @ FPR=0.01%: {avg_metrics['valid_tpr']:.4f} ± {std_metrics['valid_tpr']:.4f}")
    print(f"Avg Combined Score: {avg_metrics['combined_score']:.4f} ± {std_metrics['combined_score']:.4f}")

    print(f"\n===== Best Model from Run {best_run_id+1} =====")
    print(f"Combined Score: {best_overall_score:.4f}")
    print(f"Best Validation AUROC: {best_overall_auroc:.4f}")
    print(f"Best Validation TPR: {best_overall_tpr:.4f}")
    print(f"Saved as: best_model_run_{best_run_id+1}_score_{best_overall_score:.4f}_auroc_{best_overall_auroc:.4f}_tpr_{best_overall_tpr:.4f}_{safe_model_name}_{method}.pt")

    # 평균 성능 저장
    summary_df = pd.DataFrame({
      'metric': list(avg_metrics.keys()),
      'mean': [avg_metrics[m] for m in avg_metrics],
      'std': [std_metrics[m] for m in std_metrics]
    })
    summary_file_path = os.path.join(result_dir, f"summary_metrics_{safe_model_name}_{method}.csv")
    summary_df.to_csv(summary_file_path, index=False, encoding='utf-8')

    # best overall model 저장
    torch.save(best_overall_model_state, f"{result_dir}best_model_overall_{safe_model_name}_{method}.pt")
    print(f"Best overall model saved as: best_model_overall_{safe_model_name}_{method}.pt")

    return all_test_metrics, avg_metrics, std_metrics, best_overall_model_state, best_run_id


def main():
   # 데이터 분할 (7:1.5:1.5)

    prompt_list = rs_sample['info.essay_prompt'].unique().tolist()
    train_prompts, temp_prompts = train_test_split(prompt_list,
                                                   test_size=0.3,
                                                   random_state=42,
                                                   )
    val_prompts, test_prompts = train_test_split(temp_prompts,
                                                 test_size=0.5,
                                                 random_state=42,
                                                 )
    train_data = rs_sample[rs_sample['info.essay_prompt'].isin(train_prompts)].reset_index(drop=True)
    val_data = rs_sample[rs_sample['info.essay_prompt'].isin(val_prompts)].reset_index(drop=True)
    test_data = rs_sample[rs_sample['info.essay_prompt'].isin(test_prompts)].reset_index(drop=True)



    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    auroc_weight = 0.0  # 이 값을 조정하여 AUROC의 중요도 설정 (0~1 사이 값)

    all_test_metrics, avg_metrics, std_metrics, best_model_state, best_run_id = run_multiple_experiments(
        model_class=model_class,
        model_name=model_name,
        train_data=train_data,
        val_data=val_data,
        test_data=test_data,
        tokenizer=tokenizer,
        device=device,
        num_runs=3,
        learning_rate=3e-5,
        num_epochs=5,
        batch_size=16,
        accumulation_steps=2,
        seed=42,
        auroc_weight=auroc_weight  # AUROC 가중치 전달
    )

    # Final evaluation with best model
    print("\n===== Final Evaluation with Best Overall Model =====")
    final_model = model_class.from_pretrained(model_name, num_labels=2)
    final_model.load_state_dict(best_model_state)
    final_model.to(device)

    test_dataloader = data_preparing(test_data, tokenizer, batch_size=16, shuffle=False)
    _, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr,_ = eval_model(final_model, test_dataloader, device)


start_time = time.time()

if __name__ == "__main__":
    main()

elapsed_time_sec = time.time() - start_time
elapsed_time_min = elapsed_time_sec / 60
print(f"소요 시간: {elapsed_time_min:.2f}분")


In [ ]:
# kobert 초기화

tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")
model_class = AutoModelForSequenceClassification
model_name = "monologg/kobert"

tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

The repository for monologg/kobert contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/monologg/kobert.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


tokenization_kobert.py:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/monologg/kobert:
- tokenization_kobert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_78b3253a26.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

In [ ]:
# baseline w L1
result_dir = f"/content/drive/My Drive/cdf/"

safe_model_name = model_name.split('/')[-1] if '/' in model_name else model_name
method = "L1"

class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.texts = dataframe['txt'].tolist()
        self.labels = dataframe['target'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': tokenized['input_ids'].squeeze(0),  # 차원 축소 유지
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)  # 데이터 타입 명확히 설정
        }


def data_preparing(data_src, tokenizer, batch_size=16, shuffle=True, return_dataset=False):
    dataset = TextDataset(data_src, tokenizer)

    if return_dataset:
        return dataset

    dataloader = DataLoader(
        dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )
    return dataloader


def eval_model(
    model,
    eval_dataloader,
    device,
    num_cls=2,
    data_source=None,
    phase=None,
    wrong_predictions_list=None,
    result_file=None  # 추가
):
    total_loss = 0.0
    total_count = 0
    all_preds, all_labels, all_probs = [], [], []

    model.eval()
    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss

            total_loss += loss.item() * batch['labels'].size(0)
            total_count += batch['labels'].size(0)

            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = torch.argmax(logits, dim=-1)
            labels = batch['labels']

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = total_loss / total_count
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs) if not np.all(all_probs == all_probs[0]) else 0.0

    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    tpr_at_fpr_0001 = np.interp(0.01 / 100, fpr, tpr) if len(fpr) > 0 else 0.0

    print(f"Loss: {avg_loss:.4f} | Acc: {accuracy:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | AUROC: {auc:.4f} | TPR@FPR=0.01%: {tpr_at_fpr_0001:.4f}")

    if data_source is not None and 'txt' in data_source:
        txt_data = [text for text in data_source['txt']]
    else:
        txt_data = []

    # 결과 저장
    if result_file is not None:
        df_result = pd.DataFrame({
            'label': all_labels,
            'txt' : txt_data ,
            'pred': all_preds,
            'prob': all_probs
        })
        df_result.to_csv(result_file, index=False, encoding='utf-8-sig')

    return avg_loss, accuracy, f1, precision, recall, auc, tpr_at_fpr_0001, (fpr, tpr)



def train_model(
    model,
    train_dataloader,
    valid_dataloader,
    test_dataloader,
    device,
    learning_rate=3e-5,
    num_epochs=5,
    accumulation_steps=2,
    num_cls=2,
    run_id=0,
    early_stopping_patience=2,
    train_data=None,
    val_data=None,
    test_data=None,
    auroc_weight=0.0,
    result_file=None
):
#    optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01) # L2 규제
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    model.to(device)

    print(f'-+-+-+ Run {run_id+1}: Initial Validation Metrics +-+-+-')
    eval_model(model, valid_dataloader, device, num_cls, data_source=val_data, phase="initial_val")
    print('-+-+-+ END OF INITIAL EVALUATION +-+-+-')

    train_loss_log = []
    valid_loss_log = []
    best_combined_score = 0.0  # 최고 결합 점수 초기화
    best_model_state = None
    early_stop_counter = 0
    all_wrong_predictions = []

    # 최적 성능 기록용
    best_valid_auroc = 0.0
    best_valid_tpr = 0.0

    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_count = 0
        phase_name = f"val_epoch{epoch+1}"
        valid_result_file = f"{result_dir}valid_result_run{run_id+1}_epoch{epoch+1}_{safe_model_name}_{method}.csv"
        model.train()  # 에폭 시작 시 그래디언트 초기화

        for step, batch in enumerate(train_dataloader):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss / accumulation_steps
           # L1 추가
            l1_lambda = 0.01
            l1_penalty = sum(torch.norm(param, 1) for param in model.parameters() if param.requires_grad)
            loss += (l1_lambda * l1_penalty) / accumulation_steps
            # L1 추가

            total_train_loss += loss.item()
            total_count += 1

            loss.backward()

            if (step + 1) % accumulation_steps == 0  or (step + 1) == len(train_dataloader) :
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()


        avg_train_loss = total_train_loss / total_count
        train_loss_log.append(avg_train_loss)
        if val_data is None:
            raise ValueError("Validation data is None.")
        if test_data is None:
            raise ValueError("Test data is None.")

        print(f'Run {run_id+1}, Epoch {epoch + 1} 성능 평가:')
        # 현재 에폭에서 검증 데이터 평가 및 틀린 예측 저장
        phase_name = f"val_epoch{epoch+1}"

        avg_valid_loss, valid_acc, valid_f1, valid_precision, valid_recall, valid_auc, valid_tpr, _ = eval_model(
            model, valid_dataloader, device, num_cls,
            data_source=val_data,
            phase=phase_name,
            result_file=valid_result_file
        )
        valid_loss_log.append(avg_valid_loss)


       # AUROC와 TPR@FPR=0.01%를 결합한 점수 계산
        combined_score = auroc_weight * valid_auc + (1 - auroc_weight) * valid_tpr

        # Save the best model based on combined score
        if combined_score > best_combined_score:
            best_combined_score = combined_score
            best_valid_auroc = valid_auc  # 최고 성능일 때 AUROC 값 저장
            best_valid_tpr = valid_tpr    # 최고 성능일 때 TPR 값 저장
            best_model_state = model.state_dict().copy()
            early_stop_counter = 0
            print(f"Run {run_id+1}: New best model found with combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {valid_auc:.4f} (weight: {auroc_weight:.2f})")
            print(f"  - TPR @ FPR=0.01%: {valid_tpr:.4f} (weight: {(1-auroc_weight):.2f})")
        else:
            early_stop_counter += 1
            print(f"Run {run_id+1}: No improvement. Early stop counter = {early_stop_counter}/{early_stopping_patience}")

        # Early stopping 조건 만족 시 중단
        if early_stop_counter >= early_stopping_patience:
            print(f"Run {run_id+1}: Early stopping triggered at epoch {epoch + 1}")
            break

    # 최고 성능 모델로 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Run {run_id+1}: Restored best model for final evaluation")


    print(f'----- Run {run_id+1}: Final Evaluation on Test Data -----')
    test_result_file = f"{result_dir}test_result_run{run_id+1}_{safe_model_name}_{method}.csv"
    test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, _ = eval_model(
        model, test_dataloader, device, num_cls,
        data_source=test_data,
        phase="test",
        result_file=test_result_file
    )
    loss_log_df = pd.DataFrame({'train_loss_log': train_loss_log,'valid_loss_log': valid_loss_log})
    loss_log_df.to_csv(f"{result_dir}loss_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8')


    return train_loss_log, valid_loss_log, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score

# seed 추가
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True) # 추가


def run_multiple_experiments(
    model_class,
    model_name,
    train_data,
    val_data,
    test_data,
    tokenizer,
    device,
    num_runs=3,
    learning_rate=3e-5,
    num_epochs=5,
    batch_size=16,
    accumulation_steps=2,
    num_cls=2,
    seed=42,
    auroc_weight=0.0  # AUROC 가중치 추가
):
    # seed=42 설정
    set_seed(seed)

    # 결과 저장을 위한 리스트
    all_test_metrics = []
    best_overall_score = 0.0  # 전체 실험에서 최고 결합 점수
    best_overall_auroc = 0.0  # 최고 모델의 AUROC
    best_overall_tpr = 0.0    # 최고 모델의 TPR
    best_overall_model_state = None
    best_run_id = -1

    # 데이터로더 생성
    train_dataset = data_preparing(train_data, tokenizer, batch_size=batch_size, shuffle=True, return_dataset=True)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    valid_dataloader = data_preparing(val_data, tokenizer, batch_size=batch_size, shuffle=False)
    test_dataloader = data_preparing(test_data, tokenizer, batch_size=batch_size, shuffle=False)

    print(f"\nUsing combined metric with AUROC weight: {auroc_weight:.2f}, TPR@FPR=0.01% weight: {(1-auroc_weight):.2f}")

    for run in range(num_runs):
        print(f"\n===== Starting Run {run+1}/{num_runs} =====")

        # 새로운 모델 인스턴스 생성
        model = model_class.from_pretrained(model_name, num_labels=num_cls)

        # 모델 학습 - 원본 데이터 전달
        _, _, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score= train_model(
            model=model,
            train_dataloader=train_dataloader,
            valid_dataloader=valid_dataloader,
            test_dataloader=test_dataloader,
            train_data=train_data,
            val_data=val_data,
            test_data=test_data,
            device=device,
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            accumulation_steps=accumulation_steps,
            num_cls=num_cls,
            run_id=run
        )

        # 테스트 메트릭 저장
        run_metrics = {
            'loss': test_loss,
            'accuracy': test_acc,
            'f1': test_f1,
            'precision': test_precision,
            'recall': test_recall,
            'auc': test_auc,
            'tpr_at_fpr_0001': test_tpr,
            'valid_auroc': best_valid_auroc,
            'valid_tpr': best_valid_tpr,
            'combined_score': best_combined_score
        }
        all_test_metrics.append(run_metrics)

        # Update best overall model based on combined score
        if best_combined_score > best_overall_score:
            best_overall_score = best_combined_score
            best_overall_auroc = best_valid_auroc
            best_overall_tpr = best_valid_tpr
            best_overall_model_state = best_model_state
            best_run_id = run

            # best model 저장
            torch.save(best_model_state, f"{result_dir}best_model_run_{run+1}_score_{best_combined_score:.4f}_auroc_{best_valid_auroc:.4f}_tpr_{best_valid_tpr:.4f}_{safe_model_name}_{method}.pt")
            print(f"New overall best model saved from run {run+1}")
            print(f"  - Combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {best_valid_auroc:.4f}")
            print(f"  - TPR @ FPR=0.01%: {best_valid_tpr:.4f}")

    # 평균 성능 계산
    avg_metrics = {metric: np.mean([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}
    std_metrics = {metric: np.std([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}

    print("\n===== Average Test Results Across All Runs =====")
    print(f"Avg Test Loss: {avg_metrics['loss']:.4f} ± {std_metrics['loss']:.4f}")
    print(f"Avg Test Accuracy: {avg_metrics['accuracy']:.4f} ± {std_metrics['accuracy']:.4f}")
    print(f"Avg Test F1 Score: {avg_metrics['f1']:.4f} ± {std_metrics['f1']:.4f}")
    print(f"Avg Test Precision: {avg_metrics['precision']:.4f} ± {std_metrics['precision']:.4f}")
    print(f"Avg Test Recall: {avg_metrics['recall']:.4f} ± {std_metrics['recall']:.4f}")
    print(f"Avg Test AUROC: {avg_metrics['auc']:.4f} ± {std_metrics['auc']:.4f}")
    print(f"Avg Test TPR @ FPR=0.01%: {avg_metrics['tpr_at_fpr_0001']:.4f} ± {std_metrics['tpr_at_fpr_0001']:.4f}")
    print(f"Avg Validation AUROC: {avg_metrics['valid_auroc']:.4f} ± {std_metrics['valid_auroc']:.4f}")
    print(f"Avg Validation TPR @ FPR=0.01%: {avg_metrics['valid_tpr']:.4f} ± {std_metrics['valid_tpr']:.4f}")
    print(f"Avg Combined Score: {avg_metrics['combined_score']:.4f} ± {std_metrics['combined_score']:.4f}")

    print(f"\n===== Best Model from Run {best_run_id+1} =====")
    print(f"Combined Score: {best_overall_score:.4f}")
    print(f"Best Validation AUROC: {best_overall_auroc:.4f}")
    print(f"Best Validation TPR: {best_overall_tpr:.4f}")
    print(f"Saved as: best_model_run_{best_run_id+1}_score_{best_overall_score:.4f}_auroc_{best_overall_auroc:.4f}_tpr_{best_overall_tpr:.4f}_{safe_model_name}_{method}.pt")

    # 평균 성능 저장
    summary_df = pd.DataFrame({
      'metric': list(avg_metrics.keys()),
      'mean': [avg_metrics[m] for m in avg_metrics],
      'std': [std_metrics[m] for m in std_metrics]
    })
    summary_file_path = os.path.join(result_dir, f"summary_metrics_{safe_model_name}_{method}.csv")
    summary_df.to_csv(summary_file_path, index=False, encoding='utf-8')

    # best overall model 저장
    torch.save(best_overall_model_state, f"{result_dir}best_model_overall_{safe_model_name}_{method}.pt")
    print(f"Best overall model saved as: best_model_overall_{safe_model_name}_{method}.pt")

    return all_test_metrics, avg_metrics, std_metrics, best_overall_model_state, best_run_id


def main():
   # 데이터 분할 (7:1.5:1.5)

    prompt_list = rs_sample['info.essay_prompt'].unique().tolist()
    train_prompts, temp_prompts = train_test_split(prompt_list,
                                                   test_size=0.3,
                                                   random_state=42,
                                                   )
    val_prompts, test_prompts = train_test_split(temp_prompts,
                                                 test_size=0.5,
                                                 random_state=42,
                                                 )
    train_data = rs_sample[rs_sample['info.essay_prompt'].isin(train_prompts)].reset_index(drop=True)
    val_data = rs_sample[rs_sample['info.essay_prompt'].isin(val_prompts)].reset_index(drop=True)
    test_data = rs_sample[rs_sample['info.essay_prompt'].isin(test_prompts)].reset_index(drop=True)



    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    auroc_weight = 0.0  # 이 값을 조정하여 AUROC의 중요도 설정 (0~1 사이 값)

    all_test_metrics, avg_metrics, std_metrics, best_model_state, best_run_id = run_multiple_experiments(
        model_class=model_class,
        model_name=model_name,
        train_data=train_data,
        val_data=val_data,
        test_data=test_data,
        tokenizer=tokenizer,
        device=device,
        num_runs=3,
        learning_rate=3e-5,
        num_epochs=5,
        batch_size=16,
        accumulation_steps=2,
        seed=42,
        auroc_weight=auroc_weight  # AUROC 가중치 전달
    )

    # Final evaluation with best model
    print("\n===== Final Evaluation with Best Overall Model =====")
    final_model = model_class.from_pretrained(model_name, num_labels=2)
    final_model.load_state_dict(best_model_state)
    final_model.to(device)

    test_dataloader = data_preparing(test_data, tokenizer, batch_size=16, shuffle=False)
    _, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr,_ = eval_model(final_model, test_dataloader, device)


start_time = time.time()

if __name__ == "__main__":
    main()

elapsed_time_sec = time.time() - start_time
elapsed_time_min = elapsed_time_sec / 60
print(f"소요 시간: {elapsed_time_min:.2f}분")


Using combined metric with AUROC weight: 0.00, TPR@FPR=0.01% weight: 1.00

===== Starting Run 1/3 =====


config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 1: Initial Validation Metrics +-+-+-
Loss: 0.7471 | Acc: 0.3912 | F1: 0.5622 | Precision: 0.3912 | Recall: 0.9993 | AUROC: 0.5788 | TPR@FPR=0.01%: 0.0007
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 1, Epoch 1 성능 평가:
Loss: 0.2136 | Acc: 0.9331 | F1: 0.9208 | Precision: 0.8573 | Recall: 0.9945 | AUROC: 0.9914 | TPR@FPR=0.01%: 0.1697
Run 1: New best model found with combined score: 0.1697
  - AUROC: 0.9914 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.1697 (weight: 1.00)
Run 1, Epoch 2 성능 평가:
Loss: 0.2715 | Acc: 0.9296 | F1: 0.9077 | Precision: 0.9315 | Recall: 0.8850 | AUROC: 0.9790 | TPR@FPR=0.01%: 0.0602
Run 1: No improvement. Early stop counter = 1/2
Run 1, Epoch 3 성능 평가:
Loss: 0.4497 | Acc: 0.8878 | F1: 0.8599 | Precision: 0.8406 | Recall: 0.8802 | AUROC: 0.9500 | TPR@FPR=0.01%: 0.0090
Run 1: No improvement. Early stop counter = 2/2
Run 1: Early stopping triggered at epoch 3
Run 1: Restored best model for final evaluation
----- Run 1: Final Evaluation on Test Data -----
Loss: 0.42

...

-+-+-+ Run 2: Initial Validation Metrics +-+-+-
Loss: 0.6658 | Acc: 0.6080 | F1: 0.0028 | Precision: 0.2857 | Recall: 0.0014 | AUROC: 0.6195 | TPR@FPR=0.01%: 0.0007
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 2, Epoch 1 성능 평가:
Loss: 0.3604 | Acc: 0.8163 | F1: 0.6954 | Precision: 0.9898 | Recall: 0.5360 | AUROC: 0.9827 | TPR@FPR=0.01%: 0.2382
Run 2: New best model found with combined score: 0.2382
  - AUROC: 0.9827 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.2382 (weight: 1.00)
Run 2, Epoch 2 성능 평가:
Loss: 0.4325 | Acc: 0.7960 | F1: 0.6496 | Precision: 0.9901 | Recall: 0.4834 | AUROC: 0.9860 | TPR@FPR=0.01%: 0.1607
Run 2: No improvement. Early stop counter = 1/2
Run 2, Epoch 3 성능 평가:
Loss: 0.1743 | Acc: 0.9558 | F1: 0.9423 | Precision: 0.9638 | Recall: 0.9217 | AUROC: 0.9925 | TPR@FPR=0.01%: 0.1586
Run 2: No improvement. Early stop counter = 2/2
Run 2: Early stopping triggered at epoch 3
Run 2: Restored best model for final evaluation
----- Run 2: Final Evaluation on Test Data -----
Loss: 0.16

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 3: Initial Validation Metrics +-+-+-
Loss: 0.7912 | Acc: 0.3912 | F1: 0.5624 | Precision: 0.3912 | Recall: 1.0000 | AUROC: 0.4558 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 3, Epoch 1 성능 평가:
Loss: 0.2431 | Acc: 0.9250 | F1: 0.9120 | Precision: 0.8426 | Recall: 0.9938 | AUROC: 0.9889 | TPR@FPR=0.01%: 0.1711
Run 3: New best model found with combined score: 0.1711
  - AUROC: 0.9889 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.1711 (weight: 1.00)
Run 3, Epoch 2 성능 평가:
Loss: 0.3034 | Acc: 0.8965 | F1: 0.8773 | Precision: 0.8180 | Recall: 0.9460 | AUROC: 0.9662 | TPR@FPR=0.01%: 0.0464
Run 3: No improvement. Early stop counter = 1/2
Run 3, Epoch 3 성능 평가:
Loss: 0.3413 | Acc: 0.8840 | F1: 0.8502 | Precision: 0.8593 | Recall: 0.8414 | AUROC: 0.9516 | TPR@FPR=0.01%: 0.0429
Run 3: No improvement. Early stop counter = 2/2
Run 3: Early stopping triggered at epoch 3
Run 3: Restored best model for final evaluation
----- Run 3: Final Evaluation on Test Data -----
Loss: 0.30

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loss: 0.1643 | Acc: 0.9588 | F1: 0.9490 | Precision: 0.9551 | Recall: 0.9429 | AUROC: 0.9929 | TPR@FPR=0.01%: 0.4950
소요 시간: 124.25분


In [ ]:
# baseline w L2
result_dir = f"/content/drive/My Drive/cdf/"

safe_model_name = model_name.split('/')[-1] if '/' in model_name else model_name
method = "L2"

class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.texts = dataframe['txt'].tolist()
        self.labels = dataframe['target'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        tokenized = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': tokenized['input_ids'].squeeze(0),  # 차원 축소 유지
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)  # 데이터 타입 명확히 설정
        }


def data_preparing(data_src, tokenizer, batch_size=16, shuffle=True, return_dataset=False):
    dataset = TextDataset(data_src, tokenizer)

    if return_dataset:
        return dataset

    dataloader = DataLoader(
        dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )
    return dataloader


def eval_model(
    model,
    eval_dataloader,
    device,
    num_cls=2,
    data_source=None,
    phase=None,
    wrong_predictions_list=None,
    result_file=None  # 추가
):
    total_loss = 0.0
    total_count = 0
    all_preds, all_labels, all_probs = [], [], []

    model.eval()
    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss

            total_loss += loss.item() * batch['labels'].size(0)
            total_count += batch['labels'].size(0)

            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)[:, 1]
            preds = torch.argmax(logits, dim=-1)
            labels = batch['labels']

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = total_loss / total_count
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs) if not np.all(all_probs == all_probs[0]) else 0.0

    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    tpr_at_fpr_0001 = np.interp(0.01 / 100, fpr, tpr) if len(fpr) > 0 else 0.0

    print(f"Loss: {avg_loss:.4f} | Acc: {accuracy:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | AUROC: {auc:.4f} | TPR@FPR=0.01%: {tpr_at_fpr_0001:.4f}")

    if data_source is not None and 'txt' in data_source:
        txt_data = [text for text in data_source['txt']]
    else:
        txt_data = []

    # 결과 저장
    if result_file is not None:
        df_result = pd.DataFrame({
            'label': all_labels,
            'txt' : txt_data ,
            'pred': all_preds,
            'prob': all_probs
        })
        df_result.to_csv(result_file, index=False, encoding='utf-8-sig')

    return avg_loss, accuracy, f1, precision, recall, auc, tpr_at_fpr_0001, (fpr, tpr)



def train_model(
    model,
    train_dataloader,
    valid_dataloader,
    test_dataloader,
    device,
    learning_rate=3e-5,
    num_epochs=5,
    accumulation_steps=2,
    num_cls=2,
    run_id=0,
    early_stopping_patience=2,
    train_data=None,
    val_data=None,
    test_data=None,
    auroc_weight=0.0,
    result_file=None
):
    optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=0.1) # L2 규제
#    optimizer = AdamW(model.parameters(), lr=learning_rate)
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    model.to(device)

    print(f'-+-+-+ Run {run_id+1}: Initial Validation Metrics +-+-+-')
    eval_model(model, valid_dataloader, device, num_cls, data_source=val_data, phase="initial_val")
    print('-+-+-+ END OF INITIAL EVALUATION +-+-+-')

    train_loss_log = []
    valid_loss_log = []
    best_combined_score = 0.0  # 최고 결합 점수 초기화
    best_model_state = None
    early_stop_counter = 0
    all_wrong_predictions = []

    # 최적 성능 기록용
    best_valid_auroc = 0.0
    best_valid_tpr = 0.0

    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_count = 0
        phase_name = f"val_epoch{epoch+1}"
        valid_result_file = f"{result_dir}valid_result_run{run_id+1}_epoch{epoch+1}_{safe_model_name}_{method}.csv"
        model.train()  # 에폭 시작 시 그래디언트 초기화

        for step, batch in enumerate(train_dataloader):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            loss = outputs.loss / accumulation_steps
           # L1 추가
            # l1_lambda = 0.01
            # l1_penalty = sum(torch.norm(param, 1) for param in model.parameters() if param.requires_grad)
            # loss += (l1_lambda * l1_penalty) / accumulation_steps
            # L1 추가

            total_train_loss += loss.item()
            total_count += 1

            loss.backward()

            if (step + 1) % accumulation_steps == 0  or (step + 1) == len(train_dataloader) :
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()


        avg_train_loss = total_train_loss / total_count
        train_loss_log.append(avg_train_loss)
        if val_data is None:
            raise ValueError("Validation data is None.")
        if test_data is None:
            raise ValueError("Test data is None.")

        print(f'Run {run_id+1}, Epoch {epoch + 1} 성능 평가:')
        # 현재 에폭에서 검증 데이터 평가 및 틀린 예측 저장
        phase_name = f"val_epoch{epoch+1}"

        avg_valid_loss, valid_acc, valid_f1, valid_precision, valid_recall, valid_auc, valid_tpr, _ = eval_model(
            model, valid_dataloader, device, num_cls,
            data_source=val_data,
            phase=phase_name,
            result_file=valid_result_file
        )
        valid_loss_log.append(avg_valid_loss)


       # AUROC와 TPR@FPR=0.01%를 결합한 점수 계산
        combined_score = auroc_weight * valid_auc + (1 - auroc_weight) * valid_tpr

        # Save the best model based on combined score
        if combined_score > best_combined_score:
            best_combined_score = combined_score
            best_valid_auroc = valid_auc  # 최고 성능일 때 AUROC 값 저장
            best_valid_tpr = valid_tpr    # 최고 성능일 때 TPR 값 저장
            best_model_state = model.state_dict().copy()
            early_stop_counter = 0
            print(f"Run {run_id+1}: New best model found with combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {valid_auc:.4f} (weight: {auroc_weight:.2f})")
            print(f"  - TPR @ FPR=0.01%: {valid_tpr:.4f} (weight: {(1-auroc_weight):.2f})")
        else:
            early_stop_counter += 1
            print(f"Run {run_id+1}: No improvement. Early stop counter = {early_stop_counter}/{early_stopping_patience}")

        # Early stopping 조건 만족 시 중단
        if early_stop_counter >= early_stopping_patience:
            print(f"Run {run_id+1}: Early stopping triggered at epoch {epoch + 1}")
            break

    # 최고 성능 모델로 복원
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"Run {run_id+1}: Restored best model for final evaluation")


    print(f'----- Run {run_id+1}: Final Evaluation on Test Data -----')
    test_result_file = f"{result_dir}test_result_run{run_id+1}_{safe_model_name}_{method}.csv"
    test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, _ = eval_model(
        model, test_dataloader, device, num_cls,
        data_source=test_data,
        phase="test",
        result_file=test_result_file
    )
    loss_log_df = pd.DataFrame({'train_loss_log': train_loss_log,'valid_loss_log': valid_loss_log})
    loss_log_df.to_csv(f"{result_dir}loss_log_{run_id+1}_{safe_model_name}_{method}.csv", index=False, encoding='utf-8')


    return train_loss_log, valid_loss_log, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score

# seed 추가
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True) # 추가


def run_multiple_experiments(
    model_class,
    model_name,
    train_data,
    val_data,
    test_data,
    tokenizer,
    device,
    num_runs=3,
    learning_rate=3e-5,
    num_epochs=5,
    batch_size=16,
    accumulation_steps=2,
    num_cls=2,
    seed=42,
    auroc_weight=0.0  # AUROC 가중치 추가
):
    # seed=42 설정
    set_seed(seed)

    # 결과 저장을 위한 리스트
    all_test_metrics = []
    best_overall_score = 0.0  # 전체 실험에서 최고 결합 점수
    best_overall_auroc = 0.0  # 최고 모델의 AUROC
    best_overall_tpr = 0.0    # 최고 모델의 TPR
    best_overall_model_state = None
    best_run_id = -1

    # 데이터로더 생성
    train_dataset = data_preparing(train_data, tokenizer, batch_size=batch_size, shuffle=True, return_dataset=True)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    valid_dataloader = data_preparing(val_data, tokenizer, batch_size=batch_size, shuffle=False)
    test_dataloader = data_preparing(test_data, tokenizer, batch_size=batch_size, shuffle=False)

    print(f"\nUsing combined metric with AUROC weight: {auroc_weight:.2f}, TPR@FPR=0.01% weight: {(1-auroc_weight):.2f}")

    for run in range(num_runs):
        print(f"\n===== Starting Run {run+1}/{num_runs} =====")

        # 새로운 모델 인스턴스 생성
        model = model_class.from_pretrained(model_name, num_labels=num_cls)

        # 모델 학습 - 원본 데이터 전달
        _, _, test_loss, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr, best_model_state, best_valid_auroc, best_valid_tpr, best_combined_score= train_model(
            model=model,
            train_dataloader=train_dataloader,
            valid_dataloader=valid_dataloader,
            test_dataloader=test_dataloader,
            train_data=train_data,
            val_data=val_data,
            test_data=test_data,
            device=device,
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            accumulation_steps=accumulation_steps,
            num_cls=num_cls,
            run_id=run
        )

        # 테스트 메트릭 저장
        run_metrics = {
            'loss': test_loss,
            'accuracy': test_acc,
            'f1': test_f1,
            'precision': test_precision,
            'recall': test_recall,
            'auc': test_auc,
            'tpr_at_fpr_0001': test_tpr,
            'valid_auroc': best_valid_auroc,
            'valid_tpr': best_valid_tpr,
            'combined_score': best_combined_score
        }
        all_test_metrics.append(run_metrics)

        # Update best overall model based on combined score
        if best_combined_score > best_overall_score:
            best_overall_score = best_combined_score
            best_overall_auroc = best_valid_auroc
            best_overall_tpr = best_valid_tpr
            best_overall_model_state = best_model_state
            best_run_id = run

            # best model 저장
            torch.save(best_model_state, f"{result_dir}best_model_run_{run+1}_score_{best_combined_score:.4f}_auroc_{best_valid_auroc:.4f}_tpr_{best_valid_tpr:.4f}_{safe_model_name}_{method}.pt")
            print(f"New overall best model saved from run {run+1}")
            print(f"  - Combined score: {best_combined_score:.4f}")
            print(f"  - AUROC: {best_valid_auroc:.4f}")
            print(f"  - TPR @ FPR=0.01%: {best_valid_tpr:.4f}")

    # 평균 성능 계산
    avg_metrics = {metric: np.mean([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}
    std_metrics = {metric: np.std([run_metric[metric] for run_metric in all_test_metrics]) for metric in all_test_metrics[0].keys()}

    print("\n===== Average Test Results Across All Runs =====")
    print(f"Avg Test Loss: {avg_metrics['loss']:.4f} ± {std_metrics['loss']:.4f}")
    print(f"Avg Test Accuracy: {avg_metrics['accuracy']:.4f} ± {std_metrics['accuracy']:.4f}")
    print(f"Avg Test F1 Score: {avg_metrics['f1']:.4f} ± {std_metrics['f1']:.4f}")
    print(f"Avg Test Precision: {avg_metrics['precision']:.4f} ± {std_metrics['precision']:.4f}")
    print(f"Avg Test Recall: {avg_metrics['recall']:.4f} ± {std_metrics['recall']:.4f}")
    print(f"Avg Test AUROC: {avg_metrics['auc']:.4f} ± {std_metrics['auc']:.4f}")
    print(f"Avg Test TPR @ FPR=0.01%: {avg_metrics['tpr_at_fpr_0001']:.4f} ± {std_metrics['tpr_at_fpr_0001']:.4f}")
    print(f"Avg Validation AUROC: {avg_metrics['valid_auroc']:.4f} ± {std_metrics['valid_auroc']:.4f}")
    print(f"Avg Validation TPR @ FPR=0.01%: {avg_metrics['valid_tpr']:.4f} ± {std_metrics['valid_tpr']:.4f}")
    print(f"Avg Combined Score: {avg_metrics['combined_score']:.4f} ± {std_metrics['combined_score']:.4f}")

    print(f"\n===== Best Model from Run {best_run_id+1} =====")
    print(f"Combined Score: {best_overall_score:.4f}")
    print(f"Best Validation AUROC: {best_overall_auroc:.4f}")
    print(f"Best Validation TPR: {best_overall_tpr:.4f}")
    print(f"Saved as: best_model_run_{best_run_id+1}_score_{best_overall_score:.4f}_auroc_{best_overall_auroc:.4f}_tpr_{best_overall_tpr:.4f}_{safe_model_name}_{method}.pt")

    # 평균 성능 저장
    summary_df = pd.DataFrame({
      'metric': list(avg_metrics.keys()),
      'mean': [avg_metrics[m] for m in avg_metrics],
      'std': [std_metrics[m] for m in std_metrics]
    })
    summary_file_path = os.path.join(result_dir, f"summary_metrics_{safe_model_name}_{method}.csv")
    summary_df.to_csv(summary_file_path, index=False, encoding='utf-8')

    # best overall model 저장
    torch.save(best_overall_model_state, f"{result_dir}best_model_overall_{safe_model_name}_{method}.pt")
    print(f"Best overall model saved as: best_model_overall_{safe_model_name}_{method}.pt")

    return all_test_metrics, avg_metrics, std_metrics, best_overall_model_state, best_run_id


def main():
   # 데이터 분할 (7:1.5:1.5)

    prompt_list = rs_sample['info.essay_prompt'].unique().tolist()
    train_prompts, temp_prompts = train_test_split(prompt_list,
                                                   test_size=0.3,
                                                   random_state=42,
                                                   )
    val_prompts, test_prompts = train_test_split(temp_prompts,
                                                 test_size=0.5,
                                                 random_state=42,
                                                 )
    train_data = rs_sample[rs_sample['info.essay_prompt'].isin(train_prompts)].reset_index(drop=True)
    val_data = rs_sample[rs_sample['info.essay_prompt'].isin(val_prompts)].reset_index(drop=True)
    test_data = rs_sample[rs_sample['info.essay_prompt'].isin(test_prompts)].reset_index(drop=True)



    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    auroc_weight = 0.0  # 이 값을 조정하여 AUROC의 중요도 설정 (0~1 사이 값)

    all_test_metrics, avg_metrics, std_metrics, best_model_state, best_run_id = run_multiple_experiments(
        model_class=model_class,
        model_name=model_name,
        train_data=train_data,
        val_data=val_data,
        test_data=test_data,
        tokenizer=tokenizer,
        device=device,
        num_runs=3,
        learning_rate=3e-5,
        num_epochs=5,
        batch_size=16,
        accumulation_steps=2,
        seed=42,
        auroc_weight=auroc_weight  # AUROC 가중치 전달
    )

    # Final evaluation with best model
    print("\n===== Final Evaluation with Best Overall Model =====")
    final_model = model_class.from_pretrained(model_name, num_labels=2)
    final_model.load_state_dict(best_model_state)
    final_model.to(device)

    test_dataloader = data_preparing(test_data, tokenizer, batch_size=16, shuffle=False)
    _, test_acc, test_f1, test_precision, test_recall, test_auc, test_tpr,_ = eval_model(final_model, test_dataloader, device)


start_time = time.time()

if __name__ == "__main__":
    main()

elapsed_time_sec = time.time() - start_time
elapsed_time_min = elapsed_time_sec / 60
print(f"소요 시간: {elapsed_time_min:.2f}분")


Using combined metric with AUROC weight: 0.00, TPR@FPR=0.01% weight: 1.00

===== Starting Run 1/3 =====


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 1: Initial Validation Metrics +-+-+-
Loss: 0.7471 | Acc: 0.3912 | F1: 0.5622 | Precision: 0.3912 | Recall: 0.9993 | AUROC: 0.5788 | TPR@FPR=0.01%: 0.0007
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 1, Epoch 1 성능 평가:
Loss: 0.0167 | Acc: 0.9943 | F1: 0.9928 | Precision: 0.9897 | Recall: 0.9958 | AUROC: 0.9998 | TPR@FPR=0.01%: 0.8864
Run 1: New best model found with combined score: 0.8864
  - AUROC: 0.9998 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.8864 (weight: 1.00)
Run 1, Epoch 2 성능 평가:
Loss: 0.0088 | Acc: 0.9973 | F1: 0.9965 | Precision: 0.9938 | Recall: 0.9993 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9432
Run 1: New best model found with combined score: 0.9432
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9432 (weight: 1.00)
Run 1, Epoch 3 성능 평가:
Loss: 0.0255 | Acc: 0.9924 | F1: 0.9904 | Precision: 0.9810 | Recall: 1.0000 | AUROC: 0.9993 | TPR@FPR=0.01%: 0.0876
Run 1: No improvement. Early stop counter = 1/2
Run 1, Epoch 4 성능 평가:
Loss: 0.0148 | Acc: 0.9970 | F1: 0.9962 | Pr

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 2: Initial Validation Metrics +-+-+-
Loss: 0.7041 | Acc: 0.4191 | F1: 0.5635 | Precision: 0.3991 | Recall: 0.9584 | AUROC: 0.5678 | TPR@FPR=0.01%: 0.0000
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 2, Epoch 1 성능 평가:
Loss: 0.0298 | Acc: 0.9873 | F1: 0.9840 | Precision: 0.9697 | Recall: 0.9986 | AUROC: 0.9998 | TPR@FPR=0.01%: 0.9647
Run 2: New best model found with combined score: 0.9647
  - AUROC: 0.9998 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9647 (weight: 1.00)
Run 2, Epoch 2 성능 평가:
Loss: 0.0197 | Acc: 0.9927 | F1: 0.9906 | Precision: 0.9972 | Recall: 0.9841 | AUROC: 0.9998 | TPR@FPR=0.01%: 0.7285
Run 2: No improvement. Early stop counter = 1/2
Run 2, Epoch 3 성능 평가:
Loss: 0.0064 | Acc: 0.9978 | F1: 0.9972 | Precision: 0.9959 | Recall: 0.9986 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9917
Run 2: New best model found with combined score: 0.9917
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9917 (weight: 1.00)
Run 2, Epoch 4 성능 평가:
Loss: 0.0323 | Acc: 0.9886 | F1: 0.9857 | Pr

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-+-+-+ Run 3: Initial Validation Metrics +-+-+-
Loss: 0.6855 | Acc: 0.5901 | F1: 0.5477 | Precision: 0.4819 | Recall: 0.6343 | AUROC: 0.6425 | TPR@FPR=0.01%: 0.0007
-+-+-+ END OF INITIAL EVALUATION +-+-+-
Run 3, Epoch 1 성능 평가:
Loss: 0.0521 | Acc: 0.9821 | F1: 0.9767 | Precision: 0.9964 | Recall: 0.9578 | AUROC: 0.9995 | TPR@FPR=0.01%: 0.8324
Run 3: New best model found with combined score: 0.8324
  - AUROC: 0.9995 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.8324 (weight: 1.00)
Run 3, Epoch 2 성능 평가:
Loss: 0.0153 | Acc: 0.9962 | F1: 0.9951 | Precision: 0.9986 | Recall: 0.9917 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9882
Run 3: New best model found with combined score: 0.9882
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.9882 (weight: 1.00)
Run 3, Epoch 3 성능 평가:
Loss: 0.0051 | Acc: 0.9981 | F1: 0.9976 | Precision: 0.9965 | Recall: 0.9986 | AUROC: 1.0000 | TPR@FPR=0.01%: 0.9903
Run 3: New best model found with combined score: 0.9903
  - AUROC: 1.0000 (weight: 0.00)
  - TPR @ FPR=0.01%: 0.99

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loss: 0.0486 | Acc: 0.9888 | F1: 0.9862 | Precision: 0.9905 | Recall: 0.9819 | AUROC: 0.9994 | TPR@FPR=0.01%: 0.8724
소요 시간: 187.83분
